In [1]:
###IMPORT MODULES AS APPROPRIATE###

In [2]:
import nltk; nltk.download('stopwords')

C:\Users\brady\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\brady\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.o

True

In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\Users\brady\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
C:\Users\brady\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
C:\Users\brady\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:31: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/relea

In [4]:
###INSERT STOP WORDS AS NEEDED###

In [5]:
#NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [6]:
###IMPORT DATA - THIS IS CURRENTLY UGLY BUT WHATEVER###

###try this later:
#import csv
#reviews = [row for row in csv.reader(open('reviews.csv'))]
#print reviews

In [7]:
df = [
"Individual animals frequently exhibit repeatable differences from other members of their population, differences now commonly referred to as 'animal personality'. Personality differences can arise, for example, from differences in permanent environmental effects-including parental and epigenetic contributors- and the effect of additive genetic variation. Although several studies have evaluated the heritability of behaviour, less is known about general patterns of heritability and additive genetic variation in animal personality. As overall variation in behaviour includes both the among-individual differences that reflect different personalities and temporary environmental effects, it is possible for personality to be largely genetically influenced even when heritability of behaviour per se is quite low. The relative contribution of additive genetic variation to personality variation can be estimated whenever both repeatability and heritability are estimated for the same data. Using published estimates to address this issue, we found that approximately 52% of animal personality variation was attributable to additive genetic variation. Thus, while the heritability of behaviour is often moderate or low, the heritability of personality is much higher. Our results therefore (i) demonstrate that genetic differences are likely to be a major contributor to variation in animal personality and (ii) support the phenotypic gambit: that evolutionary inferences drawn from repeatability estimates may often be justified.", 
"Environment has a critical role in the natural selection process for Darwinian evolution. The primary molecular component currently considered for neo-Darwinian evolution involves genetic alterations and random mutations that generate the phenotypic variation required for natural selection to act. The vast majority of environmental factors cannot directly alter DNA sequence. Epigenetic mechanisms directly regulate genetic processes and can be dramatically altered by environmental factors. Therefore, environmental epigenetics provides a molecular mechanism to directly alter phenotypic variation generationally. Lamarck proposed in 1802 the concept that environment can directly alter phenotype in a heritable manner. Environmental epigenetics and epigenetic transgenerational inheritance provide molecular mechanisms for this process. Therefore, environment can on a molecular level influence the phenotypic variation directly. The ability of environmental epigenetics to alter phenotypic and genotypic variation directly can significantly impact natural selection. Neo-Lamarckian concept can facilitate neo-Darwinian evolution. A unified theory of evolution is presented to describe the integration of environmental epigenetic and genetic aspects of evolution.", 
"DNA methylation contributes to gene and transcriptional regulation in eukaryotes, and therefore has been hypothesized to facilitate the evolution of plastic traits such as sociality in insects. However, DNA methylation is sparsely studied in insects. Therefore, we documented patterns of DNA methylation across a wide diversity of insects. We predicted that underlying enzymatic machinery is concordant with patterns of DNA methylation. Finally, given the suggestion that DNA methylation facilitated social evolution in Hymenoptera, we tested the hypothesis that the DNA methylation system will be associated with presence/absence of sociality among other insect orders. We found DNA methylation to be widespread, detected in all orders examined except Diptera (flies). Whole genome bisulfite sequencing showed that orders differed in levels of DNA methylation. Hymenopteran (ants, bees, wasps and sawflies) had some of the lowest levels, including several potential losses. Blattodea (cockroaches and termites) show all possible patterns, including a potential loss of DNA methylation in a eusocial species whereas solitary species had the highest levels. Species with DNA methylation do not always possess the typical enzymatic machinery. We identified a gene duplication event in the maintenance DNA methyltransferase 1 (DNMT1) that is shared by some Hymenoptera, and paralogs have experienced divergent, nonneutral evolution. This diversity and nonneutral evolution of underlying machinery suggests alternative DNA methylation pathways may exist. Phylogenetically corrected comparisons revealed no evidence that supports evolutionary association between sociality and DNA methylation. Future functional studies will be required to advance our understanding of DNA methylation in insects.", 
" ", 
"Phenotypic characteristics of animals can change independently from changes in the genetic code. These plastic phenotypic responses are important for population persistence in changing environments. Plasticity can be induced during early development, with persistent effects on adult phenotypes, and it can occur reversibly throughout life (acclimation). These manifestations of plasticity have been viewed as separate processes. Here we argue that developmental conditions not only change mean trait values but also modify the capacity for acclimation. Acclimation counteracts the potentially negative effects of phenotype-environment mismatches resulting from epigenetic modifications during early development. Developmental plasticity is therefore also beneficial when environmental conditions change within generations. Hence, the evolution of reversible acclimation can no longer be viewed as independent from developmental processes.", 
"An organism's phenotype can be influenced by maternal cues and directly perceived environmental cues, as well as by its genotype at polymorphic loci, which can be interpreted as a genetic cue. In fluctuating environments, natural selection favors organisms that efficiently integrate different sources of information about the likely success of phenotypic alternatives. In such situations, it can be beneficial to pass on maternal cues that offspring can respond to. A maternal cue could be based on environmental cues directly perceived by the mother but also partly on cues that were passed on by the grandmother. We have used a mathematical model to investigate how the passing of maternal cues and the integration of different sources of information evolve in response to qualitatively different kinds of temporal and spatial environmental fluctuations. The model shows that the passing of maternal cues and the transgenerational integration of sources of information readily evolve. Factors such as the degree of temporal autocorrelation, the predictive accuracy of different environmental cues, and the level of gene flow strongly influence the expression of adaptive maternal cues and the relative weights given to different sources of information. We outline the main features of the relation between the characteristics of environmental fluctuations and the adaptive systems of phenotype determination and compare these predictions with empirical studies on cue integration.", 
"Understanding how organisms cope with global change is a major scientific challenge. The molecular pathways underlying rapid adaptive phenotypic responses to global change remain poorly understood. Here, we highlight the relevance of two environment-sensitive molecular elements: transposable elements (TEs) and epigenetic components (ECs). We first outline the sensitivity of these elements to global change stressors and review how they interact with each other. We then propose an integrative molecular engine coupling TEs and ECs and allowing organisms to fine-tune phenotypes in a real-time fashion, adjust the production of phenotypic and genetic variation, and produce heritable phenotypes with different levels of transmission fidelity. We finally discuss the implications of this molecular engine in the context of global change.", 
"Existing insight suggests that maternal effects have a substantial impact on evolution, yet these predictions assume that maternal effects themselves are evolutionarily constant. Hence, it is poorly understood how natural selection shapes maternal effects in different ecological circumstances. To overcome this, the current study derives an evolutionary model of maternal effects in a quantitative genetics context. In constant environments, we show that maternal effects evolve to slight negative values that result in a reduction of the phenotypic variance (canalization). By contrast, in populations experiencing abrupt change, maternal effects transiently evolve to positive values for many generations, facilitating the transmission of beneficial maternal phenotypes to offspring. In periodically fluctuating environments, maternal effects evolve according to the autocorrelation between maternal and offspring environments, favoring positive maternal effects when change is slow, and negative maternal effects when change is rapid. Generally, the strongest maternal effects occur for traits that experience very strong selection and for which plasticity is severely constrained. By contrast, for traits experiencing weak selection, phenotypic plasticity enhances the evolutionary scope of maternal effects, although maternal effects attain much smaller values throughout. As weak selection is common, finding substantial maternal influences on offspring phenotypes may be more challenging than anticipated.", 
"As climate change challenges organismal fitness by creating a phenotype-environment mismatch, phenotypic plasticity generated by epigenetic mechanisms (e.g., DNA methylation) can provide a temporal buffer for genetic adaptation. Epigenetic mechanisms may be crucial for sessile benthic marine organisms, such as reef-building corals, where ocean acidification (OA) and warming reflect in strong negative responses. We tested the potential for scleractinian corals to exhibit phenotypic plasticity associated with a change in DNA methylation in response to OA. Clonal coral fragments of the environmentally sensitive Pocillopora damicornis and more environmentally robust Montipora capitata were exposed to fluctuating ambient pH (7.9-7.65) and low pH (7.6-7.35) conditions in common garden tanks for similar to 6 weeks. M. capitata responded weakly, or acclimated more quickly, to OA, with no difference in calcification, minimal separation of metabolomic profiles, and no change in DNA methylation between treatments. Conversely, P. damicornis exhibited diminished calcification at low pH, stronger separation in metabolomic profiles, and responsiveness of DNA methylation to treatment. Our data suggest corals differ in their temporal dynamics and sensitivity for environmentally triggered real-time epigenetic reprogramming. The generation of potentially heritable plasticity via environmental induction of DNA methylation provides an avenue for assisted evolution applications in corals under rapid climate change.", 
"Migration is essential for the reproduction and survival of many animals, yet little is understood about its underlying molecular mechanisms. We used the salmonid Oncorhynchus mykiss to gain mechanistic insight into smoltification, which is a morphological, physiological and behavioural transition undertaken by juveniles in preparation for seaward migration. O. mykiss is experimentally tractable and displays intra- and interpopulation variation in migration propensity. Migratory individuals can produce nonmigratory progeny and vice versa, indicating a high degree of phenotypic plasticity. One potential way that phenotypic plasticity might be linked to variation in migration-related life history tactics is through epigenetic regulation of gene expression. To explore this, we quantitatively measured genome-scale DNA methylation in fin tissue using reduced representation bisulphite sequencing of F-2 siblings produced from a cross between steelhead (migratory) and rainbow trout (nonmigratory) lines. We identified 57 differentially methylated regions (DMRs) between smolt and resident O. mykiss juveniles. DMRs were high in magnitude, with up to 62% differential methylation between life history types, and over half of the gene-associated DMRs were in transcriptional regulatory regions. Many of the DMRs encode proteins with activity relevant to migration-related transitions (e.g. circadian rhythm pathway, nervous system development, protein kinase activity). This study provides the first evidence of a relationship between epigenetic variation and life history divergence associated with migration-related traits in any species.", 
"While traits and trait plasticity are partly genetically based, investigating epigenetic mechanisms may provide more nuanced understanding of the mechanisms underlying response to environment. Using AFLP and methylation-sensitive AFLP, we tested the hypothesis that differentiation to habitats along natural salt marsh environmental gradients occurs at epigenetic, but not genetic loci in two salt marsh perennials. We detected significant genetic and epigenetic structure among populations and among subpopulations, but we found multilocus patterns of differentiation to habitat type only in epigenetic variation for both species. In addition, more epigenetic than genetic loci were correlated with habitat in both species. When we analysed genetic and epigenetic variation simultaneously with partial Mantel, we found no correlation between genetic variation and habitat and a significant correlation between epigenetic variation and habitat in Spartina alterniflora. In Borrichia frutescens, we found significant correlations between epigenetic and/or genetic variation and habitat in four of five populations when populations were analysed individually, but there was no significant correlation between genetic or epigenetic variation and habitat when analysed jointly across the five populations. These analyses suggest that epigenetic mechanisms are involved in the response to salt marsh habitats, but also that the relationships among genetic and epigenetic variation and habitat vary by species. Site-specific conditions may also cloud our ability to detect response in replicate populations with similar environmental gradients. Future studies analysing sequence data and the correlation between genetic variation and DNA methylation will be powerful to identify the contributions of genetic and epigenetic response to environmental gradients.", 
"Epigenetic variation is being integrated into our understanding of adaptation, yet we lack models on how epigenetic mutations affect evolution that includes de novo genetic change. We model the effects of epigenetic mutations on the dynamics and endpoints of adaptive walks-a process where a series of beneficial mutations move a population towards a fitness optimum. We use an individual-based model of an asexual population, where mutational effects are drawn from Fisher's geometric model. We find cases where epigenetic mutations speed adaptation or result in populations with higher fitness. However, we also find cases where they slow adaptation or result in populations with lower fitness. The effect of epigenetic mutations on adaptive walks depends crucially on their stability and fitness effects relative to genetic mutations, with small-effect epigenetic mutations generally speeding adaptation, and epigenetic mutations with the same fitness effects as genetic mutations slowing adaptation. Our work reveals a complex relationship between epigenetic mutations and natural selection and highlights the need for empirical data.", 
"Environmental stresses experienced by individual parents can influence offspring phenotypes in ways that enhance survival under similar conditions. Although such adaptive transgenerational plasticity is well documented, its transmission mechanisms are generally unknown. One possible mechanism is environmentally induced DNA methylation changes. We tested this hypothesis in the annual plant Polygonum persicaria, a species known to express adaptive transgenerational plasticity in response to parental drought stress. Replicate plants of 12 genetic lines (sampled from natural populations) were grown in dry versus moist soil. Their offspring were exposed to the demethylating agent zebularine or to control conditions during germination and then grown in dry soil. Under control germination conditions, the offspring of drought-stressed parents grew longer root systems and attained greater biomass compared with offspring of well-watered parents of the same genetic lines. Demethylation removed these adaptive developmental effects of parental drought, but did not significantly alter phenotypic expression in offspring of well-watered parents. The effect of demethylation on the expression of the parental drought effect varied among genetic lines. Differential seed provisioning did not contribute to the effect of parental drought on offspring phenotypes. These results demonstrate that DNA methylation can mediate adaptive, genotype-specific effects of parental stress on offspring phenotypes.", 
"The role of natural selection in the evolution of adaptive phenotypes has undergone constant probing by evolutionary biologists, employing both theoretical and empirical approaches. As Darwin noted, natural selection can act together with other processes, including random changes in the frequencies of phenotypic differences that are not under strong selection, and changes in the environment, which may reflect evolutionary changes in the organisms themselves. As understanding of genetics developed after 1900, the new genetic discoveries were incorporated into evolutionary biology. The resulting general principles were summarized by Julian Huxley in his 1942 book Evolution: the modern synthesis. Here, we examine how recent advances in genetics, developmental biology and molecular biology, including epigenetics, relate to today's understanding of the evolution of adaptations. We illustrate how careful genetic studies have repeatedly shown that apparently puzzling results in a wide diversity of organisms involve processes that are consistent with neo-Darwinism. They do not support important roles in adaptation for processes such as directed mutation or the inheritance of acquired characters, and therefore no radical revision of our understanding of the mechanism of adaptive evolution is needed.", 
"The process of biological invasion exposes a species to novel pressures, in terms of both the environments it encounters and the evolutionary consequences of range expansion. Several invaders have been shown to exhibit rapid evolutionary changes in response to those pressures, thus providing robust opportunities to clarify the processes at work during rapid phenotypic transitions. The accelerating pace of invasion of cane toads (Rhinella marina) in tropical Australia during its 80-year history has been well characterized at the phenotypic level, including common-garden experiments that demonstrate heritability of several dispersal-relevant traits. Individuals from the invasion front (and their progeny) show distinctive changes in morphology, physiology and behaviour that, in combination, result in far more rapid dispersal than is true of conspecifics from long-colonized areas. The extensive body of work on cane toad ecology enables us to place into context studies of the genetic basis of these traits. Our analyses of differential gene expression from toads from both ends of this invasion-history transect reveal substantial upregulation of many genes, notably those involved in metabolism and cellular repair. Clearly, then, the dramatically rapid phenotypic evolution of cane toads in Australia has been accompanied by substantial shifts in gene expression, suggesting that this system is well suited to investigating the genetic underpinnings of invasiveness.", 
"DNA methylation in plants affects transposon silencing, transcriptional regulation and thus phenotypic variation. One unanswered question is whether DNA methylation could be involved in local adaptation of plant populations to their environments. If methylation alters phenotypes to improve plant response to the environment, then methylation sites or the genes that affect them could be a target of natural selection. Using reduced-representation bisulphite sequencing (RRBS) data, we assessed whether climate is associated with variation in DNA methylation levels among 58 naturally occurring, and species-wide samples of valley oak (Quercus lobata) collected across climate gradients. We identified the genomic context of these variants referencing a new draft valley oak genome sequence. Methylation data were obtained for 341 107 cytosines, of which we deemed 57 488 as single-methylation variants (SMVs), found in the CG, CHG and CHH sequence contexts. Environmental association analyses revealed 43 specific SMVs that are significantly associated with any of four climate variables, the majority of which are associated with mean maximum temperature. The 43 climate-associated SMVs tend to occur in or near genes, several of which have known involvement in plant response to environment. Multivariate analyses show that climate and spatial variables explain more overall variance in CG-SMVs among individuals than in SNPs, CHG-SMVs or CHH-SMVs. Together, these results from natural oak populations provide initial evidence for a role of CG methylation in locally adaptive evolution or plasticity in plant response.", 
"Comparative genomics of social insects has been intensely pursued in recent years with the goal of providing insights into the evolution of social behaviour and its underlying genomic and epigenomic basis. However, the comparative approach has been hampered by a paucity of data on some of the most informative social forms (e.g. incipiently and primitively social) and taxa (especially members of the wasp family Vespidae) for studying social evolution. Here, we provide a draft genome of the primitively eusocial model insect Polistes dominula, accompanied by analysis of caste-related transcriptome and methylome sequence data for adult queens and workers. Polistes dominula possesses a fairly typical hymenopteran genome, but shows very low genomewide GC content and some evidence of reduced genome size. We found numerous caste-related differences in gene expression, with evidence that both conserved and novel genes are related to caste differences. Most strikingly, these -omics data reveal a major reduction in one of the major epigenetic mechanisms that has been previously suggested to be important for caste differences in social insects: DNA methylation. Along with a conspicuous loss of a key gene associated with environmentally responsive DNA methylation (the de novo DNA methyltransferase Dnmt3), these wasps have greatly reduced genomewide methylation to almost zero. In addition to providing a valuable resource for comparative analysis of social insect evolution, our integrative -omics data for this important behavioural and evolutionary model system call into question the general importance of DNA methylation in caste differences and evolution in social insects.", 
"Epidemiological research since the 1980s has highlighted the consequences of early life adversity, particularly during gestation and early infancy, for adult health (the Barker hypothesis). The fast-evolving field of molecular epigenetics is providing explanatory mechanisms concerning phenotypic plasticity in response to developmental stressors and the accumulation of disease risk throughout life. In addition, there is now evidence for the heritability of poor health across generations via epigenetic modifications. This research has the potential to invoke a paradigmatic shift in how we interpret factors such as growth insults and immune response in past skeletal remains. It demonstrates that health cannot be understood in terms of immediate environmental circumstances alone. Furthermore, it requires both a theoretical and practical re-evaluation of disease biographies and the life course more generally. Individual life courses can no longer be regarded as discrete, bounded, life histories, with clearly defined beginning and end points. If socioeconomic circumstances can have intergenerational effects, including disease susceptibility and growth stunting, then individual biographies should be viewed as nested or embedded within the lives of others. This commingling of life courses may prove problematic to unravel; nevertheless, this review aims to consider the potential consequences for bioarchaeological interpretations. These include a greater consideration of: the temporal power of human skeletons and a life course approach to past health; infant health and the implications for maternal well-being; and the impact of non-proximate stressors (e.g., early life and ancestral environments) on the presence of health indicators. (C) 2015 Wiley Periodicals, Inc.", 
"Organisms develop through multiple life stages that differ in environmental tolerances. The seasonal timing, or phenology, of life-stage transitions determines the environmental conditions to which each life stage is exposed and the length of time required to complete a generation. Both environmental and genetic factors contribute to phenological variation, yet predicting their combined effect on life cycles across a geographic range remains a challenge. We linked submodels of the plasticity of individual life stages to create an integrated model that predicts life-cycle phenology in complex environments. We parameterized the model for Arabidopsis thaliana and simulated life cycles in four locations. We compared multiple genotypes by varying two parameters associated with natural genetic variation in phenology: seed dormancy and floral repression. The model predicted variation in life cycles across locations that qualitatively matches observed natural phenology. Seed dormancy had larger effects on life-cycle length than floral repression, and results suggest that a genetic cline in dormancy maintains a life-cycle length of 1 year across the geographic range of this species. By integrating across life stages, this approach demonstrates how genetic variation in one transition can influence subsequent transitions and the geographic distribution of life cycles more generally.", 
"It has long been known that adaptive evolution can occur through genetic mutations in DNA sequence, but it is unclear whether adaptive evolution can occur through analogous epigenetic mechanisms, such as through DNA methylation. If epigenetic variation contributes directly to evolution, species under threat of disease, invasive competition, climate change or other stresses would have greater stores of variation from which to draw. We looked for evidence of natural selection acting on variably methylated DNA sites using population genomic analysis across three climatologically distinct populations of valley oaks. We found patterns of genetic and epigenetic differentiations that indicate local adaptation is operating on large portions of the oak genome. While CHG methyl polymorphisms are not playing a significant role and would make poor targets for natural selection, our findings suggest that CpG methyl polymorphisms as a whole are involved in local adaptation, either directly or through linkage to regions under selection.", 
"Much work has shown that the environment can induce non-genetic changes in phenotype that span multiple generations. Theory predicts that predictable environmental variation selects for both increased within-and across-generation responses. Yet, to the best of our knowledge, there are no empirical tests of this prediction. We explored the relationship between within-versus across-generation plasticity by evaluating the influence of predator cues on the life-history traits of Daphnia ambigua. We measured the duration of predator-induced transgenerational effects, determined when transgenerational responses are induced, and quantified the cues that activate transgenerational plasticity. We show that predator exposure during embryonic development causes earlier maturation and increased reproductive output. Such effects are detectable two generations removed from predator exposure and are similar in magnitude in response to exposure to cues emitted by injured conspecifics. Moreover, all experimental contexts and traits yielded a negative correlation between within-versus across-generation responses. That is, responses to predator cues within- and across-generations were opposite in sign and magnitude. Although many models address transgenerational plasticity, none of them explain this apparent negative relationship between within- and across-generation plasticities. Our results highlight the need to refine the theory of transgenerational plasticity.", 
"Environmentally induced phenotypic plasticity may be a critical component of response to changing environments. We examined local differentiation and adaptive phenotypic plasticity in response to elevated temperature in half-sib lines collected across an elevation gradient for the alpine herb, Wahlenbergia ceracea. Using Amplified Fragment Length Polymorphism (AFLP), we found low but significant genetic differentiation between low- and high-elevation seedlings, and seedlings originating from low elevations grew faster and showed stronger temperature responses (more plasticity) than those from medium and high elevations. Furthermore, plasticity was more often adaptive for plants of low-elevation origin and maladaptive for plants of high elevation. With methylation sensitive-AFLP (MS-AFLP), we revealed an increase in epigenetic variation in response to temperature in low-elevation seedlings. Although we did not find significant direct correlations between MS-AFLP loci and phenotypes, our results demonstrate that adaptive plasticity in temperature response to warming varies over fine spatial scales and suggest the involvement of epigenetic mechanisms in this response.", 
"Phenotypic plasticity is an important mechanism for populations to buffer themselves from environmental change. While it has long been appreciated that natural populations possess genetic variation in the extent of plasticity, a surge of recent evidence suggests that epigenetic variation could also play an important role in shaping phenotypic responses. Compared with genetic variation, epigenetic variation is more likely to have higher spontaneous rates of mutation and a more sensitive reaction to environmental inputs. In our review, we first provide an overview of recent studies on epigenetically encoded thermal plasticity in animals to illustrate environmentally-mediated epigenetic effects within and across generations. Second, we discuss the role of epigenetic effects during adaptation by exploring population epigenetics in natural animal populations. Finally, we evaluate the evolutionary potential of epigenetic variation depending on its autonomy from genetic variation and its transgenerational stability. Although many of the causal links between epigenetic variation and phenotypic plasticity remain elusive, new data has explored the role of epigenetic variation in facilitating evolution in natural populations. This recent progress in ecological epigenetics will be helpful for generating predictive models of the capacity of organisms to adapt to changing climates.", 
"Methylation of DNA cytosines affects whether transposons are silenced and genes are expressed, and is a major epigenetic mechanism whereby plants respond to environmental change. Analyses of methylation-sensitive amplification polymorphism (MS-AFLP or MSAP) have been often used to assess methyl-cytosine changes in response to stress treatments and, more recently, in ecological studies of wild plant populations. MSAP technique does not require a sequenced reference genome and provides many anonymous loci randomly distributed over the genome for which the methylation status can be ascertained. Scoring of MSAP data, however, is not straightforward, and efforts are still required to standardize this step to make use of the potential to distinguish between methylation at different nucleotide contexts. Furthermore, it is not known how accurately MSAP infers genome-wide cytosine methylation levels in plants. Here, we analyse the relationship between MSAP results and the percentage of global cytosine methylation in genomic DNA obtained by HPLC analysis. A screening of literature revealed that methylation of cytosines at cleavage sites assayed by MSAP was greater than genome-wide estimates obtained by HPLC, and percentages of methylation at different nucleotide contexts varied within and across species. Concurrent HPLC and MSAP analyses of DNA from 200 individuals of the perennial herb Helleborus foetidus confirmed that methyl-cytosine was more frequent in CCGG contexts than in the genome as a whole. In this species, global methylation was unrelated to methylation at the inner CG site. We suggest that global HPLC and context-specific MSAP methylation estimates provide complementary information whose combination can improve our current understanding of methylation-based epigenetic processes in nonmodel plants.", 
"Testing for conserved and novel mechanisms underlying phenotypic evolution requires a diversity of genomes available for comparison spanning multiple independent lineages. For example, complex social behavior in insects has been investigated primarily with eusocial lineages, nearly all of which are Hymenoptera. If conserved genomic influences on sociality do exist, we need data from a wider range of taxa that also vary in their levels of sociality. Here, we present the assembled and annotated genome of the subsocial beetle Nicrophorus vespilloides, a species long used to investigate evolutionary questions of complex social behavior. We used this genome to address two questions. First, do aspects of life history, such as using a carcass to breed, predict overlap in gene models more strongly than phylogeny? We found that the overlap in gene models was similar between N. vespilloides and all other insect groups regardless of life history. Second, like other insects with highly developed social behavior but unlike other beetles, does N. vespilloides have DNA methylation? We found strong evidence for an active DNA methylation system. The distribution of methylation was similar to other insects with exons having the most methylated C pGs. Methylation status appears highly conserved; 85% of the methylated genes in N. vespilloides are also methylated in the hymentopteran Nasonia vitripennis. The addition of this genome adds a coleopteran resource to answer questions about the evolution and mechanistic basis of sociality and to address questions about the potential role of methylation in social behavior.", 
"Telomeres are emerging as a biomarker for ageing and survival, and are likely important in shaping life-history trade-offs. In particular, telomere length with which one starts in life has been linked to lifelong survival, suggesting that early telomere dynamics are somehow related to life-history trajectories. This result highlights the importance of determining the extent to which telomere length is inherited, as a crucial factor determining early life telomere length. Given the scarcity of species for which telomere length inheritance has been studied, it is pressing to assess the generality of telomere length inheritance patterns. Further, information on how this pattern changes over the course of growth in individuals living under natural conditions should provide some insight on the extent to which environmental constraints also shape telomere dynamics. To fill this gap partly, we followed telomere inheritance in a population of king penguins (Aptenodytes patagonicus). We tested for paternal and maternal influence on chick initial telomere length (10 days old after hatching), and how these relationships changed with chick age (at 70, 200 and 300 days old). Based on a correlative approach, offspring telomere length was positively associated with maternal telomere length early in life (at 10 days old). However, this relationship was not significant at older ages. These data suggest that telomere length in birds is maternally inherited. Nonetheless, the influence of environmental conditions during growth remained an important factor shaping telomere length, as the maternal link disappeared with chicks' age.", 
"Transgenerational effects can buffer populations against environmental change, yet little is known about underlying mechanisms, their persistence or the influence of environmental cue timing. We investigated mitochondrial respiratory capacity (MRC) and gene expression of marine sticklebacks that experienced acute or developmental acclimation to simulated ocean warming (21 degrees C) across three generations. Previous work showed that acute acclimation of grandmothers to 21 degrees C led to lower (optimized) offspring MRCs. Here, developmental acclimation of mothers to 21 degrees C led to higher, but more efficient offspring MRCs. Offspring with a 21 degrees C x 17 degrees C grandmother-mother environment mismatch showed metabolic compensation: their MRCs were as low as offspring with a 17 degrees C thermal history across generations. Transcriptional analyses showed primarily maternal but also grandmaternal environment effects: genes involved in metabolism and mitochondrial protein biosynthesis were differentially expressed when mothers developed at 21 degrees C, whereas 21 degrees C grandmothers influenced genes involved in hemostasis and apoptosis. Genes involved in mitochondrial respiration all showed higher expression when mothers developed at 21 degrees and lower expression in the 21 degrees C x 17 degrees C group, matching the phenotypic pattern for MRCs. Our study links transcriptomics to physiology under climate change, and demonstrates that mechanisms underlying transgenerational effects persist across multiple generations with specific outcomes depending on acclimation type and environmental mismatch between generations.", 
"The effects from multigenerational exposures to engineered nanoparticles (ENPs) in their pristine and transformed states are currently unknown despite such exposures being an increasingly common scenario in natural environments. Here, we examine how exposure over 10 generations affects the sensitivity of the nematode Caenorhabditis elegans to pristine and sulfidized Ag ENPs and AgNO3. We also include populations that were initially exposed over six generations but kept unexposed for subsequent four generations to allow recovery from exposure. Toxicity of the different silver forms decreased in the order AgNO3, Ag ENPs and Ag2S ENPs. Continuous exposure to Ag ENPs and AgNO3 caused pronounced sensitization (approx. 10-fold) in the F2 generation, which was sustained until F10. This sensitization was less pronounced for Ag2S ENP exposures, indicating different toxicity mechanisms. Subtle changes in size and lifespan were also measured. In the recovery populations, the sensitivity to Ag ENPs and AgNO3 resulting from the initial multigenerational exposure persisted. Their response sensitivity for all endpoints was most closely related to the last ancestral exposed generation (F5), rather than unexposed controls. The mechanisms of transgenerational transfer of sensitivity are probably organized through the epigenome, and we encourage others to investigate such effects as a priority for mechanistic toxicology.", 
"Epigenetic variation has been proposed to contribute to the success of asexual plants, either as a contributor to phenotypic plasticity or by enabling transient adaptation via selection on transgenerationally stable, but reversible, epialleles. While recent studies in experimental plant populations have shown the potential for epigenetic mechanisms to contribute to adaptive phenotypes, it remains unknown whether heritable variation in ecologically relevant traits is at least partially epigenetically determined in natural populations. Here, we tested the hypothesis that DNA methylation variation contributes to heritable differences in flowering time within a single widespread apomictic clonal lineage of the common dandelion (Taraxacum officinale s. lat.). Apomictic clone members of the same apomictic lineage collected from different field sites showed heritable differences in flowering time, which was correlated with inherited differences in methylation-sensitive AFLP marker profiles. Differences in flowering between apomictic clone members were significantly reduced after in vivo demethylation using the DNA methyltransferase inhibitor zebularine. This synchronization of flowering times suggests that flowering time divergence within an apomictic lineage was mediated by differences in DNA methylation. While the underlying basis of the methylation polymorphism at functional flowering time-affecting loci remains to be demonstrated, our study shows that epigenetic variation contributes to heritable phenotypic divergence in ecologically relevant traits in natural plant populations. This result also suggests that epigenetic mechanisms can facilitate adaptive divergence within genetically uniform asexual lineages.", 
"Variation in resource availability commonly exerts strong effects on fitness-related traits in wild animals. However, we know little about the molecular mechanisms that mediate these effects, or about their persistence over time. To address these questions, we profiled genome-wide whole-blood DNA methylation levels in two sets of wild baboons: (i) 'wild-feeding' baboons that foraged naturally in a savanna environment and (ii) 'Lodge' baboons that had ready access to spatially concentrated human food scraps, resulting in high feeding efficiency and low daily travel distances. We identified 1014 sites (0.20% of sites tested) that were differentially methylated between wild-feeding and Lodge baboons, providing the first evidence that resource availability shapes the epigenome in a wild mammal. Differentially methylated sites tended to occur in contiguous stretches (i.e., in differentially methylated regions or DMRs), in promoters and enhancers, and near metabolism-related genes, supporting their functional importance in gene regulation. In agreement, reporter assay experiments confirmed that methylation at the largest identified DMR, located in the promoter of a key glycolysis-related gene, was sufficient to causally drive changes in gene expression. Intriguingly, all dispersing males carried a consistent epigenetic signature of their membership in a wild-feeding group, regardless of whether males dispersed into or out of this group as adults. Together, our findings support a role for DNA methylation in mediating ecological effects on phenotypic traits in the wild and emphasize the dynamic environmental sensitivity of DNA methylation levels across the life course.", 
"Epigenetic modifications are expected to occur at a much faster rate than genetic mutations, potentially causing isolated populations to stochastically drift apart, or if they are subjected to different selective regimes, to directionally diverge. A high level of genome-wide epigenetic divergence between individuals occupying distinct habitats is therefore predicted. Here, we introduce bisulfite-converted restriction site associated DNA sequencing (bsRADseq), an approach to quantify the level of DNA methylation differentiation across multiple individuals. This reduced representation method is flexible in the extent of DNA sequence interrogated. We showcase its applicability in three natural systems, each comprising individuals adapted to divergent environments: a diploid plant (Heliosperma, Caryophyllaceae), a tetraploid plant (Dactylorhiza, Orchidaceae) and an animal (Gasterosteusaculeatus, Gasterosteidae). We present a robust bioinformatic pipeline, combining tools for RAD locus assembly, SNP calling, bisulfite-converted read mapping and DNA methylation calling to analyse bsRADseq data with or without a reference genome. Importantly, our approach accurately distinguishes between SNPs and methylation polymorphism (SMPs). Although DNA methylation frequency between different positions of a genome varies widely, we find a surprisingly high consistency in the methylation profile between individuals thriving in divergent ecological conditions, particularly in Heliosperma. This constitutive stability points to significant molecular or developmental constraints acting on DNA methylation variation. Altogether, by combining the flexibility of RADseq with the accuracy of bisulfite sequencing in quantifying DNA methylation, the bsRADseq methodology and our bioinformatic pipeline open up the opportunity for genome-wide epigenetic investigations of evolutionary and ecological relevance in non-model species, independent of their genomic features.", 
"The growing knowledge about the influence of transposable elements (TEs) on (a) long-term genome and transcriptome evolution; (b) genomic, transcriptomic and epigenetic variation within populations; and (c) patterns of somatic genetic differences in individuals continues to spur the interest of evolutionary biologists in the role of TEs in adaptive evolution. As TEs can trigger a broad range of molecular variation in a population with potentially severe fitness and phenotypic consequences for individuals, different mechanisms evolved to keep TE activity in check, allowing for a dynamic interplay between the host, its TEs and the environment in evolution. Here, we review evidence for adaptive phenotypic changes associated with TEs and the basic molecular mechanisms by which the underlying genetic changes arise: (a) domestication, (b) exaptation, (c) host gene regulation, (d) TE-mediated formation of intronless gene copies-so-called retrogenes and (e) overall increased genome plasticity. Furthermore, we review and discuss how the stress-dependent incapacitation of defence mechanisms against the activity of TEs might facilitate adaptive responses to environmental challenges and how such mechanisms might be particularly relevant in species frequently facing novel environments, such as invasive, pathogenic or parasitic species.", 
"The decreasing cost and increasing scope and power of emerging genomic technologies are reshaping the field of molecular ecology. However, many modern genomic approaches (e.g., RAD-seq) require large amounts of high-quality template DNA. This poses a problem for an active branch of conservation biology: genetic monitoring using minimally invasive sampling (MIS) methods. Without handling or even observing an animal, MIS methods (e.g., collection of hair, skin, faeces) can provide genetic information on individuals or populations. Such samples typically yield low-quality and/or quantities of DNA, restricting the type of molecular methods that can be used. Despite this limitation, genetic monitoring using MIS is an effective tool for estimating population demographic parameters and monitoring genetic diversity in natural populations. Genetic monitoring is likely to become more important in the future as many natural populations are undergoing anthropogenically driven declines, which are unlikely to abate without intensive adaptive management efforts that often include MIS approaches. Here, we profile the expanding suite of genomic methods and platforms compatible with producing genotypes from MIS, considering factors such as development costs and error rates. We evaluate how powerful new approaches will enhance our ability to investigate questions typically answered using genetic monitoring, such as estimating abundance, genetic structure and relatedness. As the field is in a period of unusually rapid transition, we also highlight the importance of legacy data sets and recommend how to address the challenges of moving between traditional and next-generation genetic monitoring platforms. Finally, we consider how genetic monitoring could move beyond genotypes in the future. For example, assessing microbiomes or epigenetic markers could provide a greater understanding of the relationship between individuals and their environment.", 
"ObjectivesDifferences in DNA methylation have been associated with early life adversity, suggesting that alterations in methylation function as one pathway through which adverse early environments are biologically embedded. This study examined associations between exposure to institutional care, quantified as the proportion of time in institutional care at specified follow-up assessment ages, and DNA methylation status in two stress-related genes: FKBP5 and SLC6A4. Materials and MethodsWe analyzed data from the Bucharest Early Intervention Project, which is a prospective study in which children reared in institutional settings were randomly assigned (mean age 22 months) to either newly created foster care or care as usual (to remain in their current placement) and prospectively followed. A group of children from the same geographic area, with no history of institutionalized caregiving, were also recruited. DNA methylation status was determined in DNA extracted from buccal epithelial cells of children at age 12. ResultsAn inverse association was identified such that more time spent in institutional care was associated with lower DNA methylation at specific CpG sites within both genes. DiscussionThese results suggest a lasting impact of early severe social deprivation on methylation patterns in these genes, and contribute to a growing literature linking early adversity and epigenetic variation in children. Am J Phys Anthropol 161:84-93, 2016.. (c) 2016 Wiley Periodicals, Inc.", 
"DNA methylation is an epigenetic mark that plays an inadequately understood role in gene regulation, particularly in nonmodel species. Because it can be influenced by the environment, DNA methylation may contribute to the ability of organisms to acclimatize and adapt to environmental change. We evaluated the distribution of gene body methylation in reef-building corals, a group of organisms facing significant environmental threats. Gene body methylation in six species of corals was inferred from in silico transcriptome analysis of CpG O/E, an estimate of germline DNA methylation that is highly correlated with patterns of methylation enrichment. Consistent with what has been documented in most other invertebrates, all corals exhibited bimodal distributions of germline methylation suggestive of distinct fractions of genes with high and low levels of methylation. The hypermethylated fractions were enriched with genes with housekeeping functions, while genes with inducible functions were highly represented in the hypomethylated fractions. High transcript abundance was associated with intermediate levels of methylation. In three of the coral species, we found that genes differentially expressed in response to thermal stress and ocean acidification exhibited significantly lower levels of methylation. These results support a link between gene body hypomethylation and transcriptional plasticity that may point to a role of DNA methylation in the response of corals to environmental change.", 
"Recent studies show that epigenetic variation in the form of DNA methylation may serve as a substrate for selection. Theory suggests that heritable epigenetic marks that increase fitness should increase in frequency in a population, and these changes may result in novel morphology, behaviour, or physiology, and ultimately reproductive isolation. Therefore, epigenetic variation might provide the first substrate for selection during the course of evolutionary divergence. This hypothesis predicts that populations in the earliest stages of divergence will differentiate in their methylome prior to any genetic differentiation. While several studies have investigated natural epigenetic variation, empirical studies that test predictions about its role in speciation are surprisingly scarce. Here, we investigate DNA methylation variation using an isoschizomeric digest method, Methyl-Sensitive Amplified Polymorphism, across multiple stages of evolutionary divergence in natural populations of North American stream fishes. We show that epigenetic differentiation between methylomes is greater than genetic divergence among closely related populations across two river drainages. Additionally, we demonstrate that epigenetic divergence is a stronger predictor of the strength of behavioural reproductive isolation and suggest that changes in the methylome could influence the evolution of reproductive isolation between species. Our findings suggest a role for epigenetics not only in the initiation of divergence, but also in the maintenance of species boundaries over greater evolutionary timescales.", 
"The development of multicellular organisms involves a delicate interplay between genetic and environmental influences. It is often useful to think of developmental systems as integrating available sources of information about current conditions to produce organisms. Genes and inherited physiology provide cues, as does the state of the environment during development. The integration systems themselves are under genetic control and subject to Darwinian selection, so we expect them to evolve to produce organisms that fit well with current ecological (including social) conditions. We argue for the scientific value of this explicitly informational perspective by providing detailed examples of how it can elucidate taxonomically diverse phenomena. We also present a general framework for linking genetic and phenotypic variation from an informational perspective. This application of Darwinian logic at the organismal level can elucidate genetic influences on phenotypic variation in novel and counterintuitive ways.", 
"Organisms can adapt to variable environments by using environmental cues to modulate developmental gene expression. In principle, maternal influences can adaptively adjust offspring phenotype when early life and adult environments match, but they may be maladaptive when future environments are not predictable. One of the best-studied 'maternal effects' is through modification of the offspring's hypothalamic-pituitary-adrenal (HPA) axis, the neuroendocrine system that controls responses to stress. In addition to the direct transfer of glucocorticoids from mother to offspring, offspring HPA function and other phenotypes can also be affected by epigenetic modifications like DNA methylation of the glucocorticoid receptor promoter. Here we examine how among-year variation in rainfall is related to DNA methylation during development and fitness in adulthood in the superb starling (Lamprotornis superbus), which lives in a climatically unpredictable environment where early life and adult environments are unlikely to match. We found that DNA methylation in the putative promoter of the glucocorticoid receptor gene is reduced in chicks - particularly in males - born following drier prebreeding periods. Additionally, DNA methylation is lower in males that become breeders than those that never breed. However, there is no relationship in females between DNA methylation and the likelihood of dispersing from the natal group to breed elsewhere. These results suggest that early life conditions may positively affect fitness in a sex-specific manner through chemical modification of an HPA-associated gene. This study is the first to show that epigenetic modifications during early life may influence the fitness of free-living organisms adapted to unpredictable environments.", 
"Personality traits are heritable and respond to natural selection, but are at the same time influenced by the ontogenetic environment. Epigenetic effects, such as DNA methylation, have been proposed as a key mechanism to control personality variation. However, to date little is known about the contribution of epigenetic effects to natural variation in behaviour. Here, we show that great tit (Parus major) lines artificially selected for divergent exploratory behaviour for four generations differ in their DNA methylation levels at the dopamine receptor D4 (DRD4) gene. This D4 receptor is statistically associated with personality traits in both humans and nonhuman animals, including the great tit. Previous work in this songbird failed to detect functional genetic polymorphisms within DRD4 that could account for the gene-trait association. However, our observation supports the idea that DRD4 is functionally involved in exploratory behaviour but that its effects are mediated by DNA methylation. While the exact mechanism underlying the transgenerational consistency of DRD4 methylation remains to be elucidated, this study shows that epigenetic mechanisms are involved in shaping natural variation in personality traits. We outline how this first finding provides a basis for investigating the epigenetic contribution to personality traits in natural systems and its subsequent role for understanding the ecology and evolution of behavioural consistency.", 
"Gene sequence similarity due to shared ancestry after a duplication event, that is paralogy, complicates the assessment of genetic variation, as sequences originating from paralogs can be difficult to distinguish. These confounded sequences are often removed prior to further analyses, leaving the underlying loci uncharacterized. Salmonids have only partially rediploidized subsequent to a whole-genome duplication; residual tetrasomic inheritance has been observed in males. We present a maximum-likelihood-based method to resolve confounded paralogous loci by observing the segregation of alleles in gynogenetic haploid offspring and demonstrate its effectiveness by constructing two linkage maps for chum salmon (Oncorhynchus keta), with and without these newly resolved loci. We find that the resolved paralogous loci are not randomly distributed across the genome. A majority are clustered in expanded subtelomeric regions of 14 linkage groups, suggesting a significant fraction of the chum salmon genome may be missed by the exclusion of paralogous loci. Transposable elements have been proposed as drivers of genome evolution and, in salmonids, may have an important role in the rediploidization process by driving differentiation between homeologous chromosomes. Consistent with that hypothesis, we find a reduced fraction of transposable element annotations among paralogous loci, and these loci predominately occur in the genomic regions that lag in the rediploidization process.", 
"Heat-shock proteins (Hsps) and their cognates are primary mitigators of cell stress. With increasingly severe impacts of climate change and other human modifications of the biosphere, the ability of the heat-shock system to affect evolutionary fitness in environments outside the laboratory and to evolve in response is topic of growing importance. Since the last major reviews, several advances have occurred. First, demonstrations of the heat-shock response outside the laboratory now include many additional taxa and environments. Many of these demonstrations are only correlative, however. More importantly, technical advances in omic quantification of nucleic acids and proteins, genomewide association analysis, and manipulation of genes and their expression have enabled the field to move beyond correlation. Several consequent advances are already evident: The pathway from heat-shock gene expression to stress tolerance in nature can be extremely complex, mediated through multiple biological processes and systems, and even multiple species. The underlying genes are more numerous, diverse and variable than previously appreciated, especially with respect to their regulatory variation and epigenetic changes. The impacts and limitations (e.g., due to trade-offs) of natural selection on these genes have become more obvious and better established. At last, as evolutionary capacitors, Hsps may have distinctive impacts on the evolution of other genes and ecological consequences.", 
"Resetting of epigenetic marks, such as DNA methylation, in germ cells or early embryos is not always complete. Epigenetic states may therefore persist, decay or accumulate across generations. In spite of mounting empirical evidence for incomplete resetting, it is currently poorly understood whether it simply reflects stochastic noise or plays an adaptive role in phenotype determination. Here, we use a simple model to show that incomplete resetting can be adaptive in heterogeneous environments. Transmission of acquired epigenetic states prevents mismatched phenotypes when the environment changes infrequently relative to generation time and when maternal and environmental cues are unreliable. We discuss how these results may help to interpret the emerging data on transgenerational epigenetic inheritance in plants and animals.", 
"Genomes of invertebrates and vertebrates exhibit highly divergent patterns of DNA methylation. Invertebrate genomes tend to be sparsely methylated, and DNA methylation is mostly targeted to a subset of transcription units (gene bodies). In a drastic contrast, vertebrate genomes are generally globally and heavily methylated, punctuated by the limited local hypo-methylation of putative regulatory regions such as promoters. These genomic differences also translate into functional differences in DNA methylation and gene regulation. Although promoter DNA methylation is an important regulatory component of vertebrate gene expression, its role in invertebrate gene regulation has been little explored. Instead, gene body DNA methylation is associated with expression of invertebrate genes. However, the evolutionary steps leading to the differentiation of invertebrate and vertebrate genomic DNA methylation remain unresolved. Here we analyzed experimentally determined DNA methylation maps of several species across the invertebrate-vertebrate boundary, to elucidate how vertebrate gene methylation has evolved. We show that, in contrast to the prevailing idea, a substantial number of promoters in an invertebrate basal chordate Ciona intestinalis are methylated. Moreover, gene expression data indicate significant, epigenomic context-dependent associations between promoter methylation and expression in C. intestinalis. However, there is no evidence that promoter methylation in invertebrate chordate has been evolutionarily maintained across the invertebrate-vertebrate boundary. Rather, body-methylated invertebrate genes preferentially obtain hypo-methylated promoters among vertebrates. Conversely, promoter methylation is preferentially found in lineage- and tissue-specific vertebrate genes. These results provide important insights into the evolutionary origin of epigenetic regulation of vertebrate gene expression.", 
"Animal and plant centromeres are embedded in repetitive satellite DNA, but are thought to be epigenetically specified. To define genetic characteristics of centromeres, we surveyed satellite DNA from diverse eukaryotes and identified variation in <10-bp dyad symmetries predicted to adopt non-B-form conformations. Organisms lacking centromeric dyad symmetries had binding sites for sequence-specific DNA-binding proteins with DNA-bending activity. For example, human and mouse centromeres are depleted for dyad symmetries, but are enriched for non-B-form DNA and are associated with binding sites for the conserved DNA-binding protein CENP-B, which is required for artificial centromere function but is paradoxically nonessential. We also detected dyad symmetries and predicted non-B-form DNA structures at neocentromeres, which form at ectopic loci. We propose that centromeres form at non-B-form DNA because of dyad symmetries or are strengthened by sequence-specific DNA binding proteins. This may resolve the CENP-B paradox and provide a general basis for centromere specification.", 
"Epigenetic mechanisms such as changes in DNA methylation have the potential to affect the resilience of species to climate change, but little is known about the response of the methylome to changes in environmental temperature in animals. Using reduced representation bisulfite sequencing, we assessed the effects of development temperature and adult acclimation temperature on DNA methylation levels in threespine stickleback (Gasterosteus aculeatus). Across all treatments, we identified 2130 differentially methylated cytosines distributed across the genome. Both increases and decreases in temperature during development and with thermal acclimation in adults increased global DNA methylation levels. Approximately 25% of the differentially methylated regions (DMRs) responded to both developmental temperature and adult thermal acclimation, and 50 DMRs were common to all treatments, demonstrating a core response of the epigenome to thermal change at multiple time scales. We also identified differentially methylated loci that were specific to a particular developmental or adult thermal response, which could facilitate the accumulation of epigenetic variation between natural populations that experience different thermal regimes. These data demonstrate that thermal history can have long-lasting effects on the epigenome, highlighting the role of epigenetic modifications in the response to temperature change across multiple time scales.", 
"The diversity of sex chromosomes among amniotes is the product of independent evolution of different systems in different lineages, defined by novel sex-determining genes. Convergent evolution is very common, suggesting that some genes are particularly adept at taking on a sex-determining role. Comparative gene mapping, and more recently whole genome sequencing, have now turned up other surprising relationships; different regions of the amniote genome that have become sex determining in some taxa seem to share synteny, or share sequence, in others. Is this, after all, evidence that these regions were once linked in a super-sex chromosome that underwent multiple fission in different ways in different amniote lineages? Or does it signify that special properties of sex chromosomes (paucity of active genes, low recombination, epigenetic regulation to achieve dosage compensation) predispose particular chromosomes to a sex-determining role?", 
" ", 
"Epigenetic changes can occur due to extracellular environmental conditions. Consequently, epigenetic mechanisms can play an intermediate role to translate environmental signals to intracellular changes. Such a role might be particularly important in plants, which often show strong local adaptation and have the potential for heritable epigenetic states. However, little is currently known about the role of epigenetic variation in the ecological mechanisms of adaptation. Here, we used multivariate redundancy analyses to examine genomewide associations between DNA methylation polymorphisms and climate variation in two independent panels of Arabidopsis accessions, including 122 Eurasian accessions as well as in a regional panel of 148 accessions in Sweden. At the single-nucleotide methylation level, climate and space (geographic spatial structure) explain small yet significant amount of variation in both panels. On the other hand, when viewed in a context of genomic clusters of methylated and unmethylated cytosines, climate and space variables explain much greater amounts of variation in DNA methylation than those explained by variation at the single-nucleotide level. We found that the single-nucleotide methylation polymorphisms with the strongest associations with climate were enriched in transposable elements and in potentially RNA-directed methylation contexts. When viewed in the context of genomic clusters, variation of DNA methylation at different sequence contexts exhibit distinctive segregation along different axes of variation in the redundancy analyses. Genomewide methylation showed much stronger associations with climate within the regional panel (Sweden) compared to the global (Eurasia). Together, these findings indicate that genetic and epigenetic variation across the genome may play a role in response to climate conditions and local adaptation.", 
"The three-spined stickleback (Gasterosteus aculeatus) represents a convenient model to study microevolution-adaptation to a freshwater environment. Although genetic adaptations to freshwater environments are well-studied, epigenetic adaptations have attracted little attention. In this work, we investigated the role of DNA methylation in the adaptation of the marine stickleback population to freshwater conditions. DNA methylation profiling was performed in marine and freshwater populations of sticklebacks, as well as in marine sticklebacks placed into a freshwater environment and freshwater sticklebacks placed into seawater. We showed that the DNA methylation profile after placing a marine stickleback into fresh water partially converged to that of a freshwater stickleback. For six genes including ATP4A ion pump and NELL1, believed to be involved in skeletal ossification, we demonstrated similar changes in DNA methylation in both evolutionary and short-term adaptation. This suggested that an immediate epigenetic response to freshwater conditions can be maintained in freshwater population. Interestingly, we observed enhanced epigenetic plasticity in freshwater sticklebacks that may serve as a compensatory regulatory mechanism for the lack of genetic variation in the freshwater population. For the first time, we demonstrated that genes encoding ion channels KCND3, CACNA1FB, and ATP4A were differentially methylated between the marine and the freshwater populations. Other genes encoding ion channels were previously reported to be under selection in freshwater populations. Nevertheless, the genes that harbor genetic and epigenetic changes were not the same, suggesting that epigenetic adaptation is a complementary mechanism to selection of genetic variants favorable for freshwater environment.", 
"Epigenetic modifications, such as DNA methylation or histone modifications, can be transmitted between cellular or organismal generations. However, there are no experiments measuring their role in adaptation, so here we use experimental evolution to investigate how epigenetic variation can contribute to adaptation. We manipulated DNA methylation and histone acetylation in the unicellular green alga Chlamydomonas reinhardtii both genetically and chemically to change the amount of epigenetic variation generated or transmitted in adapting populations in three different environments (salt stress, phosphate starvation, and high CO2) for two hundred asexual generations. We find that reducing the amount of epigenetic variation available to populations can reduce adaptation in environments where it otherwise happens. From genomic and epigenomic sequences from a subset of the populations, we see changes in methylation patterns between the evolved populations over-represented in some functional categories of genes, which is consistent with some of these differences being adaptive. Based on whole genome sequencing of evolved clones, the majority of DNA methylation changes do not appear to be linked to cis-acting genetic mutations. Our results show that transgenerational epigenetic effects play a role in adaptive evolution, and suggest that the relationship between changes in methylation patterns and differences in evolutionary outcomes, at least for quantitative traits such as cell division rates, is complex.", 
"Hybridization has many and varied impacts on the process of speciation. Hybridization may slow or reverse differentiation by allowing gene flow and recombination. It may accelerate speciation via adaptive introgression or cause near-instantaneous speciation by allopolyploidization. It may have multiple effects at different stages and in different spatial contexts within a single speciation event. We offer a perspective on the context and evolutionary significance of hybridization during speciation, highlighting issues of current interest and debate. In secondary contact zones, it is uncertain if barriers to gene flow will be strengthened or broken down due to recombination and gene flow. Theory and empirical evidence suggest the latter is more likely, except within and around strongly selected genomic regions. Hybridization may contribute to speciation through the formation of new hybrid taxa, whereas introgression of a few loci may promote adaptive divergence and so facilitate speciation. Gene regulatory networks, epigenetic effects and the evolution of selfish genetic material in the genome suggest that the DobzhanskyMuller model of hybrid incompatibilities requires a broader interpretation. Finally, although the incidence of reinforcement remains uncertain, this and other interactions in areas of sympatry may have knock-on effects on speciation both within and outside regions of hybridization.", 
"Experimental evolution is the study of evolutionary processes occurring in experimental populations in response to conditions imposed by the experimenter. This research approach is increasingly used to study adaptation, estimate evolutionary parameters, and test diverse evolutionary hypotheses. Long applied in vaccine development, experimental evolution also finds new applications in biotechnology. Recent technological developments provide a path towards detailed understanding of the genomic and molecular basis of experimental evolutionary change, while new findings raise new questions that can be addressed with this approach. However, experimental evolution has important limitations, and the interpretation of results is subject to caveats resulting from small population sizes, limited timescales, the simplified nature of laboratory environments, and, in some cases, the potential to misinterpret the selective forces and other processes at work.", 
"Epigenetics investigates heritable changes in gene expression that occur without changes in DNA sequence. Several epigenetic mechanisms, including DNA methylation and histone modifications, can change genome function under exogenous influence. We review current evidence indicating that epigenetic alterations mediate effects caused by exposure to environmental toxicants. Results obtained from animal models indicate that in utero or early-life environmental exposures produce effects that can be inherited transgenerationally and are accompanied by epigenetic alterations. The search for human equivalents of the epigenetic mechanisms identified in animal models is under way. Recent investigations have identified a number of environmental toxicants that cause altered methylation of human repetitive elements or genes. Some exposures can alter epigenetic states and the same and/or similar epigenetic alterations can be found in patients with the disease of concern. On the basis of current evidence, we propose possible models for the interplay between environmental exposures and the human epigenome. Several investigations have examined the relationship between exposure to environmental chemicals and epigenetics, and have identified toxicants that modify epigenetic states. Whether environmental exposures have transgenerational epigenetic effects in humans remains to be elucidated. In spite of the current limitations, available evidence supports the concept that epigenetics holds substantial potential for furthering our understanding of the molecular mechanisms of environmental toxicants, as well as for predicting health-related risks due to conditions of environmental exposure and individual susceptibility. Heredity (2010) 105, 105-112; doi:10.1038/hdy.2010.2; published online 24 February 2010", 
"Transcription, translation and subsequent protein modification represent the transfer of genetic information from the archival copy of DNA to the short-lived messenger RNA, usually with subsequent production of protein. Although all cells in an organism contain essentially the same DNA, cell types and functions differ because of qualitative and quantitative differences in their gene expression. Thus, control of gene expression is at the heart of differentiation and development. Epigenetic processes, including DNA methylation, histone modification and various RNA-mediated processes, are thought to influence gene expression chiefly at the level of transcription; however, other steps in the process (for example, translation) may also be regulated epigenetically. The following paper will outline the role epigenetics is believed to have in influencing gene expression. Heredity (2010) 105, 4-13; doi:10.1038/hdy.2010.54; published online 12 May 2010", 
"During the past decade there has been a tremendous resurgence of interest in polyploidy that has in large part been stimulated by the development of increasingly powerful genetic and genomic tools. The result has been numerous new insights into the genomic and genetic consequences of polyploidy. The plethora of new discoveries has dramatically reshaped traditional views and concomitantly revealed that polyploidy is a highly dynamic and ubiquitous process. These recent advances in our understanding of polyploidy have stimulated numerous reviews, most focused on the various genetic, epigenetic, and genomic consequences of polyploid evolution. Whereas genetic and genomic attributes of polyploidization have received considerable attention, other crucial areas of polyploid evolution have received much less (e.g., ecology, pollination biology, physiology). The focus of this paper is not to review again recent discoveries, but to emphasize what we do not yet know about polyploidy, which despite all that has been learned about genome doubling is still an enormous amount. Our list is not meant to be comprehensive, but includes a range of topics that we have placed in several general categories, including mode of formation, ecological and physiological consequences, and genomic rules. Questions include: What is (are) the most frequent mechanism(s) of polyploidization? What factors promote/facilitate polyploidization? What factors favor autopolyploid vs. allopolyploid formation? Do multiple origins result in lineages with differing evolutionary trajectories and/or cryptic species? Our major goals are to stimulate discussion and promote further research.", 
"Rapidly accumulating evidence shows that herbivore and pathogen attack of plants can generate particular defense phenotypes across generations. What was once thought to be an oddity of plant defense induction now appears to be a taxonomically widespread phenomenon with strong potential to impact the ecology and evolution of species interactions. DNA methylation, histone modifications, and small RNAs each contribute to transgenerational defense initiation; examples in several species demonstrate that this induction can last for multiple generations. Priming of the offspring generation for more rapid induction following subsequent attack has also been reported. The extent to which transgenerational induction is predictable, detectable in nature, and subject to manipulation will determine the ability of researchers to decipher its role in plant-herbivore and plant-pathogen interactions.", 
"Aside from its selective role in filtering inter-individual variation during evolution by natural selection, the environment also plays an instructive role in producing variation during development. External environmental cues can influence developmental rates and/or trajectories and lead to the production of distinct phenotypes from the same genotype. This can result in a better match between adult phenotype and selective environment and thus represents a potential solution to problems posed by environmental fluctuation. The phenomenon is called adaptive developmental plasticity. The study of developmental plasticity integrates different disciplines (notably ecology and developmental biology) and analyses at all levels of biological organization, from the molecular regulation of changes in organismal development to variation in phenotypes and fitness in natural populations. Here, we focus on recent advances and examples from morphological traits in animals to provide a broad overview covering (i) the evolution of developmental plasticity, as well as its relevance to adaptive evolution, (ii) the ecological significance of alternative environmentally induced phenotypes, and the way the external environment can affect development to produce them, (iii) the molecular mechanisms underlying developmental plasticity, with emphasis on the contribution of genetic, physiological and epigenetic factors, and (iv) current challenges and trends, including the relevance of the environmental sensitivity of development to studies in ecological developmental biology, biomedicine and conservation biology.", 
"Organisms often respond to environmental changes by producing alternative phenotypes. Epigenetic processes such as DNA methylation may contribute to environmentally induced phenotypic variation by modifying gene expression. Changes in DNA methylation, unlike DNA mutations, can be influenced by the environment; they are stable at the time scale of an individual and present different levels of heritability. These characteristics make DNA methylation a potentially important molecular process to respond to environmental change. The aim of this review is to present the implications of DNA methylation on phenotypic variations driven by environmental changes. More specifically, we explore epigenetic concepts concerning phenotypic change in response to the environment and heritability of DNA methylation, namely the Baldwin effect and genetic accommodation. Before addressing this point, we report major differences in DNA methylation across taxa and the role of this modification in producing and maintaining environmentally induced phenotypic variation. We also present the different methods allowing the detection of methylation polymorphism. We believe this review will be helpful to molecular ecologists, in that it highlights the importance of epigenetic processes in ecological and evolutionary studies.", 
"Nongenetic inheritance is a potentially important but poorly understood factor in population responses to rapid environmental change. Accumulating evidence indicates that nongenetic inheritance influences a diverse array of traits in all organisms and can allow for the transmission of environmentally induced phenotypic changes (acquired traits), as well as spontaneously arising and highly mutable variants. We review models of adaptation to changing environments under the assumption of a broadened model of inheritance that incorporates nongenetic mechanisms of transmission, and survey relevant empirical examples. Theory suggests that nongenetic inheritance can increase the rate of both phenotypic and genetic change and, in some cases, alter the direction of change. Empirical evidence shows that a diversity of phenotypes spanning a continuum from adaptive to pathological can be transmitted nongenetically. The presence of nongenetic inheritance therefore complicates our understanding of evolutionary responses to environmental change. We outline a research program encompassing experimental studies that test for transgenerational effects of a range of environmental factors, followed by theoretical and empirical studies on the population-level consequences of such effects.", 
"Allopolyploidy is an evolutionary and mechanistically intriguing process, in that it entails the reconciliation of two or more sets of diverged genomes and regulatory interactions. In this study, we explored gene expression patterns in interspecific hybrid F-1, and synthetic and natural allopolyploid cotton using RNA-Seq reads from leaf transcriptomes. We determined how the extent and direction of expression level dominance (total level of expression for both homoeologs) and homoeolog expression bias (relative contribution of homoeologs to the transcriptome) changed from hybridization through evolution at the polyploid level and following cotton domestication. Genome-wide expression level dominance was biased toward the A-genome in the diploid hybrid and natural allopolyploids, whereas the direction was reversed in the synthetic allopolyploid. This biased expression level dominance was mainly caused by up-or downregulation of the homoeolog from the 'non-dominant' parent. Extensive alterations in homoeolog expression bias and expression level dominance accompany the initial merger of two diverged diploid genomes, suggesting a combination of regulatory (cis or trans) and epigenetic interactions that may arise and propagate through the transcriptome network. The extent of homoeolog expression bias and expression level dominance increases over time, from genome merger through evolution at the polyploid level. Higher rates of transgressive and novel gene expression patterns as well as homoeolog silencing were observed in natural allopolyploids than in F1 hybrid and synthetic allopolyploid cottons. These observations suggest that natural selection reconciles the regulatory mismatches caused by initial genomic merger, while new gene expression conditions are generated for evaluation by selection. Heredity (2013) 110, 171-180; doi:10.1038/hdy.2012.94; published online 21 November 2012", 
"Epigenetic variation is likely to contribute to the phenotypic plasticity and adaptative capacity of plant species, and may be especially important for long-lived organisms with complex life cycles, including forest trees. Diverse environmental stresses and hybridization/polyploidization events can create reversible heritable epigenetic marks that can be transmitted to subsequent generations as a form of molecular memory. Epigenetic changes might also contribute to the ability of plants to colonize or persist in variable environments. In this review, we provide an overview of recent data on epigenetic mechanisms involved in developmental processes and responses to environmental cues in plant, with a focus on forest tree species. We consider the possible role of forest tree epigenetics as a new source of adaptive traits in plant breeding, biotechnology, and ecosystem conservation under rapid climate change.", 
"Inheritance-the influence of ancestors on the phenotypes of their descendants-translates natural selection into evolutionary change. For the past century, inheritance has been conceptualized almost exclusively as the transmission of DNA sequence variation from parents to offspring in accordance with Mendelian rules, but advances in cell and developmental biology have now revealed a rich array of inheritance mechanisms. This empirical evidence calls for a unified conception of inheritance that combines genetic and nongenetic mechanisms and encompasses the known range of transgenerational effects, including the transmission of genetic and epigenetic variation, the transmission of plastic phenotypes (acquired traits), and the effects of parental environment and genotype on offspring phenotype. We propose a unified theoretical framework based on the Price equation that can be used to model evolution under an expanded inheritance concept that combines the effects of genetic and nongenetic inheritance. To illustrate the utility and generality of this framework, we show how it can be applied to a variety of scenarios, including nontransmissible environmental noise, maternal effects, indirect genetic effects, transgenerational epigenetic inheritance, RNA-mediated inheritance, and cultural inheritance.", 
"The consequences of early developmental conditions for performance in later life are now subjected to convergent interest from many different biological sub-disciplines. However, striking data, largely from the biomedical literature, show that environmental effects experienced even before conception can be transmissible to subsequent generations. Here, we review the growing evidence from natural systems for these cross-generational effects of early life conditions, showing that they can be generated by diverse environmental stressors, affect offspring in many ways and can be transmitted directly or indirectly by both parental lines for several generations. In doing so, we emphasize why early life might be so sensitive to the transmission of environmentally induced effects across generations. We also summarize recent theoretical advancements within the field of developmental plasticity, and discuss how parents might assemble different 'internal' and 'external' cues, even from the earliest stages of life, to instruct their investment decisions in offspring. In doing so, we provide a preliminary framework within the context of adaptive plasticity for understanding inter-generational phenomena that arise from early life conditions.", 
"The relationship between genotype (which is inherited) and phenotype (the target of selection) is mediated by environmental inputs on gene expression, trait development, and phenotypic integration. Phenotypic plasticity or epigenetic modification might influence evolution in two general ways: (1) by stimulating evolutionary responses to environmental change via population persistence or by revealing cryptic genetic variation to selection, and (2) through the process of genetic accommodation, whereby natural selection acts to improve the form, regulation, and phenotypic integration of novel phenotypic variants. We provide an overview of models and mechanisms for how such evolutionary influences may be manifested both for plasticity and epigenetic marking. We point to promising avenues of research, identifying systems that can best be used to address the role of plasticity in evolution, as well as the need to apply our expanding knowledge of genetic and epigenetic mechanisms to our understanding of how genetic accommodation occurs in nature. Our review of a wide variety of studies finds widespread evidence for evolution by genetic accommodation.", 
"Epigenetic information includes heritable signals that modulate gene expression but are not encoded in the primary nucleotide sequence. We have studied natural epigenetic variation in three allotetraploid sibling orchid species (Dactylorhiza majalis s.str, D. traunsteineri s.l., and D. ebudensis) that differ radically in geography/ecology. The epigenetic variation released by genome doubling has been restructured in species-specific patterns that reflect their recent evolutionary history and have an impact on their ecology and evolution, hundreds of generations after their formation. Using two contrasting approaches that yielded largely congruent results, epigenome scans pinpointed epiloci under divergent selection that correlate with eco-environmental variables, mainly related to water availability and temperature. The stable epigenetic divergence in this group is largely responsible for persistent ecological differences, which then set the stage for species-specific genetic patterns to accumulate in response to further selection and/or drift. Our results strongly suggest a need to expand our current evolutionary framework to encompass a complementary epigenetic dimension when seeking to understand population processes that drive phenotypic evolution and adaptation.", 
"Genomics promises exciting advances towards the important conservation goal of maximizing evolutionary potential, notwithstanding associated challenges. Here, we explore some of the complexity of adaptation genetics and discuss the strengths and limitations of genomics as a tool for characterizing evolutionary potential in the context of conservation management. Many traits are polygenic and can be strongly influenced by minor differences in regulatory networks and by epigenetic variation not visible in DNA sequence. Much of this critical complexity is difficult to detect using methods commonly used to identify adaptive variation, and this needs appropriate consideration when planning genomic screens, and when basing management decisions on genomic data. When the genomic basis of adaptation and future threats are well understood, it may be appropriate to focus management on particular adaptive traits. For more typical conservations scenarios, we argue that screening genome-wide variation should be a sensible approach that may provide a generalized measure of evolutionary potential that accounts for the contributions of small-effect loci and cryptic variation and is robust to uncertainty about future change and required adaptive response(s). The best conservation outcomes should be achieved when genomic estimates of evolutionary potential are used within an adaptive management framework.", 
"Heritable phenotypic variation in plants can be caused not only by underlying genetic differences, but also by variation in epigenetic modifications such as DNA methylation. However, we still know very little about how relevant such epigenetic variation is to the ecology and evolution of natural populations. We conducted a greenhouse experiment in which we treated a set of natural genotypes of Arabidopsis thaliana with the demethylating agent 5-azacytidine and examined the consequences of this treatment for plant traits and their phenotypic plasticity. Experimental demethylation strongly reduced the growth and fitness of plants and delayed their flowering, but the degree of this response varied significantly among genotypes. Differences in genotypes' responses to demethylation were only weakly related to their genetic relatedness, which is consistent with the idea that natural epigenetic variation is independent of genetic variation. Demethylation also altered patterns of phenotypic plasticity, as well as the amount of phenotypic variation observed among plant individuals and genotype means. We have demonstrated that epigenetic variation can have a dramatic impact on ecologically important plant traits and their variability, as well as on the fitness of plants and their ecological interactions. Epigenetic variation may thus be an overlooked factor in the evolutionary ecology of plant populations.", 
"Although there is keen interest in the potential adaptive value of epigenetic variation, it is unclear what conditions favor the stability of these variants either within or across generations. Because epigenetic modifications can be environmentally sensitive, existing theory on adaptive phenotypic plasticity provides relevant insights. Our consideration of this theory suggests that stable maintenance of environmentally induced epigenetic states over an organism's lifetime is most likely to be favored when the organism accurately responds to a single environmental change that subsequently remains constant, or when the environmental change cues an irreversible developmental transition. Stable transmission of adaptive epigenetic states from parents to offspring may be selectively favored when environments vary across generations and the parental environment predicts the offspring environment. The adaptive value of stability beyond a single generation of parent-offspring transmission likely depends on the costs of epigenetic resetting. Epigenetic stability both within and across generations will also depend on the degree and predictability of environmental variation, dispersal patterns, and the (epi)genetic architecture underlying phenotypic responses to environment. We also discuss conditions that favor stability of random epigenetic variants within the context of bet hedging. We conclude by proposing research directions to clarify the adaptive significance of epigenetic stability.", 
"The role that epigenetic inheritance can play in adaptation may differ between sexuals and asexuals because (1) the dynamics of adaptation differ under sexual and asexual reproduction and the opportunities offered by epigenetic inheritance may affect these dynamics differently; and (2) in asexual reproduction epigenetic reprogramming mechanisms that are associated with meiosis can be bypassed, which could promote the buildup of epigenetic variation in asexuals. Here, we evaluate current evidence for an epigenetic contribution to adaptation in asexuals. We argue that two aspects of epigenetic variation should have particular relevance for asexuals, namely epigenetics-mediated phenotypic plasticity within and between generations, and heritable variation via stochastic epimutations. An evaluation of epigenetic reprogramming mechanisms suggests that some, but not all, forms of asexual reproduction enhance the likelihood of stable transmission of epigenetic marks across generations compared to sexual reproduction. However, direct tests of these predicted sexual-asexual differences are virtually lacking. Stable transmission of DNA methylation, transcriptomes, and phenotypes from parent to clonal offspring are demonstrated in various asexual species, and clonal genotypes from natural populations show habitat-specific DNA methylation. We discuss how these initial observations can be extended to demonstrate an epigenetic contribution to adaptation.", 
"Individual variation in ecologically important features of organisms is a crucial element in ecology and evolution, yet disentangling its underlying causes is difficult in natural populations. We applied a genomic scan approach using amplified fragment length polymorphism (AFLP) markers to quantify the genetic basis of long-term individual differences in herbivory by mammals at a wild population of the violet Viola cazorlensis monitored for two decades. In addition, methylation-sensitive amplified polymorphism (MSAP) analyses were used to investigate the association between browsing damage and epigenetic characteristics of individuals, an aspect that has been not previously explored for any wild plant. Structural equation modelling was used to identify likely causal structures linking genotypes, epigenotypes and herbivory. Individuals of V. cazorlensis differed widely in the incidence of browsing mammals over the 20-year study period. Six AFLP markers (1.6% of total) were significantly related to herbivory, accounting altogether for 44% of population-wide variance in herbivory levels. MSAP analyses revealed considerable epigenetic variation among individuals, and differential browsing damage was significantly related to variation in multilocus epigenotypes. In addition, variation across plants in epigenetic characteristics was related to variation in several herbivory-related AFLP markers. Statistical comparison of alternative causal models suggested that individual differences in herbivory are the outcome of a complex causal structure where genotypes and epigenotypes are interconnected and have direct and indirect effects on herbivory. Insofar as methylation states of MSAP markers influential on herbivory are transgenerationally heritable, herbivore-driven evolutionary changes at the study population will involve correlated changes in genotypic and epigenotypic distributions.", 
"Because the maintenance of proper developmental temperatures during avian incubation is costly to parents, embryos of many species experience pronounced variation in incubation temperature. However, the effects of such temperature variation on nestling development remain relatively unexplored. To investigate this, we artificially incubated wild blue tit (Cyanistes caeruleus L.) clutches at 35.0 degrees, 36.5 degrees, or 38.0 degrees C for two-thirds of the incubation period. We returned clutches to their original nests before hatching and subsequently recorded nestling growth and resting metabolic rate. The length of the incubation period decreased with temperature, whereas hatching success increased. Nestlings from the lowest incubation temperature group had shorter tarsus lengths at 2 weeks of age, but body mass and wing length were not affected by temperature. In addition, nestlings from the lowest temperature group had a significantly higher resting metabolic rate compared with mid- and high-temperature nestlings, which may partly explain observed size differences between the groups. These findings suggest that nest microclimate can influence nestling phenotype, but whether observed differences carry over to later life-history stages remains unknown.", 
"Epigenetic mechanisms are proposed as an important way in which the genome responds to the environment. Epigenetic marks, including DNA methylation and Histone modifications, can be triggered by environmental effects, and lead to permanent changes in gene expression, affecting the phenotype of an organism. Epigenetic mechanisms have been proposed as key in plasticity, allowing environmental exposure to shape future gene expression. While we are beginning to understand how these mechanisms have roles in human biology and disease, we have little understanding of their roles and impacts on ecology and evolution. In this review, we discuss different types of epigenetic marks, their roles in gene expression and plasticity, methods for assaying epigenetic changes, and point out the future advances we require to understand fully the impact of this field. J. Exp. Zool. (Mol. Dev. Evol.) 322B: 208-220, 2014. (c) 2014 Wiley Periodicals, Inc.", 
"Transposable elements (TEs) are a dominant feature of most flowering plant genomes. Together with other accepted facilitators of evolution, accumulating data indicate that TEs can explain much about their rapid evolution and diversification. Genome size in angiosperms is highly correlated with TE content and the overwhelming bulk (> 80%) of large genomes can be composed of TEs. Among retro-TEs, long terminal repeats (LTRs) are abundant, whereas DNA-TEs, which are often less abundant than retro-TEs, are more active. Much adaptive or evolutionary potential in angiosperms is due to the activity of TEs (active TE-Thrust), resulting in an extraordinary array of genetic changes, including gene modifications, duplications, altered expression patterns, and exaptation to create novel genes, with occasional gene disruption. TEs implicated in the earliest origins of the angiosperms include the exapted Mustang, Sleeper, and Fhy3/Far1 gene families. Passive TE-Thrust can create a high degree of adaptive or evolutionary potential by engendering ectopic recombination events resulting in deletions, duplications, and karyotypic changes. TE activity can also alter epigenetic patterning, including that governing endosperm development, thus promoting reproductive isolation. Continuing evolution of long-lived resprouter angiosperms, together with genetic variation in their multiple meristems, indicates that TEs can facilitate somatic evolution in addition to germ line evolution. Critical to their success, angiosperms have a high frequency of polyploidy and hybridization, with resultant increased TE activity and introgression, and beneficial gene duplication. Together with traditional explanations, the enhanced genomic plasticity facilitated by TE-Thrust, suggests a more complete and satisfactory explanation for Darwin's abominable mystery: the spectacular success of the angiosperms.", 
"DNA methylation is an important, heritable epigenetic modification in most eukaryotic organisms that is connected with numerous biological processes. To study the impact of natural epigenetic variation in an ecological or evolutionary context, epigenetic studies are increasingly using methylation-sensitive amplification polymorphism (MSAP) for surveys at the population or species level. However, no consensus exists on how to interpret and score the multistate information obtained from the MSAP banding patterns. Here, we review the previously used scoring approaches for population epigenetic studies and develop new alternatives. To assess effects of the different approaches on parameters of epigenetic diversity and differentiation, we applied eight scoring schemes to a case study of three populations of the plant species Viola elatior. For a total number of 168 detected polymorphic MSAP fragments, the number of ultimately scored polymorphic epiloci ranged between 78 and 286 depending on the particular scoring scheme. Both, estimates of epigenetic diversity and differentiation varied strongly between scoring approaches. However, linear regression and PCoA revealed qualitatively similar patterns, suggesting that the scoring approaches are largely consistent. For single-locus analyses of MSAP data, for example the search for loci under selection, we advocate a new scoring approach that separately takes into account different methylation types and thus seems appropriate for drawing more detailed conclusions in ecological or evolutionary contexts. An R script (MSAP_score.r) for scoring and basic data analysis is provided.", 
"Karten's neocortex hypothesis holds that many component cell populations of the sauropsid dorsal ventricular ridge (DVR) are homologous to particular cell populations in layers of auditory and visual tectofugal-recipient neocortex of mammals (i.e., temporal neocortex), as well as to some amygdaloid populations. The claustroamygdalar hypothesis, based on gene expression domains, proposes that mammalian homologues of DVR are found in the claustrum, endopiriform nuclei, and/or pallial amygdala. Because hypotheses of homology need to account for the totality of the evidence, the available data on multiple forebrain features of sauropsids and mammals are reviewed here. While some genetic data are compatible with the claustroamygdalar hypothesis, and developmental (epigenetic) data are indecisive, hodological, morphological, and topographical data favor the neocortex hypothesis and are inconsistent with the claustroamygdalar hypothesis. Detailed studies of gene signaling cascades that establish neuronal cell-type identity in DVR, tectofugal-recipient neocortex, and claustroamygdala will be needed to resolve this debate about the evolution of neocortex.", 
"Maternal effects are now universally recognised as a form of nongenetic. parental influence on offspring but, until recently, paternal effects were regarded as an anomaly. Although it is now clear that paternal effects are both widespread and important, their proximate basis and evolutionary consequences have received little attention and remain poorly understood. In particular, because many paternal effects are mediated by maternal responses such as differential allocation, the boundary between paternal and maternal effects is sometimes blurred. We distinguish here three basic types of paternal effect and clarify the role of maternal responses in these effects. We also outline key questions that can serve as a road map for research on the proximate basis and evolutionary implications of paternal effects.", 
"There is increasing evidence for epigenetically mediated transgenerational inheritance across taxa. However, the evolutionary implications of such alternative mechanisms of inheritance remain unclear. Herein, we show that epigenetic mechanisms can serve two fundamentally different functions in transgenerational inheritance: (i) selection-based effects, which carry adaptive information in virtue of selection over many generations of reliable transmission; and (ii) detection-based effects, which are a transgenerational form of adaptive phenotypic plasticity. The two functions interact differently with a third form of epigenetic information transmission, namely information about cell state transmitted for somatic cell heredity in multicellular organisms. Selection-based epigenetic information is more likely to conflict with somatic cell inheritance than is detection-based epigenetic information. Consequently, the evolutionary implications of epigenetic mechanisms are different for unicellular and multicellular organisms, which underscores the conceptual and empirical importance of distinguishing between these two different forms of transgenerational epigenetic effect.", 
"A dramatic rise in obesity has occurred among humans within the last several decades. Little is known about whether similar increases in obesity have occurred in animals inhabiting human-influenced environments. We examined samples collectively consisting of over 20 000 animals from 24 populations (12 divided separately into males and females) of animals representing eight species living with or around humans in industrialized societies. In all populations, the estimated coefficient for the trend of body weight over time was positive (i.e. increasing). The probability of all trends being in the same direction by chance is 1.2 x 10(-7). Surprisingly, we find that over the past several decades, average mid-life body weights have risen among primates and rodents living in research colonies, as well as among feral rodents and domestic dogs and cats. The consistency of these findings among animals living in varying environments, suggests the intriguing possibility that the aetiology of increasing body weight may involve several as-of-yet unidentified and/or poorly understood factors (e.g. viral pathogens, epigenetic factors). This finding may eventually enhance the discovery and fuller elucidation of other factors that have contributed to the recent rise in obesity rates.", 
"Retrotransposable elements comprise around 50% of the mammalian genome. Their activity represents a constant threat to the host and has prompted the development of adaptive control mechanisms to protect genome architecture and function. To ensure their propagation, retrotransposons have to mobilize in cells destined for the next generation. Accordingly, these elements are particularly well suited to transcriptional networks associated with pluripotent and germinal states in mammals. The relaxation of epigenetic control that occurs in the early developing germline constitutes a dangerous window in which retrotransposons can escape from host restraint and massively expand. What could be observed as risky behavior may turn out to be an insidious strategy developed by germ cells to sense retrotransposons and hold them back in check. Herein, we review recent insights that have provided a detailed picture of the defense mechanisms that concur toward retrotransposon silencing in mammalian genomes, and in particular in the germline. In this lineage, retrotransposons are hit at multiple stages of their life cycle, through transcriptional repression, RNA degradation and translational control. An organized cross-talk between PIWI-interacting small RNAs (piRNAs) and various nuclear and cytoplasmic accessories provides this potent and multilayered response to retrotransposon unleashing in early germ cells. Heredity (2010) 105, 92-104; doi:10.1038/hdy.2010.53; published online 5 May 2010", 
"Chromosomal sex determination systems create the opportunity for the evolution of selfish genetic elements that increase the transmission of one sex chromosome at the expense of its homolog. Because such selfish elements on sex chromosomes can reduce fertility and distort the sex ratio of progeny, unlinked suppressors are expected to evolve, bringing different regions of the genome into conflict over the meiotic transmission of the sex chromosomes. Here we argue that recurrent genetic conflict over sex chromosome transmission is an important evolutionary force that has shaped a wide range of seemingly disparate phenomena including the epigenetic regulation of genes expressed in the germline, the distribution of genes in the genome, and the evolution of hybrid sterility between species.", 
"In this study msap, an R package which analyses methylation-sensitive amplified polymorphism (MSAP or MS-AFLP) data is presented. The program provides a deep analysis of epigenetic variation starting from a binary data matrix indicating the banding pattern between the isoesquizomeric endonucleases HpaII and MspI, with differential sensitivity to cytosine methylation. After comparing the restriction fragments, the program determines if each fragment is susceptible to methylation (representative of epigenetic variation) or if there is no evidence of methylation (representative of genetic variation). The package provides, in a user-friendly command line interface, a pipeline of different analyses of the variation (genetic and epigenetic) among user-defined groups of samples, as well as the classification of the methylation occurrences in those groups. Statistical testing provides support to the analyses. A comprehensive report of the analyses and several useful plots could help researchers to assess the epigenetic and genetic variation in their MSAP experiments. msap is downloadable from CRAN (http://cran.r-project.org/) and its own webpage (http://msap.r-forge.R-project.org/). The package is intended to be easy to use even for those people unfamiliar with the R command line environment. Advanced users may take advantage of the available source code to adapt msap to more complex analyses.", 
"In this paper we describe a novel approach that may shed light on the genomic DNA methylation of organisms with non-resolved genomes. The LUminometric Methylation Assay (LUMA) is permissive for genomic DNA methylation studies of any genome as it relies on the use of methyl-sensitive and -insensitive restriction enzymes followed by polymerase extension via Pyrosequencing technology. Here, LUMA was used to characterize genomic DNA methylation in the lower brain stem region from 47 polar bears subsistence hunted in central East Greenland between 1999 and 2001. In these samples, average genomic DNA methylation was 57.9% +/- 6.69 (SD; range was 42.0 to 72.4%). When genomic DNA methylation was related to brain mercury (Hg) exposure levels, an inverse association was seen between these two variables for the entire study population (P for trend = 0.17). After dichotomizing animals by gender and controlling for age, a negative trend was seen amongst male animals (P for trend = 0.07) but no associations were found in female bears. Such sexually dimorphic responses have been found in other toxicological studies. Our results show that genomic DNA methylation can be quantitatively studied in a highly reproducible manner in tissue samples from a wild organism with a non-resolved genome. As such, LUMA holds great promise as a novel method to explore consequential questions across the ecological sciences that may require an epigenetic understanding.", 
"In addition to genetic differences between individuals as a result of nucleotide sequence variation, epigenetic changes that occur as a result of DNA methylation may also contribute to population niche width by enhancing phenotypic plasticity, although this intriguing possibility remains essentially untested. Using the nectar-living yeast Metschnikowia reukaufii as study subject, we examine the hypothesis that changes in genome-wide DNA methylation patterns underlie the ability of this fugitive species to exploit a broad resource range in its heterogeneous and patchy environment. Data on floral nectar characteristics and their use by M. reukaufii in the wild were combined with laboratory experiments and methylation-sensitive amplified polymorphism (MSAP) analyses designed to detect epigenetic responses of single genotypes to variations in sugar environment that mimicked those occurring naturally in nectar. M. reukaufii exploited a broad range of resources, occurring in nectar of 48% of species and 52% of families surveyed, and its host plants exhibited broad intra- and interspecific variation in sugar-related nectar features. Under experimental conditions, sugar composition, sugar concentration and their interaction significantly influenced the mean probability of MSAP markers experiencing a transition from unmethylated to methylated state. Alterations in methylation status were not random but predictably associated with certain markers. The methylation inhibitor 5-azacytidine (5-AzaC) had strong inhibitory effects on M. reukaufii proliferation in sugar-containing media, and a direct relationship existed across sugar X concentration experimental levels linking inhibitory effect of 5-AzaC and mean per-marker probability of genome-wide methylation. Environmentally induced DNA methylation polymorphisms allowed genotypes to grow successfully in extreme sugar environments, and the broad population niche width of M. reukaufii was largely made possible by epigenetic changes enabling genotype plasticity in resource use.", 
"Long terminal repeat-retrotransposons (LTR-RTs) are the most abundant class of transposable elements (TEs) in plants. They strongly impact the structure, function, and evolution of their host genome, and, in particular, their role in genome size variation has been clearly established. However, the dynamics of the process through which LTR-RTs have differentially shaped plant genomes is still poorly understood because of a lack of comparative studies. Using a new robust and automated family classification procedure, we exhaustively characterized the LTR-RTs in eight plant genomes for which a high-quality sequence is available (i.e., Arabidopsis thaliana, A. lyrata, grapevine, soybean, rice, Brachypodium dystachion, sorghum, and maize). This allowed us to perform a comparative genome-wide study of the retrotranspositional landscape in these eight plant lineages from both monocots and dicots. We show that retrotransposition has recurrently occurred in all plant genomes investigated, regardless their size, and through bursts, rather than a continuous process. Moreover, in each genome, only one or few LTR-RT families have been active in the recent past, and the difference in genome size among the species studied could thus mostly be accounted for by the extent of the latest transpositional burst(s). Following these bursts, LTR-RTs are efficiently eliminated from their host genomes through recombination and deletion, but we show that the removal rate is not lineage specific. These new findings lead us to propose a new model of TE-driven genome evolution in plants.", 
"In recent years, a number of researchers have advocated extending the modern synthesis in evolutionary biology. One of the core arguments made in favour of an extension comes from work on soft inheritance systems, including transgenerational epigenetic effects, cultural transmission and niche construction. In this study, we outline this claim and then take issue with it. We argue that the focus on soft inheritance has led to a conflation of proximate and ultimate causation, which has in turn obscured key questions about biological organization and calibration across the life span to maximize average lifetime inclusive fitness. We illustrate this by presenting hypotheses that we believe incorporate the core phenomena of soft inheritance and will aid in understanding them.", 
"Background: The Poly(ADP-ribose) polymerase (PARP) superfamily was originally identified as enzymes that catalyze the attachment of ADP-ribose subunits to target proteins using NAD+ as a substrate. The family is characterized by the catalytic site, termed the PARP signature. While these proteins can be found in a range of eukaryotes, they have been best studied in mammals. In these organisms, PARPs have key functions in DNA repair, genome integrity and epigenetic regulation. More recently it has been found that proteins within the PARP superfamily have altered catalytic sites, and have mono(ADP-ribose) transferase (mART) activity or are enzymatically inactive. These findings suggest that the PARP signature has a broader range of functions that initially predicted. In this study, we investigate the evolutionary history of PARP genes across the eukaryotes. Results: We identified in silico 236 PARP proteins from 77 species across five of the six eukaryotic supergroups. We performed extensive phylogenetic analyses of the identified PARPs. They are found in all eukaryotic supergroups for which sequence is available, but some individual lineages within supergroups have independently lost these genes. The PARP superfamily can be subdivided into six clades. Two of these clades were likely found in the last common eukaryotic ancestor. In addition, we have identified PARPs in organisms in which they have not previously been described. Conclusions: Three main conclusions can be drawn from our study. First, the broad distribution and pattern of representation of PARP genes indicates that the ancestor of all extant eukaryotes encoded proteins of this type. Second, the ancestral PARP proteins had different functions and activities. One of these proteins was similar to human PARP1 and likely functioned in DNA damage response. The second of the ancestral PARPs had already evolved differences in its catalytic domain that suggest that these proteins may not have possessed poly(ADP-ribosyl)ation activity. Third, the diversity of the PARP superfamily is larger than previously documented, suggesting as more eukaryotic genomes become available, this gene family will grow in both number and type.", 
"Male rats (Rattus novergicus) infected with protozoan Toxoplasma gondii relinquish their innate aversion to the cat odours. This behavioural change is postulated to increase transmission of the parasite to its definitive felid hosts. Here, we show that the Toxoplasma gondii infection institutes an epigenetic change in the DNA methylation of the arginine vasopressin promoter in the medial amygdala of male rats. Infected animals exhibit hypomethylation of arginine vasopressin promoter, leading to greater expression of this nonapeptide. The infection also results in the greater activation of the vasopressinergic neurons after exposure to the cat odour. Furthermore, we show that loss of fear in the infected animals can be rescued by the systemic hypermethylation and recapitulated by directed hypomethylation in the medial amygdala. These results demonstrate an epigenetic proximate mechanism underlying the extended phenotype in the Rattus novergicus-Toxoplasma gondii association. See also the Perspective by Jaroslav Flegr", 
"Selection for enhanced cognitive traits is hypothesized to produce enhancements to brain structures that support those traits. Although numerous studies suggest that this pattern is robust, there are several mechanisms that may produce this association. First, cognitive traits and their neural underpinnings may be fixed as a result of differential selection on cognitive function within specific environments. Second, these relationships may be the product of the selection for plasticity, where differences are produced owing to an individual's experiences in the environment. Alternatively, the relationship may be a complex function of experience, genetics and/or epigenetic effects. Using a well-studied model species (black-capped chickadee, Poecile atricapillus), we have for the first time, to our knowledge, addressed these hypotheses. We found that differences in hippocampal (Hp) neuron number, neurogenesis and spatial memory previously observed in wild chickadees persisted in hand-raised birds from the same populations, even when birds were raised in an identical environment. These findings reject the hypothesis that variation in these traits is owing solely to differences in memory-based experiences in different environments. Moreover, neuron number and neurogenesis were strikingly similar between captive-raised and wild birds from the same populations, further supporting the genetic hypothesis. Hp volume, however, did not differ between the captive-raised populations, yet was very different in their wild counterparts, supporting the experience hypothesis. Our results indicate that the production of some Hp factors may be inherited and largely independent of environmental experiences in adult life, regardless of their magnitude, in animals under high selection pressure for memory, while traits such as volume may be more plastic and modified by the environment.", 
"Early-life stress caused by the deprivation of maternal care has been shown to have long-lasting effects on the hypothalamic-pituitary-adrenal (HPA) axis in offspring of uniparental mammalian species. We asked if deprivation of maternal care in biparental species alters stress responsiveness of offspring, using a biparental avian species-the zebra finch, Taeniopygia guttata. In our experiment, one group of birds was raised by both male and female parents (control), and another was raised by males alone (maternally deprived). During adulthood, offspring of both groups were subjected to two stressors (restraint and isolation), and corticosterone concentrations were measured. Additionally, we measured baseline levels of the two corticosteroid receptors-glucocorticoid receptor (GR) and mineralocorticoid receptor (MR)-in the hippocampus, hypothalamus and cerebellum. Our results suggest that maternally deprived offspring are hyper-responsive to isolation in comparison with controls. Furthermore, mRNA levels of both GR and MR receptors are altered in maternally deprived offspring in comparison with controls. Thus, absence of maternal care has lasting consequences for HPA function in a biparental species where paternal care is available.", 
"DNA methylation is an epigenetic mechanism that has the potential to affect plant phenotypes and that is responsive to environmental and genomic stresses such as hybridization and polyploidization. We explored de novo methylation variation that arises during the formation of triploid asexual dandelions from diploid sexual mother plants using methylation-sensitive amplified fragment length polymorphism (MS-AFLP) analysis. In dandelions, triploid apomictic asexuals are produced from diploid sexual mothers that are fertilized by polyploid pollen donors. We asked whether the ploidy level change that accompanies the formation of new asexual lineages triggers methylation changes that contribute to heritable epigenetic variation within novel asexual lineages. Comparison of MS-AFLP and AFLP fragment inheritance in a diploid x triploid cross revealed de novo methylation variation between triploid F(1) individuals. Genetically identical offspring of asexual F(1) plants showed modest levels of methylation variation, comparable to background levels as observed among sibs in a long-established asexual lineage. Thus, the cross between ploidy levels triggered de novo methylation variation between asexual lineages, whereas it did not seem to contribute directly to variation within new asexual lineages. The observed background level of methylation variation suggests that considerable autonomous methylation variation could build up within asexual lineages under natural conditions.", 
"DNA methylation is a fundamental epigenetic mark known to have wide-ranging effects on gene regulation in a variety of animal taxa. Comparative genomic analyses can help elucidate the function of DNA methylation by identifying conserved features of methylated genes and other genomic regions. In this study, we used computational approaches to distinguish genes marked by heavy methylation from those marked by little or no methylation in the pea aphid, Acyrthosiphon pisum. We investigated if these two classes had distinct evolutionary histories and functional roles by conducting comparative analysis with the honeybee, Apis (Ap.) mellifera. We found that highly methylated orthologs in A. pisum and Ap. mellifera exhibited greater conservation of methylation status, suggesting that highly methylated genes in ancestral species may remain highly methylated over time. We also found that methylated genes tended to show different rates of evolution than unmethylated genes. In addition, genes targeted by methylation were enriched for particular biological processes that differed from those in relatively unmethylated genes. Finally, methylated genes were preferentially ubiquitously expressed among alternate phenotypes in both species, whereas genes lacking signatures of methylation were preferentially associated with condition-specific gene expression. Overall, our analyses support a conserved role for DNA methylation in insects with comparable methylation systems.", 
"The epigenetic phenomenon of genomic imprinting has motivated the development of numerous theories for its evolutionary origins and genomic distribution. In this review, we examine the three theories that have best withstood theoretical and empirical scrutiny. These are: Haig and colleagues' kinship theory; Day and Bonduriansky's sexual antagonism theory; and Wolf and Hager's maternal-offspring coadaptation theory. These theories have fundamentally different perspectives on the adaptive significance of imprinting. The kinship theory views imprinting as a mechanism to change gene dosage, with imprinting evolving because of the differential effect that gene dosage has on the fitness of matrilineal and patrilineal relatives. The sexual antagonism and maternal-offspring coadaptation theories view genomic imprinting as a mechanism to modify the resemblance of an individual to its two parents, with imprinting evolving to increase the probability of expressing the fitter of the two alleles at a locus. In an effort to stimulate further empirical work on the topic, we carefully detail the logic and assumptions of all three theories, clarify the specific predictions of each and suggest tests to discriminate between these alternative theories for why particular genes are imprinted.", 
"Inbreeding depression (i.e. negative fitness effects of inbreeding) is central in evolutionary biology, affecting numerous aspects of population dynamics and demography, such as the evolution of mating systems, dispersal behaviour and the genetics of quantitative traits. Inbreeding depression is commonly observed in animals and plants. Here, we demonstrate that, in addition to genetic processes, epigenetic processes may play an important role in causing inbreeding effects. We compared epigenetic markers of outbred and inbred offspring of the perennial plant Scabiosa columbaria and found that inbreeding increases DNA methylation. Moreover, we found that inbreeding depression disappears when epigenetic variation is modified by treatment with a demethylation agent, linking inbreeding depression firmly to epigenetic variation. Our results suggest an as yet unknown mechanism for inbreeding effects and demonstrate the importance of evaluating the role of epigenetic processes in inbreeding depression.", 
"Age is a fundamental aspect of animal ecology, but is difficult to determine in many species. Humpback whales exemplify this as they have a lifespan comparable to humans, mature sexually as early as 4 years and have no reliable visual age indicators after their first year. Current methods for estimating humpback age cannot be applied to all individuals and populations. Assays for human age have recently been developed based on age-induced changes in DNA methylation of specific genes. We used information on age-associated DNA methylation in human and mouse genes to identify homologous gene regions in humpbacks. Humpback skin samples were obtained from individuals with a known year of birth and employed to calibrate relationships between cytosine methylation and age. Seven of 37 cytosines assayed for methylation level in humpback skin had significant age-related profiles. The three most age-informative cytosine markers were selected for a humpback epigenetic age assay. The assay has an R-2 of 0.787 (P = 3.04e-16) and predicts age from skin samples with a standard deviation of 2.991 years. The epigenetic method correctly determined which of parent-offspring pairs is the parent in more than 93% of cases. To demonstrate the potential of this technique, we constructed the first modern age profile of humpback whales off eastern Australia and compared the results to population structure 5 decades earlier. This is the first epigenetic age estimation method for a wild animal species and the approach we took for developing it can be applied to many other non-model organisms.", 
"DNA methylation is an epigenetic mark that has key roles in the control of genome activity in plants and mammals. It is critical for the stable silencing of repeat elements and is also involved in the epigenetic regulation of some genes. Despite similarities in the controlling functions of DNA methylation, its dynamics and deposition patterns differ in several respects between plants and mammals. One of the most striking differences is that plants tend to propagate pre-existing DNA methylation states across generations, whereas mammals re-establish them genome wide at every generation. Here, we review our current understanding of DNA methylation in the flowering plant Arabidopsis. We discuss in particular the role of RNAi in the incremental methylation and silencing of repeat elements over successive generations. We argue that para-mutation, an epigenetic phenomenon first described in maize, is an extreme manifestation of this RNAi-dependent pathway. Heredity (2010) 105, 14-23; doi:10.1038/hdy.2010.52; published online 12 May 2010", 
"The functional and evolutionary significance of DNA methylation in insect genomes remains to be resolved. Nasonia is well situated for comparative analyses of DNA methylation and genome evolution, since the genomes of a moderately distant outgroup species as well as closely related sibling species are available. Using direct sequencing of bisulfite-converted DNA, we uncovered a substantial level of DNA methylation in 17 of 18 Nasonia vitripennis genes and a strong correlation between methylation level and CpG depletion. Notably, in the sex-determining locus transformer, the exon that is alternatively spliced between the sexes is heavily methylated in both males and females, whereas other exons are only sparsely methylated. Orthologous genes of the honeybee and Nasonia show highly similar relative levels of CpG depletion, despite similar to 190 My divergence. Densely and sparsely methylated genes in these species also exhibit similar functional enrichments. We found that the degree of CpG depletion is negatively correlated with substitution rates between closely related Nasonia species for synonymous, nonsynonymous, and intron sites. This suggests that mutation rates increase with decreasing levels of germ line methylation. Thus, DNA methylation is prevalent in the Nasonia genome, may participate in regulatory processes such as sex determination and alternative splicing, and is correlated with several aspects of genome and sequence evolution.", 
"Over the last four decades, bioarchaeology has experienced significant technical growth and theoretical maturation. Early 21st century bioarchaeology may also be enhanced from a renewed engagement with the concept of biological stress. New insights on biological stress and disease can be gained from cross-disciplinary perspectives regarding human skeletal variation and disease. First, pathophysiologic and molecular signaling mechanisms can provide more precise understandings regarding formation of pathological phenotypes in bone. Using periosteal new bone formation as an example, various mechanisms and pathways are explored in which new bone can be formed under conditions of biological stress, particularly in bone microenvironments that involve inflammatory changes. Second, insights from human biology are examined regarding some epigenetic factors and disease etiology. While epigenetic effects on stress and disease outcomes appear profoundly influential, they are mostly invisible in skeletal tissue. However, some indirect and downstream effects, such as the developmental origins of adult health outcomes, may be partially observable in bioarchaeological data. Emerging perspectives from the human microbiome are also considered. Microbiomics involves a remarkable potential to understand ancient biology, disease, and stress. Third, tools from epidemiology are examined that may aid bioarchaeologists to better cope with some of the inherent limitations of skeletal samples to better measure and quantify the expressions of skeletal stress markers. Such cross-disciplinary synergisms hopefully will promote more complete understandings of health and stress in bioarchaeological science. (C) 2014 Wiley Periodicals, Inc.", 
"An appreciation of the fundamental principles of evolutionary biology provides new insights into major diseases and enables an integrated understanding of human biology and medicine. However, there is a lack of awareness of their importance amongst physicians, medical researchers, and educators, all of whom tend to focus on the mechanistic (proximate) basis for disease, excluding consideration of evolutionary (ultimate) reasons. The key principles of evolutionary medicine are that selection acts on fitness, not health or longevity; that our evolutionary history does not cause disease, but rather impacts on our risk of disease in particular environments; and that we are now living in novel environments compared to those in which we evolved. We consider these evolutionary principles in conjunction with population genetics and describe several pathways by which evolutionary processes can affect disease risk. These perspectives provide a more cohesive framework for gaining insights into the determinants of health and disease. Coupled with complementary insights offered by advances in genomic, epigenetic, and developmental biology research, evolutionary perspectives offer an important addition to understanding disease. Further, there are a number of aspects of evolutionary medicine that can add considerably to studies in other domains of contemporary evolutionary studies.", 
"Variation of DNA methylation is thought to play an important role for rapid adjustments of plant populations to dynamic environmental conditions, thus compensating for the relatively slow response time of genetic adaptations. However, genetic and epigenetic variation of wild plant populations has not yet been directly compared in fast changing environments. Here, we surveyed populations of Viola elatior from two adjacent habitat types along a successional gradient characterized by strong differences in light availability. Using amplified fragment length polymorphisms (AFLP) and methylation-sensitive amplification polymorphisms (MSAP) analyses, we found relatively low levels of genetic (H-gen' = 0.19) and epigenetic (H-epi' = 0.23) diversity and high genetic (phi(ST) = 0.72) and epigenetic (phi(ST) = 0.51) population differentiation. Diversity and differentiation were significantly correlated, suggesting that epigenetic variation partly depends on the same driving forces as genetic variation. Correlation-based genome scans detected comparable levels of genetic (17.0%) and epigenetic (14.2%) outlier markers associated with site specific light availability. However, as revealed by separate differentiation-based genome scans for AFLP, only few genetic markers seemed to be actually under positive selection (0-4.5%). Moreover, principal coordinates analyses and Mantel tests showed that overall epigenetic variation was more closely related to habitat conditions, indicating that environmentally induced methylation changes may lead to convergence of populations experiencing similar habitat conditions and thus may play a major role for the transient and/or heritable adjustment to changing environments. Additionally, using a new MSAP-scoring approach, we found that mainly the unmethylated (phi(ST) = 0.60) and CG-methylated states (phi(ST) = 0.46) of epiloci contributed to population differentiation and putative habitat-related adaptation, whereas CHG-hemimethylated states (phi(ST) = 0.21) only played a marginal role.", 
"Epigenetic information is an important mediator of the relationship between genotype and phenotype in eukaryotic organisms. One of the most important and widely conserved forms of epigenetic information is the methylation of genes. However, the function of intragenic DNA methylation remains poorly understood. The goal of this study was to gain greater understanding of the nature of intragenic methylation by determining its role in the multilayered epigenetic landscape of insects. We first investigated the evolutionary lability of DNA methylation by examining whether methylation patterns were conserved in the fire ant and honey bee. We found that DNA methylation was targeted to largely overlapping sets of orthologs in both species. Next, we compared intragenic DNA methylation levels in the fire ant and honey bee to comprehensive epigenetic and gene-regulatory data from Drosophila melanogaster orthologs. We observed striking evidence of a conserved association between DNA methylation in fire ants and honey bees, and several active histone modifications, constitutive gene expression, and broad promoter architecture in D. melanogaster. Overall, our study illustrates that DNA methylation is a single component of a conserved, integrated, multilayered epigenetic and regulatory landscape in insect genomes.", 
"Modern evolutionary biology is founded on the Mendelian-genetic model of inheritance, but it is now clear that this model is incomplete. Empirical evidence shows that environment (encompassing all external influences on the genome) can impose transgenerational effects and generate heritable variation for a broad array of traits in animals, plants, and other organisms. Such effects can be mediated by the transmission of epigenetic, cytoplasmic, somatic, nutritional, environmental, and behavioral variation. Building on the work of many authors, we outline a general framework for conceptualizing nongenetic inheritance and its evolutionary implications. This framework shows that, by decoupling phenotypic change from the genotype, nongenetic inheritance can circumvent the limitations of genetic inheritance and thereby influence population dynamics and alter the fitness landscape. The weight of theory and empirical evidence indicates that nongenetic inheritance is a potent factor in evolution that can engender outcomes unanticipated under the Mendelian-genetic model.", 
"To study the consequences of hybridization and genome duplication on polyploid genome evolution and adaptation, we used independently formed hybrids (Spartina x townsendii and Spartina x neyrautii) that originated from natural crosses between Spartina alterniflora, an American introduced species, and the European native Spartina maritima. The hybrid from England, S. x townsendii, gave rise to the invasive allopolyploid, salt-marsh species, Spartina anglica. Recent studies indicated that allopolyploid speciation may be associated with rapid genetic and epigenetic changes. To assess this in Spartina, we performed AFLP (amplified fragment length polymorphism) and MSAP (methylation sensitive amplification polymorphism) on young hybrids and the allopolyploid. By comparing the subgenomes in the hybrids and the allopolyploid to the parental species, we inferred structural changes that arose repeatedly in the two independently formed hybrids. Surprisingly, 30% of the parental methylation patterns are altered in the hybrids and the allopolyploid. This high level of epigenetic regulation might explain the morphological plasticity of Spartina anglica and its larger ecological amplitude. Hybridization rather than genome doubling seems to have triggered most of the methylation changes observed in Spartina anglica.", 
"Phenotypic plasticity is the development of different phenotypes from a single genotype, depending on the environment. Such plasticity is a pervasive feature of life, is observed for various traits and is often argued to be the result of natural selection. A thorough study of phenotypic plasticity should thus include an ecological and an evolutionary perspective. Recent advances in large-scale gene expression technology make it possible to also study plasticity from a molecular perspective, and the addition of these data will help answer long-standing questions about this widespread phenomenon. In this review, we present examples of integrative studies that illustrate the molecular and cellular mechanisms underlying plastic traits, and show how new techniques will grow in importance in the study of these plastic molecular processes. These techniques include: (i) heterologous hybridization to DNA microarrays; (ii) next generation sequencing technologies applied to transcriptomics; (iii) techniques for studying the function of noncoding small RNAs; and (iv) proteomic tools. We also present recent studies on genetic model systems that uncover how environmental cues triggering different plastic responses are sensed and integrated by the organism. Finally, we describe recent work on changes in gene expression in response to an environmental cue that persist after the cue is removed. Such long-term responses are made possible by epigenetic molecular mechanisms, including DNA methylation. The results of these current studies help us outline future avenues for the study of plasticity.", 
"The Modern Synthesis (MS) is the current paradigm in evolutionary biology. It was actually built by expanding on the conceptual foundations laid out by its predecessors, Darwinism and neo-Darwinism. For sometime now there has been talk of a new Extended Evolutionary Synthesis (EES), and this article begins to outline why we may need such an extension, and how it may come about. As philosopher Karl Popper has noticed, the current evolutionary theory is a theory of genes, and we still lack a theory of forms. The field began, in fact, as a theory of forms in Darwin's days, and the major goal that an EES will aim for is a unification of our theories of genes and of forms. This may be achieved through an organic grafting of novel concepts onto the foundational structure of the MS, particularly evolvability, phenotypic plasticity, epigenetic inheritance, complexity theory, and the theory of evolution in highly dimensional adaptive landscapes.", 
"The genus Silene, studied by Darwin, Mendel and other early scientists, is re-emerging as a system for studying interrelated questions in ecology, evolution and developmental biology. These questions include sex chromosome evolution, epigenetic control of sex expression, genomic conflict and speciation. Its well-studied interactions with the pathogen Microbotryum has made Silene a model for the evolution and dynamics of disease in natural systems, and its interactions with herbivores have increased our understanding of multi-trophic ecological processes and the evolution of invasiveness. Molecular tools are now providing new approaches to many of these classical yet unresolved problems, and new progress is being made through combining phylogenetic, genomic and molecular evolutionary studies with ecological and phenotypic data. Heredity (2009) 103, 5-14; doi: 10.1038/hdy.2009.34; published online 15 April 2009", 
"Understanding the developmental and genetic basis for evolutionarily significant morphological variation in complex phenotypes such as the mammalian skull is a challenge because of the sheer complexity of the factors involved. We hypothesize that even in this complex system, the expression of phenotypic variation is structured by the interaction of a few key developmental processes. To test this hypothesis, we created a highly variable sample of crania using four mouse mutants and their wild-type controls from similar genetic backgrounds with developmental perturbations to particular cranial regions. Using geometric morphometric methods we compared patterns of size, shape, and integration in the sample within and between the basicranium, neurocranium, and face. The results highlight regular and predictable patterns of covariation among regions of the skull that presumably reflect the epigenetic influences of the genetic perturbations in the sample. Covariation between relative widths of adjoining regions is the most dominant factor, but there are other significant axes of covariation such as the relationship between neurocranial size and basicranial flexion. Although there are other sources of variation related to developmental perturbations not analyzed in this study, the patterns of covariation created by the epigenetic interactions evident in this sample may underlie larger scale evolutionary patterns in mammalian craniofacial form.", 
"Form and genes often tell different stories about the evolution of animals, with molecular data generally considered to be more objective than morphological data. However, form provides the basis for the description of organisms, and the study of fossils crucially depends on morphology. Complex organisms tend to evolve as 'mosaics', in which parts may be modified at varying rates and in response to different selective pressures. Thus, individual anatomical regions may contain different phylogenetic signals. In the present study, we used computerized methods to 'dissect' the skulls of a primate clade, the guenons, into functional and developmental modules (FDM). The potential of different modules as proxies for phylogenetic divergence in modern lineages was investigated. We found that the chondrocranium was the only FDM in which shape consistently had a strong and significant phylogenetic signal. This region might be less susceptible to epigenetic factors and thus more informative about phylogeny. The examination of the topology of trees from the chondrocranium suggested that the main differences evolved at the time of the radiation of terrestrial and arboreal guenons. However, phylogenetic reconstructions were found to be strongly affected by sampling error, with more localized anatomical regions (i.e. smaller/less complex FDMs) generally producing less reproducible tree topologies. This finding, if confirmed in other groups, implies that the utility of specific FDMs for phylogenetic inference could, in many cases, be hampered by the low reproducibility of results. The study also suggested that uncertainties due to sampling error may be larger than those from character sampling. This might have implications for phylogenetic analyses, which typically provide estimates of support of tree nodes based on characters but do not generally take into account the effect of sampling error on the tree topology. Nonetheless, studies of the potential of different FDMs as proxies for phylogenetic divergence in modern lineages, such as the present study, provide a framework that may help in modelling the morphological evolution of present and fossil species. (C) 2008 The Linnean Society of London.", 
"This article introduces a special issue on evolutionary innovation and morphological novelty, two interrelated themes that have received a remarkable increase of attention over the past few years. We begin with a discussion of the question of whether innovation and novelty represent distinct evolutionary problems that require a distinct conceptualization. We argue that the mechanisms of innovation and their phenotypic results-novelty-can only be properly addressed if they are distinguished from the standard evolutionary themes of variation and adaptation, and we present arguments for making such a distinction. We propose that origination, the first formation of biological structures, is another distinct problem of morphological evolution, and that together with innovation and novelty it constitutes a conceptual complex we call the innovation triad. We define a problem agenda of the triad, which separates the analysis of the initiating conditions from the mechanistic realization of innovation, and we discuss the theoretical problems that arise from treating innovation as distinct from variation. Further, we categorize the empirical approaches that address themes of the innovation triad in recognizing four major strands of research: the morphology and systematics program, the gene regulation program, the epigenetic program, and the theoretical biology program. We provide examples of each program, giving priority to contributions in the present issue. In conclusion, we observe that the innovation triad is one of the defining topics of EvoDevo research and may represent its most pertinent contribution to evolutionary theory. We point out that an inclusion of developmental systems properties into evolutionary theory represents a shift of explanatory emphasis from the external factors of natural selection to the internal dynamics of developmental systems, complementing adaptation with emergence, and contingency with inherency.", 
"Most bilaterians specify primordial germ cells (PGCs) during early embryogenesis using either inherited cytoplasmic germ line determinants (preformation) or induction of germ cell fate through signaling pathways (epigenesis). However, data from nonbilaterian animals suggest that ancestral metazoans may have specified germ cells very differently from most extant bilaterians. Cnidarians and sponges have been reported to generate germ cells continuously throughout reproductive life, but previous studies on members of these basal phyla have not examined embryonic germ cell origin. To try to define the embryonic origin of PGCs in the sea anemone Nematostella vectensis, we examined the expression of members of the vasa and nanos gene families, which are critical genes in bilaterian germ cell specification and development. We found that vasa and nanos family genes are expressed not only in presumptive PGCs late in embryonic development, but also in multiple somatic cell types during early embryogenesis. These results suggest one way in which preformation in germ cell development might have evolved from the ancestral epigenetic mechanism that was probably used by a metazoan ancestor.", 
"It is widely recognized that maternal phenotype can have important effects on offspring, but paternal phenotype is generally assumed to have no influence in animals lacking paternal care. Nonetheless, selection may favour the transfer of environmentally acquired condition to offspring from both parents. Using a split-brood, cross-generational laboratory design, we manipulated a key environmental determinant of condition - larval diet quality - of parents and their offspring in the fly Telostylinus angusticollis, in which there is no evidence of paternal provisioning. Parental diet did not affect offspring survival, but high-condition mothers produced larger eggs, and their offspring developed more rapidly when on a poor larval diet. Maternal condition had no effect on adult body size of offspring. By contrast, large, high-condition fathers produced larger offspring, and follow-up assays showed that this paternal effect can be sufficient to increase mating success of male offspring and fecundity of female offspring. Our findings suggest that both mothers and fathers transfer their condition to offspring, but with effects on different offspring traits. Moreover, our results suggest that paternal effects can be important even in species lacking conventional forms of paternal care. In such species, the transfer of paternal condition to offspring could contribute to indirect selection on female mate preferences.", 
"We describe a new hypothesis for the development of autism, that it is driven by imbalances in brain development involving enhanced effects of paternally expressed imprinted genes, deficits of effects from maternally expressed genes, or both. This hypothesis is supported by: (1) the strong genomic-imprinting component to the genetic and developmental mechanisms of autism, Angelman syndrome, Rett syndrome and Turner syndrome; (2) the core behavioural features of autism, such as self-focused behaviour, altered social interactions and language, and enhanced spatial and mechanistic cognition and abilities, and (3) the degree to which relevant brain functions and structures are altered in autism and related disorders. The imprinted brain theory of autism has important implications for understanding the genetic, epigenetic, neurological and cognitive bases of autism, as ultimately due to imbalances in the outcomes of intragenomic conflict between effects of maternally vs. paternally expressed genes.", 
"Mimulus guttatus (yellow monkeyflower) frequently produce glandular trichomes, a trait that may resist herbivory. Constitutive production of trichomes is variable both within and among populations of M. guttatus and most of this variation is genetic. This study demonstrates that damage on early leaves can induce increased trichome production on later leaves, a plastic response that is likely adaptive. Moreover, this study shows that this induction can be maternally transmitted, increasing trichome density in progeny before they experience herbivory. This transgenerational response must involve a yet undescribed epigenetic mechanism. These experiments also show genetic variation among plants in the capacity for both within and between plant generation induction. Despite the clear evolutionary importance of variation in constitutive and induced herbivory-resistance traits, few other studies have noted genetic variation in both within a plant species.", 
"The study of insect satellite DNAs (satDNAs) indicates the evolutionary conservation of certain features despite their sequence heterogeneity. Such features can include total length, monomer length, motifs, particular regions and/or secondary and tertiary structures. satDNAs may act as protein-binding sites, structural domains or sites for epigenetic modifications. The selective constraints in the evolution of satDNAs may be due to the satDNA sequence interaction with specific proteins important in heterochromatin formation and possible a role in controlling gene expression. The transcription of satDNA has been described in vertebrates, invertebrates and plants. In insects, differential satDNA expression has been observed in different cells, developmental stages, sex and caste of the individuals. These transcription differences may suggest their involvement in gene-regulation processes. In addition, the satDNA or its transcripts appear to be involved in heterochromatin formation and in chromatin-elimination processes. The importance of transposable elements to insect satDNA is shown by their presence as a constituent of satDNA in several species of insects (including possible active elements). In addition, they may be involved in the formation of centromeres and telomeres and in the homogenization and expansion of satDNA.", 
"Several models explain how a complex integrated system like the rodent mandible can arise from multiple developmental modules. The models propose various integrating mechanisms, including epigenetic effects of muscles on bones. We test five for their ability to predict correlations found in the individual (symmetric) and fluctuating asymmetric (FA) components of shape variation. We also use exploratory methods to discern patterns unanticipated by any model. Two models fit observed correlation matrices from both components: (1) parts originating in same mesenchymal condensation are integrated, (2) parts developmentally dependent on the same muscle form an integrated complex as do those dependent on teeth. Another fits the correlations observed in FA: each muscle insertion site is an integrated unit. However, no model fits well, and none predicts the complex structure found in the exploratory analyses, best described as a reticulated network. Furthermore, no model predicts the correlation between proximal parts of the condyloid and coronoid, which can exceed the correlations between proximal and distal parts of the same process. Additionally, no model predicts the correlation between molar alveolus and ramus and/or angular process, one of the highest correlations found in the FA component. That correlation contradicts the basic premise of all five developmental models, yet it should be anticipated from the epigenetic effects of mastication, possibly the primary morphogenetic process integrating the jaw coupling forces generated by muscle contraction with those experienced at teeth.", 
"Accumulating evidence suggests that hybrid genetic dysfunctions accrue not only because of sequence divergence of incompatible alleles but also result from a broad variety of mechanisms related to the maintenance of chromatin integrity. For example, it has been observed that hybridization in plants and mammals disrupts patterns of DNA methylation and imprinting. These epigenetic changes can be associated with transcriptional activation and mobilization of transposable elements in hybrids. It raises a question of how these alterations are matched by small regulatory RNAs, such as piwi-interacting RNAs, which play a potent role in both suppressing transposable elements and epigenetic control. The review offers a handful of glimpses into these complex dynamics.", 
"Molar microwear fabrics in extant mammals vary with diet and, more particularly, the physical properties of the items that are consumed. Praeanthropus afarensis is well represented in the fossil record over a prolonged and radiometrically controlled temporal span, and reasonably robust paleoecological reconstructions are available for the various localities from which it is known. We therefore examined molar microwear in this species to determine whether diet varied in relation to time or in response to different ecological conditions. Of more than 70 specimens of Pr. afarensis that contain one or more worn permanent molars, only 19 were found to be suitable for microwear analysis. These derive from eight temporal horizons in the Laetolil Beds and Hadar Formation spanning approximately 400 kyr (3.6-3.2 Ma). Six paleoecological categories have been reconstructed for these horizons, and these were ranked on the basis of floral cover. None of the microwear variables observed for Pr. afarensis is significantly associated with either temporal or paleoecological rank. Thus, microwear and, by extension, diet does not appear to have altered significantly in Pr. afarensis through time or in response to different paleoecological circumstances. The wear pattern that appears to have characterized Pr. afarensis overlaps extensively that of Gorilla gorilla beringei and differs notably from the fabrics of extant primates (e.g., Cebus apella and Cercocebus albigena) that consume hard objects. The high proportion of scratches on Pr. afarensis molars suggests the inclusion of fine abrasives in or on the food items consumed by those individuals sampled in this study. Although Pr. afarensis may have been morphologically equipped to process hard, brittle items, the microwear data suggest that it did not necessarily do so, even in the face of varying environmental circumstances. Explanatory scenarios that describe Pr. afarensis as part of an evolutionary trajectory involving a more heavily masticated diet with an increased reliance on hard, brittle items need to be reconsidered. However, fallback foods that were consumed during relatively short, albeit critical periods may have exerted sufficient selective pressure to explain the evolution of the comparatively robust Pr. afarensis trophic apparatus. Because it is unlikely that many individuals from such restricted temporal intervals would be sampled in the paleontological record, we suggest that the most productive approach to the elucidation of paleodiet is the integration of genetic (morphological) and epigenetic (microwear and isotopic) lines of evidence. (c) 2006 Elsevier Ltd. All rights reserved.", 
"Cytosine methylation is important in the epigenetic regulation of gene expression and development in plants and has been implicated in silencing duplicate genes after polyploid formation in several plant groups. Relatively little information exists, however, on levels and patterns of methylation polymorphism (MP) at homologous loci within species. Here we explored the levels and patterns of methylation-polymorphism diversity at CCGG sites within allotetraploid cotton, Gossypium hirsutum, using a methylation-sensitive amplified fragment length polymorphism screen and a selected set of 20 G. hirsutum accessions for which we have information on genetic polymorphism levels and relationships. Methylation and MP exist at high levels within G. hirsutum: of 150 HpaII/MspI sites surveyed, 48 were methylated at the inner cytosine (32%) and 32 of these were polymorphic (67%). Both these values are higher than comparable measures of genetic diversity using restriction fragment length polymorphisms. The high percentage of methylation-polymorphic sites and potential relationship to gene expression Underscore the potential significance of MP within and among populations. We speculate that biased correlation of methylation-polymorphic sites and genes in cotton may be a consequence of polyploidy and the attendant doubling of all genes.", 
"Hvbridization and polyploidization are now recognized as major phenomena in the evolution of plants, promoting,Q genetic diversity, adaptive radiation and speciation. Modern molecular techniques have recently provided evidence that allopolyploidy can induce several types of genetic and epigenetic events that are of critical importance for the evolutionary success of these plants: (1) chromosomal rearrangements within one or both parental genomes contribute toward proper meiotic pairing and isolation of the hybrid from its progenitors; (2) demethylation and activation of dormant transposable elements may trigger insertional mutagenesis and changes in local patterns of gene expression, facilitating rapid genomic reorganization; (3) rapid and reproducible loss of low copy DNA sequences appears to result in further differentiation of homoeologous chromosomes; and (4) organ-specific up- or down-regulation of one of the duplicated genes, may result in unequal expression or silencing one copy. All these alterations also have the potential, while stabilizing allopolyploid genomes, to produce novel expression patterns and new phenotypes, which together with increased heterozygosity and gene redundancy might confer on allopolyploids an elevated evolutionary potential, with effects at scales ranging from molecular to ecological. Although important advances have been rnade in understanding genomic responses to allopolyploidization, further insights are still expected to be gained in the near future, Such as the direction and nature of the diploidization process, functional relevance of gene expression alterations, molecular mechanisms that result in adaptation to different ecologies/habitats, and ecological and evolutionary implications of recurrent polyploidization.", 
"Planarians are highly regenerative organisms with the ability to remake all their cell types, including the germ cells. The germ cells have been suggested to arise from totipotent neoblasts through epigenetic mechanisms. Nanos is a zinc-finger protein with a widely conserved role in the maintenance of germ cell identity. In this work, we describe the expression of a planarian nanos-like gene Smednos in two kinds of precursor cells namely, primordial germ cells and eye precursor cells, during both development and regeneration of the planarian Schmidtea mediterranea. In sexual planarians, Smednos is expressed in presumptive male primordial germ cells of embryos from stage 8 of embryogenesis and throughout development of the male gonads and in the female primordial germ cells of the ovary. Thus, upon hatching, juvenile planarians do possess primordial germ cells. In the asexual strain, Smednos is expressed in presumptive male and female primordial germ cells. During regeneration, Smednos expression is maintained in the primordial germ cells, and new clusters of Smednos-positive cells appear in the regenerated tissue. Remarkably, during the final stages of development (stage 8 of embryogenesis) and during regeneration of the planarian eye, Smednos is expressed in cells surrounding the differentiating eye cells, possibly corresponding to eye precursor cells. Our results suggest that similar genetic mechanisms might be used to control the differentiation of precursor cells during development and regeneration in planarians.", 
"Ty3/gypsy retrotransposons are rare in mammalian genomes despite their abundance in invertebrate and other vertebrate classes. Here we identify a family of nine conserved mammalian genes with homology to Ty3/gypsy retrotransposons but which have lost their ability to autonomously retrotranspose. Of these, five map to the X chromosome while the remaining four are autosomal. Comparative phylogenetic analyses show them to have strongest homology to the sushi-ichi element from Fugu rubripes. Two of the autosomal gene members, Peg10 and Rtl1, are known to be imprinted, being expressed from the paternally inherited chromosome homologue. This suggests, consistent with the host-parasite response theory of the evolution of the imprinting mechanism, that parental-origin specific epigenetic control may be mediated by genomic parasitic elements such as these. Alternatively, these elements may preferentially integrate into regions that are differentially modified on the two homologous chromosomes such as imprinted domains and the X chromosome and acquire monoallelic expression. We assess the imprinting status of the remaining autosomal members of this family and show them to be biallelically expressed in embryo and placenta. Furthermore, the methylation status of Rtl1 was assayed throughout development and was found to resemble that of actively, silenced repetitive elements rather than imprinted sequences. This indicates that the ability to undergo genomic imprinting is not an inherent property of all members of this family of retroelements. Nonetheless, the conservation but functional divergence between the different members suggests that they have undergone positive selection and acquired distinct endogenous functions within their mammalian hosts.", 
"Human promoters divide into 2 classes, the low CpG (LCG) and the high CpG (HCG), based on their CpG dinucleotide content. The LCG class of promoters is hypermethylated and is associated with tissue-specific genes, whereas the HCG class is hypomethylated and associated with broadly expressed genes. By analyzing several chordate genomes separated for hundreds of millions of years, here we show that the divide between low CpG and high CpG promoters is conserved in several distantly related vertebrate taxa (including human, chicken, frog, lizard, and fish) but not in close invertebrate outgroups (sea squirts). Furthermore, LCG and HCG promoters are distinctively associated with tissue-specific and broadly expressed genes in these distantly related vertebrate taxa. Our results indicate that the function of DNA methylation on gene expression is conserved across these vertebrate taxa and suggest that the 2 classes of promoters have evolved early in vertebrate evolution, as a consequence of the advent of global DNA methylation.", 
"The vertebrate limb has provided evolutionary and developmental biologists with grist for theory and experiment for at least a century. Its most salient features are its pattern of discrete skeletal elements, the general proximodistal increase in element number as development proceeds, and the individualization of size and shape of the elements in line with functional requirements. Despite increased knowledge of molecular changes during limb development, however, the mechanisms for origination and innovation of the vertebrate limb pattern are still uncertain. We suggest that the bauplan of the limb is based on an interplay of genetic and epigenetic processes; in particular, the self-organizing properties of precartilage mesenchymal tissue are proposed to provide the basis for its ability to generate regularly spaced nodules and rods of cartilage. We provide an experimentally based core set of cellular and molecular processes in limb mesenchyme that, under realistic conditions, exhibit the requisite self-organizing behavior for pattern origination. We describe simulations that show that under limb bud-like geometries the core mechanism gives rise to skeletons with authentic proximodistal spatiotemporal organization. Finally, we propose that evolution refines skeletal templates generated by this process by mobilizing accessory molecular and biomechanical regulatory processes to shape the developing limb and its individual elements. Morphological innovation may take place when such modulatory processes exceed a threshold defined by the dynamics of the skeletogenic system and elements are added or lost.", 
"The mammalian mandible is a developmentally modular but functionally integrated system. Whether morphological integration can evolve to match the optimal pattern of functional integration may depend on the developmental origin of integration, specifically, on the role that direct epigenetic interactions play in shaping integration. These interactions are hypothesized to integrate modules and also to be highly conservative, potentially constraining the evolution of integration. Using the fox squirrel (Sciurus niger) mandible as a model system, we test five a priori developmental hypotheses that predict mandibular integration and we also explore for correlations between shapes of mandibular regions not anticipated by any of the developmental models. To determine whether direct epigenetic interactions are highly conserved in rodents, we examine the correlation structure of fluctuating asymmetry, and compare integration patterns between fox squirrels and prairie deer mice (Peromyscus maniculatus bairdii). In fox squirrels, we find a correlation structure unanticipated by all a priori developmental models: adjacent parts along the proximodistal jaw axis are correlated whereas more distant ones are not. The most notable exception is that the shape of the anterior incisor alveolus is correlated with the shape of the ramus (FA component) or coronoid (symmetric component). Those exceptions differ between species; in prairie deer mice, the molar alveolus is connected to more parts, and the incisor alveolus to fewer, than in fox squirrels. The structure of integration suggests that the mandible cannot be decomposed into parts but rather is a single connected unit, a result consistent with its functional integration. That match between functional and developmental integration may arise, at least in part, from function-induced growth, building developmental integration into the functional system and enabling direct epigenetic interactions to evolve when function does.", 
"Human reproductive ecology (HRE) is the study of the mechanisms that link variation in reproductive traits with variation in local habitats. Empirical and theoretical contributions from biological anthropology, physiology, and demography have established the foundation necessary for developing a comprehensive understanding, grounded in life history theory (LHT), of temporal, individual, and populational variation in women's reproductive functioning. LHT posits that natural selection leads to the evolution of mechanisms that tend to allocate resources to the competing demands of growth, reproduction, and survival such that fitness is locally maximized. (That is, among alternative allocation patterns exhibited in a population, those having the highest inclusive fitness will become more common over generational time.) Hence, strategic modulation of reproductive effort is potentially adaptive because investment in a new conception may risk one's own survival, future reproductive opportunities, and/or current offspring survival. The hypothalamic-pituitary-ovarian (HPO) axis is the principal neuroendocrine pathway by which the human female modulates reproductive functioning according to the changing conditions in her habitat. Adjustments of reproductive investment in a potential conception are manifested in temporal and individual variation in ovarian cycle length, ovulation, hormone levels, and the probability of conception. Understanding the extent and causes of adaptive and non-adaptive variation in ovarian functioning is fundamental to ascertaining the proximate and remote determinants of human reproductive patterns. In this review I consider what is known and what still needs to be learned of the ecology of women's reproductive biology, beginning with a discussion of the principal explanatory frameworks in HRE and the biometry of ovarian functioning. Turning next to empirical studies, it is evident that marked variation between cycles, women, and populations is the norm rather than an aberration. Other than woman's age, the determinants of these differences are not well characterized, although developmental conditions, dietary practices, genetic variation, and epigenetic mechanisms have all been hypothesized to play some role. It is also evident that the reproductive functioning of women born and living in arduous conditions is not analogous to that of athletes, dieters, or even the lower end of the normal range of HPO functioning in wealthier populations. Contrary to the presumption that humans have low fecundity and an inefficient reproductive system, both theory and present evidence suggest that we may actually have very high fecundity and a reproductive system that has evolved to be flexible, ruthlessly efficient and, most importantly, strategic. Yrbk Phys Anthropol 52:95-136, 2009. (C) 2009 Wiley-Liss, Inc.", 
"Background: The evolution of antibiotic resistance in bacteria is a topic of major medical importance. Evolution is the result of natural selection acting on variant phenotypes. Both the rigid base sequence of DNA and the more plastic expression patterns of the genes present define phenotype. Results: We investigated the evolution of resistant E. coli when exposed to low concentrations of antibiotic. We show that within an isogenic population there are heritable variations in gene expression patterns, providing phenotypic diversity for antibiotic selection to act on. We studied resistance to three different antibiotics, ampicillin, tetracycline and nalidixic acid, which act by inhibiting cell wall synthesis, protein synthesis and DNA synthesis, respectively. In each case survival rates were too high to be accounted for by spontaneous DNA mutation. In addition, resistance levels could be ramped higher by successive exposures to increasing antibiotic concentrations. Furthermore, reversion rates to antibiotic sensitivity were extremely high, generally over 50%, consistent with an epigenetic inheritance mode of resistance. The gene expression patterns of the antibiotic resistant E. coli were characterized with microarrays. Candidate genes, whose altered expression might confer survival, were tested by driving constitutive overexpression and determining antibiotic resistance. Three categories of resistance genes were identified. The endogenous beta-lactamase gene represented a cryptic gene, normally inactive, but when by chance expressed capable of providing potent ampicillin resistance. The glutamate decarboxylase gene, in contrast, is normally expressed, but when overexpressed has the incidental capacity to give an increase in ampicillin resistance. And the DAM methylase gene is capable of regulating the expression of other genes, including multidrug efflux pumps. Conclusion: In this report we describe the evolution of antibiotic resistance in bacteria mediated by the epigenetic inheritance of variant gene expression patterns. This provides proof in principle that epigenetic inheritance, as well as DNA mutation, can drive evolution.", 
"Many fungi have heterokaryotic life stages in which genetically different nuclei inhabit the same cell. In basidiomycetes, the heterokaryon is the product of mating and represents a genomic union very similar to a diploid thallus, yet the maintenance of unfused nuclei suggests a more complex association of the two genomes relative to diploidy. In species with variable numbers of nuclei per heterokaryotic cell, nuclear ratios within a mycelium may possibly become imbalanced (differ from 1:1) due to nuclear competition. In this study, heterokaryons of the basidiomycete Heterobasidion parviporum were examined to determine the effects of genotype and environment on nuclear ratios within vegetative mycelia. The data reveal that nuclear ratios are frequently imbalanced, generally stable over time, and genetically determined. The nuclear ratios were affected by environment, but the observed nuclear ratios did not follow the expectations of strong selection acting on a population of nuclei. Instead, these ratios were largely driven by genetic effects and epigenetic effects. Finally, the data suggest that nuclear ratio imbalance also affects both gene transcription and growth rate, implying that heterokaryotic basidiomycetes are not functionally equivalent to diploid individuals and have a higher potential for genotypic and phenotypic variation.", 
"Greater offspring predation favors evolution of faster development among species. We hypothesized that greater offspring predation exerts selection on mothers to increase levels of anabolic androgens in egg yolks to achieve faster development. Here, we tested whether (1) concentrations of yolk androgens in passerine species were associated with offspring predation and (2) embryo and nestling development rates were associated with yolk androgen concentrations. We examined three androgens that increase in potency along the synthesis pathway: androstenedione (A(4)) to testosterone (T) to 5 alpha-dihydrotestosterone (5 alpha-DHT). Concentrations of none of these steroids were related to clutch size; only A, was allometrically related to egg volume. Species that experience greater predation showed higher yolk concentrations of T and 5 alpha-DHT. Higher concentrations of T and particularly 5 alpha-DHT were strongly correlated with faster development during the embryo period and less so during the nestling period. Development rates were most strongly correlated with 5 alpha-DHT, suggesting that potency increases along the androgen synthesis pathway and that effects are mediated by the androgen receptor pathway. These results are consistent with the hypothesis that selection for faster development by time-dependent offspring mortality may be achieved epigenetically by varying embryo exposure to maternal anabolic steroids.", 
"Studies of microbial eukaryotes have been pivotal in the discovery of biological phenomena, including RNA editing, self-splicing RNA, and telomere addition. Here we extend this list by demonstrating that genome architecture, namely the extensive processing of somatic (macronuclear) genomes in some ciliate lineages, is associated with elevated rates of protein evolution. Using newly developed likelihood-based procedures for studying molecular evolution, we investigate 6 genes to compare 1) ciliate protein evolution to that of 3 other clades of eukaryotes (plants, animals, and fungi) and 2) protein evolution in ciliates with extensively processed macronuclear genomes to that of other ciliate lineages. In 5 of the 6 genes, ciliates are estimated to have a higher ratio of nonsynonymous/synonymous substitution rates, consistent with an increase in the rate of protein diversification in ciliates relative to other eukaryotes. Even more striking, there is a significant effect of genome architecture within ciliates as the most divergent proteins are consistently found in those lineages with the most highly processed macronuclear genomes. We propose a model whereby genome architecture-specifically chromosomal processing, amitosis within macronuclei, and epigenetics-allows ciliates to explore protein space in a novel manner. Further, we predict that examination of diverse eukaryotes will reveal additional evidence of the impact of genome architecture on molecular evolution.", 
"Gene duplication is a major force for generating evolutionary novelties that lead to adaptations to environments. We previously identified two paralogs encoding phytochrome A (phyA), GmphyA1 and GmphyA2, in soybean, a paleopolyploid species. GmphyA2 is encoded by the E4 locus responsible for photoperiod sensitivity. In photoperiod insensitive lines, GmphyA2 is inactivated by the insertion of a retrotransposon in exon 1. Here, we describe the detailed characterization of the element and its evolutionary significance inferred from the distribution of the allele that harbors the element. Structural characteristics indicated that the element, designated SORE-1, is a novel Ty1/copia-like retrotransposon in soybean, which was phylogenetically related to the Sto-4, BARE-1, and RIRE1 elements. The element was transcriptionally active, and the transcription was partially repressed by an epigenetic mechanism. Sequences homologous with SORE-1 were detected in a genome sequence database of soybean, most of which appeared silent. GmphyA2 that harbors the SORE-1 insertion was detected only in cultivated soybean lines grown in northern regions of Japan, consistent with the notion that photoperiod insensitivity caused by the dysfunction of GmphyA2 is one of genetic changes that allowed soybean cultivation at high latitudes. Taking into account that genetic redundancy is conferred by the two phyA genes, we propose a novel model for the consequences of gene duplication and transposition of retrotransposons: when the gene is duplicated, retrotransposon insertion that causes the loss of a gene function can lead to adaptive evolution while the organism is sustained by the buffering effect brought about by gene duplication.", 
"Background: Genomic imprinting is an epigenetic source of variation in quantitative traits that results from monoallelic gene expression, where commonly either only the paternally- or the maternally-derived allele is expressed. Imprinting has been shown to affect a diversity of complex traits in a variety of species. For several such quantitative traits sex-dependent genetic effects have been discovered, but whether imprinting effects also show such sex-dependence has yet to be explored. Moreover, theoretical work on the evolution of sex-dependent genomic imprinting effects makes specific predictions about the phenotypic patterns of such effects, which, however, have not been assessed empirically to date. Results: Using a genome-scan for loci affecting a set of complex growth and body composition traits from an intercross between two divergent mouse strains, we investigated possible sex-dependent imprinting effects. Our results demonstrate for the first time the existence of genomic imprinting effects that depend on sex and are not related to sex-chromosome effects. We detected a total of 13 loci on 11 chromosomes that showed significant differences between the sexes in imprinting effects. Most loci showed imprinting effects in only one sex, with eight imprinted effects found in males and six in females. One locus showed sex-dependent imprinting effects in both sexes for different traits. The absence of an imprinting effect in one sex was not necessarily indicative of the overall inactivity of the locus in that sex, as for several loci a significant additive or dominance effect was detected. Moreover, three loci exhibited significant additive effects in both sexes but their imprinting effect was restricted to one sex. Conclusion: Our results clearly show that imprinting effects can be sex-dependent and also suggest that new candidate imprinted loci can be detected when taking account of sex-specific imprinting effects. However, predictions made about the evolution of sex-dependent imprinting effects and associated phenotypic patterns cannot be unequivocally supported at present and further research into the selection pressures applied to the strains of mice used in our study is required.", 
"Modular variation of multivariate traits results from modular distribution of effects of genetic and epigenetic interactions among those traits. However, statistical methods rarely detect truly modular patterns, possibly because the processes that generate intramodular associations may overlap spatially. Methodologically, this overlap may cause multiple patterns of modularity to be equally consistent with observed covariances. To deal with this indeterminacy, the present study outlines a framework for testing a priori hypotheses of modularity in which putative modules are mathematically represented as multidimensional subspaces embedded in the data. Model expectations are computed by subdividing the data into arrays of variables, and intermodular interactions are represented by overlapping arrays. Covariance structures are thus modeled as the outcome of complex and nonorthogonal intermodular interactions. This approach is demonstrated by analyzing mandibular modularity in nine rodent species. A total of 620 models are fit to each species, and the most strongly supported are heuristically modified to improve their fit. Five modules common to all species are identified, which approximately map to the developmental modules of the mandible. Within species, these modules are embedded within larger super-modules, suggesting that these conserved modules act as building blocks from which covariation patterns are built.", 
"Sexual reproduction implies high costs, but it is difficult to give evidence for evolutionary advantages that would explain the predominance of meiotic sex in eukaryotes. A combinational theory discussing evolution, maintenance and loss of sex may resolve the problem. The main function of sex is the restoration of DNA and consequently a higher quality of offspring. Recombination at meiosis evolved, perhaps, as a repair mechanism of DNA strand damages. This mechanism is most efficient for DNA restoration in multicellular eukaryotes, because the initial cell starts with a re-optimized genome, which is passed to all the daughter cells. Meiosis acts also as creator of variation in haploid stages, in which selection can purge most efficiently deleterious mutations. A prolonged diploid phase buffers the effects of deleterious recessive alleles as well as epigenetic defects and is thus optimal for prolonged growth periods. For complex multicellular organisms, the main advantage of sexuality is thus the alternation of diploid and haploid stages, combining advantages of both. A loss of sex is constrained by several, partly group-specific, developmental features. Hybridization may trigger shifts from sexual to asexual reproduction, but crossing barriers of the parental sexual species limit this process. For the concerted break-up of meiosis-outcrossing cycles plus silencing of secondary features, various group-specific changes in the regulatory system may be required. An establishment of asexuals requires special functional modifications and environmental opportunities. Costs for maintenance of meiotic sex are consequently lower than a shift to asexual reproduction. Heredity (2009) 103, 445-457; doi:10.1038/hdy.2009.85; published online 22 July 2009", 
"Environmental factors can induce significant epigenetic changes that may also be inherited by future generations. The maternally inherited symbiont of arthropods Wolbachia pipientis is an excellent candidate as an 'environmental' factor promoting trans-generational epigenetic changes: by establishing intimate relationships with germ-line cells, epigenetic effects of Wolbachia symbiosis would be manifested as a,maternal effect', in which infection of the mother modulates the offspring phenotype. In the leafhopper Zyginidia pullula, Wolbachia feminizes genetic males, leaving them as intersexes. With the exception of male chitinous structures that are present in the last abdominal segment, feminized males display phenotypic features that are typical of females. These include ovaries that range from a typical histological architecture to an altered structure. Methylation-sensitive random amplification of polymorphic DNA profiles show that they possess a female genomic imprint. On the other hand, some rare feminized males bear testes instead of ovaries. These specimens possess a Wolbachia density approximately four orders of magnitude lower than feminized males with ovaries and maintain a male genome-methylation pattern. Our results indicate that Wolbachia infection disrupts male imprinting, which dramatically influences the expression of genes involved in sex differentiation and development, and the alteration occurs only if Wolbachia exceeds a density threshold. Thus, a new Wolbachia's role as an environmental evolutionary force, inducing epigenetic trans-generational changes, should now be considered.", 
"When in evolutionary history did long-range projectile weapons become an important component of hunting toolkits? The archeological evidence for the development of projectile weaponry is complex and generally indirect, and has led to different conclusions about the origin and spread of this technology. Lithic evidence from the Middle Stone Age (MSA) has led some researchers to suggest that true long-range projectile weaponry developed in Africa perhaps as early as 80,000 years ago, and was part of the subsistence toolkit carried by modern humans who expanded out of Africa after 50,000 years ago. Alternatively, temporal patterns in the morphology of pointed lithics has led others to posit an independent, convergent origin of projectile weaponry in Africa, the Near East, and Europe during the interval between 50,000-40,000 years ago. By either scenario, projectile weapons would not have been a component of the hunting arsenal of Neandertals, but may have been in use by European early modern humans and thus, projectile technology may have entered into the competitive dynamics that existed between these two groups. The origins of projectile weapons can be addressed, in part, through analyses of the skeletal remains of the prehistoric humans who made and used them. Habitual behavior patterns-including those related to the production and use of technology-can be imprinted on the skeleton through both genetic and epigenetic pathways. Recent studies in the field of sports medicine indicate that individuals who engage in habitual throwing have increased humeral retroversion angles in their throwing arms and a greater degree of bilateral asymmetry in retroversion angles than do non-throwers. This contribution investigates humeral torsion through analysis of the retroversion angle in samples of Eurasian Neandertals, European early modern humans of the middle and late Upper Paleolithic, and comparative samples of recent humans. This analysis was conducted under the assumption that if throwing-based projectile weaponry was used by early modern Europeans but not Neandertals, Upper Paleolithic samples should be similar to recent human groups engaged in habitual throwing in the degree of humeral retroversion in the dominant limb and in bilateral asymmetry in this feature. Neandertals on the other hand, would not be expected to show marked asymmetry in humeral retroversion. Consistent with other studies, Neandertals exhibit increased retroversion angles (decreased humeral torsion or a more posteriorly oriented humeral head) relative to most modern human samples, although this appears more likely related to body form and overall activity levels than to habitual throwing. Although Neandertals with bilaterally preserved humeri sufficient for measurement are rare (consisting of only two males and one female), levels of bilateral asymmetry in humeral retroversion are low, suggesting a lack of regular throwing. While patterning across fossil and comparative samples in levels of humeral retroversion was not clear cut, males of both the middle and late Upper Paleolithic demonstrate a high level of bilateral asymmetry, comparable to or in excess of that seen in samples of throwing athletes. This may indicate habitual use of throwing-based projectile weaponry by middle Upper Paleolithic times. Small sample sizes and relatively great variance in the fossil samples makes these results, however, suggestive rather than conclusive. (C) 2008 Elsevier Ltd. All rights reserved.", 
"Satellite DNA is a major component of centromeric heterochromatin in most multicellular eukaryotes, where it is typically organized into megabase-sized tandem arrays. It has recently been demonstrated that small interfering RNAs (siRNAs) processed from centromeric satellite repeats can be involved in epigenetic chromatin modifications which appear to underpin centromere function. However, the structural organization and evolution of the centromeric satellite DNA is still poorly understood. We analyzed the centromeric satellite repeat arrays from rice chromosomes 1 and 8 and identified higher order structures and local homogenization of the CentO repeats in these 2 centromeres. We also cloned the CentO repeats from the CENH3-associated nucleosomes by a chromatin immunoprecipitation (ChIP)-based method. Sequence variability analysis of the ChIPed CentO repeats revealed a single variable domain within the repeat. We detected transcripts derived from both strands of the CentO repeats. The CentO transcripts are processed into siRNA, suggesting a potential role of this satellite repeat family in epigenetic chromatin modification.", 
"DNA methylation is one of the epigenetic and hereditary mechanisms regulating genetic expression in mammalian cells. In this review, we propose how certain natural agents, through their dietary consumption, could induce changes in physiological aspects in mammalian mothers, leading to alterations in DNA methylation patterns of the developing fetus and to the emergence of new phenotypes and evolutionary change. Nevertheless, we hypothesize that this process would require (i) certain key periods in the ontogeny of the organism where the environmental stimuli could produce effects, (ii) particular environmental agents as such stimuli, and (iii) that a genomic persistent change be consequently produced in a population. Depending on the persistence of the environmental stimuli and on whether the affected genes are imprinted genes, induced changes in DNA methylation patterns could become persistent. Moreover, some fragments could be more frequently methylated than others over several generations, leading to biased base change and evolutionary consequences.", 
"Changes in DNA methylation patterns during embryo development and differentiation processes are linked to the transcriptional plasticity of our genome. However, little is known about the evolutionary conservation of DNA methylation patterns and the evolutionary impact of epigenetic differences between closely related species. Here we compared the methylation patterns of CpG islands (CGIs) in the promoter regions of seven genes in humans and chimpanzees. We identified a block of CpGs in the cell cycle-related kinase (CCRK) gene that is more methylated in the adult human cortex than in the chimpanzee cortex and, in addition, it exhibits considerable intraspecific variation both in humans and chimpanzees. The species-specifically methylated region (SMR) lies between the almost completely methylated 5' region and the completely demethylated 3' region of the presumed CCRK CGI promoter. It is part of an Alu-Sg1 repeat that has been integrated into the promoter region in a common ancestor of humans and New World monkeys. This SMR is relatively hypomethylated in the rhesus monkey cortex and more or less completely methylated in the baboon cortex, indicating extraordinary methylation dynamics during primate evolution. The mRNA expression level of CCRK has also changed during the course of primate evolution. CCRK is expressed at much higher levels in human and baboon cortices, which display an average SMR methylation of 70% and 100%, respectively, than in chimpanzee and rhesus macaque cortices with an average SMR methylation of 35% and 40%, respectively. The observed evolutionary dynamics suggests a possibility that CCRK has been important for evolution of the primate brain.", 
"The goal of this short review is to consider the interrelated phenomena of phenotypic variation and genetic constraint with respect to plant diversity. The unique aspects of plants, including sessile habit, modular growth and diverse developmental programs expressed at the phytomer level, merit a specific examination of the genetic basis of their phenotypic variation, and how they experience and escape genetic constraint. Numerous QTL studies with wild and domesticated plants reveal that most phenotypic traits are polygenic but vary in the number and effect of the loci contributing, from a few loci of large effects to many with small effects. Further, somatic mutations, developmental plasticity and epigenetic variation, especially gene methylation, can contribute to increases in phenotypic variation. The flip side of these processes, genetic constraint, can similarly be the result of many factors, including pleiotropy, canalization and genetic redundancy. Genetic constraint is not only a mechanism to prevent change, however, it can also serve to direct evolution along certain paths. Ultimately, genetic constraint often comes full circle and is released through events such as hybridization, genome duplication and epigenetic remodeling. We are just beginning to understand how these processes can operate simultaneously during the evolution of ecologically important traits in plants.", 
"Developmentally regulated genome rearrangements (DRGR)-processes that alter genomes either in specific cells or during specific life cycle stages-are widespread throughout eukaryotes. This contrasts with the view that genome structure and content remain essentially constant throughout an organism's life cycle. Here we review three categories of developmentally regulated genome processing in eukaryotes: genome-wide rearrangements, targeted rearrangements, and a special case of amplification of ribosomal DNA genes. Mapping these types of DRGR onto eukaryotic phylogeny indicates that each type of processing is found in multiple independent lineages. We propose that such genome rearrangements were present within the last common ancestor of extant eukaryotes, and that future research will yield evidence of homologous epigenetic mechanisms underlying genome processing among diverse eukaryotes.", 
"Rapid recent developments in DNA sequencing and genetic marker technologies call for the establishment of cost-effective, automated phenotyping assays for evolutionary biology and systematics, so that the effects of DNA polymorphisms and epigenetic changes on the phenotype call be evaluated. We discuss the use of digital image-based morphometrics in evolutionary biology and systematics with special emphasis oil Studies of taxa with porous genomes, i.e., taxa that do not conform to the traditional view of whole-genome isolation between species. We outline the phenomenon of the 'phenotypic mosaic' in taxa with porous genomes using well-documented examples from the literature and describe three important challenges arising for taxonomists: (1) character conflict in phylogenetic studies, (2) biased sampling of traits in morphological Studies, and (3) cryptic ecological speciation. We demonstrate the use of geometric morphometrics using a combined molecular and morphometric dataset from an interspecific hybrid zone between two divergent Eurasian species of Populus, P. alba (white poplar) and P. tremula (European aspen). Elliptic Fourier analysis (EFA)-based morphometric data were collected for 527 leaf specimens from 84 trees with known genomic composition as determined by a set of 30 nuclear DNA microsatellites. In addition, to demonstrate the case of scoring functionally relevant phenotypes via digital image analysis, quantitative differences in leaf reflectance were examined and their structural basis determined using scanning electron microscopy (SEM). The EFA results indicate a heritable inter-individual component for symmetric aspects and all important intra-individual component for asymmetric aspects of variation in leaf outlines. Symmetric traits displayed a striking variety of phenotypes in hybrids compared to their parental species, consistent with the notion of the phenotypic mosaic. Linear discriminant analysis of these morphometric traits revealed (1) clear differentiation between parental species and (2) divergence between recombinant hybrids and their sympatric backcross parent P. alba, mediated primarily by two P. tremula-like and two transgressive traits. Our results demonstrate the usefulness of geometric morphometrics to interrogate multiple independent phenotypic characters and detect individual traits affected by introgression and divergence in taxa with porous genomes. Digital image-based morphometrics holds great promise for large-scale Studies of relationships between DNA polymorphism and phenotypes in evolutionary biology.", 
"The interaction between maternally provided environment and offspring genotype is a major determinant of offspring development and fitness in many organisms. Recent research has demonstrated that not only genetic effects, but also epigenetic effects may be subject to modifications by the maternal environment. Genomic imprinting resulting in parent-of-origin-dependent gene expression is among the best studied of epigenetic effects. However, very little is known about the degree to which genomic imprinting effects can be modulated by the maternally provided environment, which has important implications for phenotypic plasticity. In this study, we investigated this unresolved question using a cross-fostering design in which mouse pups were nursed by either their own or an unrelated mother. We scanned the entire genome to search for quantitative trait loci whose effects depend on cross-fostering and detected 10 of such loci. Of the 10 loci, 4 showed imprinting by cross-foster interactions. Inmost cases, the interaction effect was due to the presence of an effect in either cross-fostered or non-cross-fostered animals. Our results demonstrate that genomic imprinting effects may often be modified by the maternal environment and that such interactions can impact key fitness-related traits suggesting a greater plasticity of genomic imprinting than previously assumed.", 
"In the early 1930s, the synthesis of Darwinian natural selection, mutation, and Mendelian genetics gave rise to the paradigm of 'modern Darwinism', also known as 'neo-Darwinism'. This has contributed greatly to our understanding. But increasing knowledge of other mechanisms, including endosymbiosis, genetic and genomic duplication, polyploidy, hybridization, epigenetics, horizontal gene transfer in prokaryotes, and the modern synthesis of embryonic development and evolution, has widened our horizons to a diversity of possibilities for change. All of these can be gathered under the umbrella concept of 'genomic creativity', which, in partnership with natural selection, affords a more comprehensive modern explanation of evolution. (c) 2006 The Linnean Society of London, Biological Journal of the Linnean Society, 2006, 88, 655-672.", 
"The primary goal of virtually all organisms is to produce genetic offspring, thereby passing on their genes to future generations. Offspring production, however, is limited by available resources within an environment. Moreover, distributing sufficient energy among competing physiological systems is challenging and can result in trade-offs between self-maintenance and offspring investment when resources are limited. In the current study, we tested the hypothesis that the adipose hormone leptin is involved in mediating energetic trade-offs between competing physiological systems. Specifically, we tested the effects of elevated maternal leptin on investment into offspring production versus self maintenance (immune function), in the Siberian hamster (Phodopus sungorus). The current study provides the first evidence that leptin serves as a signal to mothers of available energy resulting in epigenetic effects. Therefore, elevated leptin allows females to retain more embryos to parturition, and rear more offspring to weaning via reduced maternal infanticide. Innate immune response was suppressed seemingly as a result of these enlarged litters, suggesting that the observed fitness increase is not without costs to the mother. Collectively, these findings suggest that leptin plays a critical role in allowing mothers to determine how much energy to invest in the production and care of young versus self-maintenance.", 
"Sequential segmentation from a posterior proliferative zone is considered to be the primitive mechanism of segmentation in arthropods. Several studies of embryonic and post-embryonic development and gene expression suggest that this occurs in all major arthropod taxa. Sequential segmentation is often associated with the idea of posterior production of body units that accumulate along the main body axis. However, the precise mechanism of sequential segmentation has not been identified yet, and, while searching for the genetic circuitry able to generate a first periodic pattern in the embryo, we can at least outline the distinctive role in segmentation of a proliferative zone. A perusal of myriapod segmentation patterns suggests that these patterns result from multi-layered developmental processes, where gene expression and epigenetic mechanisms interact in a nonstrictly hierarchical way. The posterior zone is possibly a zone of periodic signal production, but, in general, the resulting segmental pattern is not completely attributable to the activity of the signal generator. In this sense, a posterior proliferative zone would be more a segmental organizer than a segment generator.", 
"The controversy surrounding the alleged Lamarckian fraud of Paul Kammerer's midwife toad experiments has intrigued generations of biologists. A re-examination of his descriptions of hybrid crosses of treated and nontreated toads reveals parent-of-origin effects like those documented in epigenetic inheritance. Modification of the extracellular matrix of the egg as described by Kammerer provides a plausible cause for altered gene methylation patterns. Traits such as altered egg and adult body size in Kammerer's treated toads are inherited epigenetically in other tetrapods. A preliminary model involving the environmental silencing of a maternally inherited allele can be attempted to explain the midwife toad experiments. Given available molecular tools and our current understanding of epigenetics, new experimentation with the midwife toad is strongly encouraged. J. Exp. Zool. (Mol. Dev. Evol.) 312B:667-678, 2009. (C) 2009 Wiley-Liss, Inc.", 
"So far, in maize, three classes of mutants involved in phytic acid biosynthesis have been isolated: lpa1, lpa2 and lpa3. In 2007, a gene tagging experiment performed by Shi et al. found that mutations in ZmMRP4 ( multidrug resistance-associated proteins 4) gene cause lpa1 phenotype. In previous studies, we isolated and described a single recessive lpa mutation ( originally named lpa241), which was allelic to the lpa1-1 mutant, and was consequently renamed lpa1-241. It showed a decrease in the expression of the myo-inositol (Ins)-3-phosphate synthase gene (mips1S). In this study, we present genetic and molecular analyses of the lpa1-241 mutation that indicate an epigenetic origin of this trait, that is, a paramutagenic interaction that results in meiotically heritable changes in ZmMRP4 gene expression, causing a strong pleiotropic effect on the whole plant. The use of a 5-Azacytidine treatment provided data suggesting an association between gene methylation and the lpa1-241 phenotype. To our knowledge, this is the first report of a paramutagenic activity not involving flavonoid biosynthesis in maize, but regarding a key enzyme of an important metabolic pathway in plants.", 
"Sex is generally thought of as meiosis, conjugation, and syngamy, with the primary function of sex believed to be genetic mixing. However, conjugation does not occur with complete automixis, whereas syngamy does not occur with restitutional automixis. Self sex in the forms of automixis and autogamy does not include genetic mixing. Yet sex, including self sex, is necessary for most eukaryotic lineages. What is the purpose of sex without genetic mixing? Obligate self sex is not an evolutionary dead end, but holds the key to understanding the evolutionary origin, function, maintenance, and ubiquity of sex. We extend the rejuvenescence hypothesis that sex provides a necessary developmental reset for multicellular eukaryotes and even many unicellular eukaryotes. Sex reduces additive genetic variance of epigenetic signals, especially cytosine methylation, and of ploidy levels. Furthermore, we argue that syngamy is a modified form of meiosis that maintains ploidy and resets epigenetic signals. Epigenetic resetting is consistent with sex being induced by starvation or desiccation. Diminution of additive genetic variance is consistent with the origin and maintenance of an adaptive trait, sex, that has been present for approximately two billion years. (C) 2009 The Linnean Society of London, Biological Journal of the Linnean Society, 2009, 98, 707-728.", 
"5S ribosomal DNA is a highly conserved tandemly repeated multigenic family. As suggested for a long time, we have shown that only a fraction of the 5S rRNA genes are expressed in Arabidopsis thaliana. In Xenopus laevis, there is a developmental control of the expression of the 5S rRNA genes with only one of the two 5S rDNA families expressed during oogenesis. For both Arabidopsis and Xenopus, the strongest transcription of 5S rRNA, respectively in the seed and during oogenesis is correlated with heterogeneity in the transcribed 5S rRNAs. Epigenetic mechanisms such as modification of the chromatin structure are involved in the transcriptional regulation of the 5S rRNA genes in both organisms. In Arabidopsis, two silencing pathways, methylation-dependent (RNAi) and methylationin-dependent (MOM pathway), are involved in the silencing of a 5S rDNA fraction.", 
"The purpose of this study is to test two competing models regarding the origins of Early Intermediate Period (AD 200-750) sacrificial victims from the Huacas de Moche site using the matrix correlation method. The first model posits the sacrificial victims represent local elites who lost competitions in ritual battles with one another while the other model suggests the victims were nonlocal warriors captured during warfare with nearby polities. We estimate biodistances for sacrificial victims from Huaca de la Luna Plaza 3C (AD 300550) with eight previously reported samples from the north coast of Peru using both the mean measure of divergence (MMD) and Mahalanobis' distance (d(2)). Hypothetical matrices are developed based upon the assumptions of each of the two competing models regarding the origins of Moche sacrificial victims. When the MMD matrix is compared to the two hypothetical matrices using a partial-Mantel test (Smouse et al.: Syst Zool 35 (1986) 627-632), the ritual combat model (i.e. local origins) has a low and nonsignificant correlation (r = 0.134, P = 0.163), while the nonlocal origins model is highly correlated and significant (r = 0.688, P = 0.001). Comparisons of the d(2) results and the two hypothetical matrices also produced low and nonsignificant correlation for the ritual combat model (r = 0.210, P = 0.212), while producing a higher and statistically significant result with the nonlocal origins model (r = 0.676, P = 0.002). We suggest that the Moche sacrificial victims represent nonlocal warriors captured in territorial combat with nearby competing polities.", 
"Studies on the origin of species have focused largely on anatomy, yet animal populations are generally limited by energy. Animals can adapt to available energy resources at three levels: (1) evolution of different anatomical forms between groups of animals through nuclear DNA (nDNA) mutations, permitting exploitation of alternative energy reservoirs and resulting in new species with novel niches, (2) evolution of different physiologies within intraspecific populations through mutations in mitochondrial DNA (mtDNA) and nDNA bioenergetic genes, permitting adjustment to energetic variation within a species' niche, and (3) epigenomic regulation of dispersed bioenergetic genes within an individual via mitochondrially generated high-energy intermediates, permitting individual adjustment to environmental fluctuations. Because medicine focuses on changes within our species, clinically relevant variation is more likely to involve changes in bioenergetics than anatomy. This may explain why mitochondrial diseases and epigenomic diseases frequently have similar phenotypes and why epigenomic diseases are being found to involve mitochondrial dysfunction. Therefore, common complex diseases may be the result of changes in any of a large number of mtDNA and nDNA bioenergetic genes or to altered epigenomic regulation of these bioenergetic genes. All of these changes result in similar bioenergetic failure and consequently related phenotypes.", 
"We discuss the changing use of epigenetics, a term coined by Conrad Waddington in the 1940s, and how the epigenetic approach to development differs from the genetic approach. Originally, epigenetics referred to the study of the way genes and their products bring the phenotype into being. Today, it is primarily concerned with the mechanisms through which cells become committed to a particular form or function and through which that functional or structural state is then transmitted in cell lineages. We argue that modern epigenetics is important not only because it has practical significance for medicine, agriculture, and species conservation, but also because it has implications for the way in which we should view heredity and evolution. In particular, recognizing that there are epigenetic inheritance systems through which non-DNA variations can be transmitted in cell and organismal lineages broadens the concept of heredity and challenges the widely accepted gene-centered neo-Darwinian version of Darwinism.", 
"Allopolyploid speciation is widespread in plants, yet the molecular requirements for successful orchestration of coordinated gene expression for two divergent and reunited genomes are poorly understood. Recent studies in several plant systems have revealed that allopolyploid genesis under both synthetic and natural conditions often is accompanied by rapid and sometimes evolutionarily conserved epigenetic changes, including alteration in cytosine methylation patterns, rapid silencing in ribosomal RNA and protein-coding genes, and de-repression of dormant transposable elements. These changes are inter-related and likely arise from chromatin remodeling and its effects on epigenetic codes during and subsequent to allopolyploid formation. Epigenetic modifications could produce adaptive epimutations and novel phenotypes, some of which may be evolutionarily stable for millions of years, thereby representing a vast reservoir of latent variation that may be episodically released and made visible to selection. This epigenetic variation may contribute to several important attributes of allopolyploidy, including functional diversification or subfunctionalization of duplicated genes, genetic and cytological diploidization, and quenching of incompatible inter-genomic interactions that are characteristic of allopolyploids. It is likely that the evolutionary success of allopolyploidy is in part attributable to epigenetic phenomena that we are only just beginning to understand. (C) 2003 Elsevier Inc. All rights reserved.", 
"Novel phenotypes often arise in generations immediately following polyploidization. Previous studies have shown that separate lineages derived from a resynthesized Brassica napus allopolyploid rapidly evolved heritable differences in flowering time. These early-flowering and late-flowering polyploid lines were expected to be genetically identical because they were derived from a single, chromosome-doubled amphihaploid plant. In this study, we investigated the molecular genetic basis for these flowering time differences. We assessed the diploid B. rapa and B. oleracea parents and the early- and late-flowering B. napus lineages for changes in genome structure, and for changes in transcript levels of four sets of FLOWERING LOCUS C (FLC) genes. No such changes were observed for BnFLC1, but we detected chromosomal rearrangements (e.g. de novo non-reciprocal transpositions) and changes in transcript level for BnFLC2 and BnFLC3 between the early- and late-flowering B. nap us. A chromosomal rearrangement of a genomic segment containing BnFLC3 was responsible for 29% of the phenotypic variation among the B. napus lines. Expression of BnFLC5 was silenced in all polyploids, although no changes in genome structure were detected. An ongoing investigation of 50 identical B. napus allopolyploids may further reveal the dynamics of changes in phenotype, genome and transcriptome at the early stages in polyploid evolution. (C) 2004 The Linnean Society of London.", 
"Reconstructing the transition to bipedality is key to understanding early hominin evolution. Because it is the best-known early hominin species, Australopithecus afarensis forms a baseline for interpreting locomotion in all early hominins. While most researchers agree that A. afarensis individuals were habitual bipeds, they disagree over the importance of arboreality for them. There are two main reasons for the disagreement. First, there are divergent perspectives on how to interpret primitive characters. Primitive traits may be retained by stabilizing selection, pleiotropy, or other ontogenetic mechanisms. Alternately, they could be in the process of being reduced, or they simply could be selectively neutral. Second, researchers are asking fundamentally different questions about the fossils. Some are interested in reconstructing the history of selection that shaped A. afarensis, while others are interested in reconstructing A. afarensis behavior. By explicitly outlining whether we are interested in reconstructing selective history or behavior, we can develop testable hypotheses to govern our investigations of the fossils. To infer the selective history that shaped a taxon, we must first consider character polarity. Derived traits that enhance a particular function, are found to be associated with that function in extant homologs, and that epigenetically sensitive data indicate were actually being used for that function, can be interpreted as adaptations. The null hypothesis to explain the retention of primitive traits is that of selective neutrality, or nonaptation. Disproving this requires demonstration of active stabilizing or negative selection (disaptation). Stabilizing selection can be inferred when primitive traits compromise a derived function clearly of adaptive value. Prolonged stasis, continued use of the trait for a particular function, or no change in variability in the trait are evidence that can support a hypothesis of adaptation for primitive traits, but still do not falsify the null hypothesis. Disaptation, or negative selection, should result in a trait being reduced or lost. To infer the behaviors of a fossil species, we must first determine its adaptations, use this to make hypotheses about its behavior, and test these hypotheses using epigenetically sensitive traits that are modified by an individual's activity pattern. When the A. afarensis data are evaluated using this framework, it is clear that these hominins had undergone selection for habitual bipedality, but the null hypothesis of nonaptation to explain the retention of primitive, ape-like characters cannot be falsified at present. The apparent stasis in Australopithecus postcranial form is currently the strongest evidence for stabilizing selection maintaining its primitive features. Evidence from features affected by individual behaviors during ontogeny shows that A. afarensis individuals were habitually traveling bipedally, but evidence presented for arboreal behavior so far is not conclusive. By clearly identifying the questions we are asking about early hominin fossils, refining our knowledge about character polarities, and elucidating the factors influencing morphology, we will be able to progress in our understanding of the posture and locomotion of A. afarensis and all early hominins. Am J Phys Anthropol 45:185-215, 2002. (C) 2002 Wiley-Liss, Inc.", 
"The idea of genetic assimilation, that environmentally induced phenotypes may become genetically fixed and no longer require the original environmental stimulus, has had varied success through time in evolutionary biology research. Proposed by Waddington in the 1940s, it became an area of active empirical research mostly thanks to the efforts of its inventor and his collaborators. It was then attacked as of minor importance during the hardening of the neo-Darwinian synthesis and was relegated to a secondary role for decades. Recently, several papers have appeared, mostly independently of each other, to explore the likelihood of genetic assimilation as a biological phenomenon and its potential importance to our understanding of evolution. In this article we briefly trace the history of the concept and then discuss theoretical models that have newly employed genetic assimilation in a variety of contexts. We propose a typical scenario of evolution of genetic assimilation via an intermediate stage of phenotypic plasticity and present potential examples of the same. We also discuss a conceptual map of current and future lines of research aimed at exploring the actual relevance of genetic assimilation for evolutionary biology.", 
"In the past few years we have analysed alterations in genome structure and expression that occur in wheat upon allopolyploidization. Our major findings in natural and synthetic allopolyploid wheat are reviewed here. It was found that allopolyploidization brings about rapid genome evolution through the instantaneous generation of a variety of cardinal genetic and epigenetic alterations comprising: (1) non-random elimination of coding and non-coding DNA sequences, (2) epigenetic changes such as DNA methylation of coding and non-coding DNA leading, among others, to gene silencing, and (3) activation of retroelements, which in turn alters the expression of adjacent genes. These changes were reproducible, occurring in the F1 hybrids or in the first generation(s) of a series of nascent allopolyploids corresponding to various interspecific and intergeneric combinations. Moreover, these changes were similar to those that occurred twice in nature: first, at the transition from diploid to tetraploid wheat (similar to0.5 Mya) and, second, at the transition from tetraploid to hexaploid wheat (similar to9500 years ago). Elimination of non-coding sequences augments the differentiation of homoeologous chromosomes at the polyploid level, thus increasing the physical divergence between homoeologues and contributing to the diploid-like meiotic behaviour of polyploid wheat. Transcriptional and post-transcriptional alterations of gene activity, including transcriptional activation of retroelements, led to novel expression patterns. These phenomena emphasize the plasticity of the genome with regard to both structure and gene expression. This plasticity in turn might improve the adaptability of the newly formed allopolyploids and facilitate their rapid and successful establishment in nature. (C) 2004 The Linnean Society of London.", 
"One continuous thread in this volume is the name of Conrad H. Waddington (1905-1975), the developmental biologist known as the inventor of the term epigenetics. After some biographical notes on his life, this article explores the meaning of the Waddingtonian equation and the context wherein it was developed. This equation holds that epigenesis + genetics = epigenetics, and refers in retrospect to the debate on epigenesis versus preformationism in neoclassical embryology. Whereas Waddington actualized this debate by linking epigenesis to developmental biology and preformation to genetics, thereby stressing the importance of genetic action in causal embryology, today's epigenetics more and more offers the possibility to enfeeble biological thinking in terms of genes only, as it expands the gene-centric view in biology by introducing a flexible and pragmatically oriented hierarchy of crucial genomic contexts that go beyond the organism.", 
"In plants, naturally occurring methylation of genes can affect the level of gene expression. Variation among individuals in the degree of methylation of a gene, termed epialleles, produces novel phenotypes that are heritable across generations. To date, ecologically important genes with methylated epialleles have been found to affect floral shape, vegetative and seed pigmentation, pathogen resistance and development in plants. Currently, the extent to which epiallelic variation is an important common contributor to phenotypic variation in natural plant populations and its fitness consequences are not known. Because epiallele phenotypes can have identical underlying DNA sequences, response to selection on these phenotypes is likely to differ from expectations based on traditional models of microevolution. Research is needed to understand the role of epialleles in natural plant populations. Recent advances in molecular genetic techniques could enable population biologists to screen for epiallelic variants within plant populations and disentangle epigenetic from more standard genetic sources of phenotypic variance, such as additive genetic variance, dominance variance, epistasis and maternal genetic effects.", 
"Spartina anglica arose during the end of the 19th century in England by hybridization between the indigenous Spartina maritima and the introduced East American Spartina alterniflora and following genome duplication of the hybrid (S. x townsendii). This system allows investigations of the early evolutionary changes that accompany stabilization of a new allopolyploid species in natural populations. Various molecular data indicate that S. anglica has resulted from a unique parental genotype. This young species contains two distinctly divergent homoeologous genomes that have not undergone extensive change since their reunion. No burst of retroelements has been encountered in the F-1 hybrid or in the allopolyploid, suggesting a 'structural genomic stasis' rather than 'rapid genomic changes'. However, modifications of the methylation patterns in the genomes of S. x townsendii and S. anglica indicate that in this system, epigenetic changes have followed both hybridization and polyploidization. (C) 2004 The Linnean Society of London.", 
"We investigated the. randomness and uniqueness of human iris patterns by mathematically comparing 2.3 million different pairs of eve images. The phase structure of each iris pattern was extracted by demodulation with quadrature wavelets spanning several scales of analysis. The resulting distribution of phase sequence variation anion different eyes was precisely binomial, revealing 244 independent degrees of freedom. This amount of statistical variability corresponds to an entropy (information density) of about 3.2 bits mm(-2) over the iris. It implies that the probability of two different irides agreeing by chance in more than 70% of their phase sequence is about one in 7 billion. We also compared images of genetically identical irides, from the left and right eyes of 324 persons, and from monozygotic twins. Their relative phase sequence variation generated the same statistical distribution as did unrelated eyes. This indicates that apart from overall form and colour. iris patterns are determined epigenetically by random events in the morphogenesis of this tissue. The resulting diversity, and the combinatorial complexity created by so many dimensions of random variation, mean that the failure of a simple test of statistical independence performed on iris patterns can serve as a reliable rapid basis for automatic personal identification.", 
"The study of evolution has increasingly incorporated considerations of history, scale, and hierarchy, in terms of both the origin of variation and the sorting of that variation. Although the macroevolutionary exploration of developmental genetics has lust begun, considerable progress has been made in understanding the origin of evolutionary novelty in terms of the potential for coordinated morphological change and the potential for imposing uneven probabilities on different evolutionary directions. Global or whole-organism heterochrony, local heterochrony (affecting single structures, regions, or organ systems) and heterotopies (changes in the location of developmental events), and epigenetic mechanisms (which help to integrate the developing parts of an organism into a functional whole) together contribute to profound nonlinearities between genetic and morphologic change, by permitting the generation and accommodation of evolutionary novelties without pervasive, coordinated genetic changes; the limits of these developmental processes are poorly understood, however. The discordance across hierarchical levels in the production of evolutionary novelties through time, and among latitudes and environments, is an intriguing paleontological pattern whose explanation is controversial, in part because separating effects of genetics and ecology has proven difficult. At finer scales, species in the fossil record tend to be static over geologic time, although this stasis-to which there are gradualistic exceptions-generally appears to be underlain by extensive, nondirectional change rather than absolute invariance. Only a few studies have met the necessary protocols for the analysis of evolutionary tempo and mode at the species level, and so the distribution of evolutionary patterns among clades, environments, and modes of life remains poorly understood. Sorting among taxa is widely accepted in principle as an evolutionary mechanism, but detailed analyses are scarce; if geographic range or population density can be treated as traits above the organismic level, then the paleontological and macroecological literature abounds in potential raw material for such analyses. Even if taxon sorting operates on traits that are not emergent at the species level, the differential speciation and extinction rates can shape large-scale evolutionary patterns in ways that are not simple extrapolations from shortterm evolution at the organismal level. Changes in origination and extinction rates can evidently be mediated by interactions with other clades, although such interactions need to be studied in a geographically explicit fashion before the relative roles of biotic and physical factors can be assessed. Incumbency effects are important at many scales, with the most dramatic manifestation being the postextinction diversifications that follow the removal of incumbents. However, mass extinctions are evolutionarily important not only for the removal of dominant taxa, which can occur according to rules that differ from those operating during times of lower extinction intensity, but also for the dramatic diversifications that follow upon the removal or depletion of incumbents. Mass extinctions do not entirely reset the evolutionary clock, so survivors can exhibit unbroken evolutionary continuity trends that suffer setbacks but then resume, or failure to participate in the recovery.", 
"It is well established that asexually reproducing viruses and prokaryotes mutate rapidly. In contrast, the eukaryotic clone is often still treated as if it is genetically homogeneous within and between populations, i.e. that it is assumed to show genetic fidelity However, such fidelity has rarely been tested empirically using the range of high-resolution molecular markers now available, culminating with direct sequencing of the DNA. If such a biological entity as a 'clone' really did exist, it would be a fantastic entity, differing from everything else known in biology, i.e. it would possess a population mean but no variance for any particular trait. It would not be amenable to selection and adaptive variation and would thus be unchanging in time and space. In this paper, we argue that the general acceptance of clonal fidelity is a scientific convenience, since the rate of asexual reproduction of eukaryotes is not as fast as that of bacteria and hence it is easier to accept fidelity as a 'fact' rather than test for it. We propose that part of the acceptance of fidelity may have a cultural basis and thereby is a kind of 'pre-Darwinian relic'. Instead, a clonal genotype is perhaps largely a function of marker resolution, i.e. dependent on the number and type of markers employed. If this is so and were enough of the genome explored, perhaps each individual within a clone would be found to differ genetically at particular regions of the chromosomes. The question of what constitutes a clone is not just a semantic one and impacts directly on recent attempts to understand and produce 'artificial' clones, especially of mammals. New research is already confirming that mutations and epigenetic influences play a crucial role in the success of cloning attempts. (C) 2003 The Linnean Society of London.", 
"We have examined the methylation status of the zebrafish genome during early embryogenesis and we find evidence that methylation fluxes do occur in that organism. The parental genetic contributions to the zygote are, initially, differently methylated with the genome of the sperm being hypermethylated relative to the genome of the oocyte. Post-fertilization there is an immediate decrease in methylation of the embryonic genome but the methylation begins to increase rapidly and is re-established by the gastrulation stage. These results are consistent with the results of Santos et al. (Dev Biol 241:172-182, 2002), who examined the methylation of early mouse embryos, and this conservation argues that demethylation/re-methylation is an important part of vertebrate development.", 
"The crop species within the genus Brassica have highly replicated genomes. Three base 'diploid' species, Brassica oleracea, B. nigra and B. rapa, are likely ancient polyploids, and three derived allopolyploid species, B. carinata, B. juncea and B. napus, are created from the interspecific hybridization of these base genomes. The base Brassica genome is thought to have hexaploid ancestry, and both recent and ancient polyploidization events have been proposed to generate a large number of genome rearrangements and novel genetic variation for important traits. Here, we revisit and refine these hypotheses. We have examined the B. oleracea linkage map using the Arabidopsis thaliana genome sequence as a template and suggest that there is strong evidence for genome replication and rearrangement within the base Brassicas, but less evidence for genome triplication. We show that novel phenotypic variation within the base Brassicas can be achieved by replication of a single gene, BrFLC, that acts additively to influence flowering time. Within the derived allopolyploids, intergenomic heterozygosity is associated with higher seed yields. Some studies have reported that de novo genomic variation occurs within derived polyploid genomes, whereas other studies have not detected these changes. We discuss reasons for these different findings. Large translocations and tetrasomic inheritance can explain some but not all genomic changes within the polyploids. Transpositions and other small-scale sequence changes probably also have contributed to genomic novelty. Our results have shown that the Brassica genomes are remarkably plastic, and that polyploidy generates novel genetic variation through gene duplication, intergenomic heterozygosity and perhaps epigenetic change. (C) 2004 The Linnean Society of London.", 
"Gene duplication is commonly regarded as the main evolutionary path toward the gain of a new function. However, even with gene duplication, there is a loss-versus-gain dilemma: most newly born duplicates degrade to pseudogenes, since degenerative mutations are much more frequent than advantageous ones. Thus, something additional seems to be needed to shift the loss versus gain equilibrium toward functional divergence. We suggest that epigenetic silencing of duplicates might play this role in evolution. This study began when we noticed in a previous publication (Lynch M, Conery JS [2000] Science 291:1151-1155) that the frequency of functional young gene duplicates is higher in organisms that have cytosine methylation (H. sapiens, M. musculus, and A. thaliana) than in organisms that do not have methylated genomes (S. cerevisiae, D. melanogaster, and C. elegans). We find that genome data analysis confirms the likelihood of much more efficient functional divergence of gene duplicates in mammals and plants than in yeast, nematode, and fly. We have also extended the classic model of gene duplication, in which newly duplicated genes have exactly the same expression pattern, to the case when they are epigenetically silenced in a tissue- and/or developmental stage-complementary manner. This exposes each of the duplicates to negative selection, thus protecting from pseudogenization. Our analysis indicates that this kind of silencing (i) enhances evolution of duplicated genes to new functions, particularly in small populations, (ii) is quite consistent with the subfunctionalization model when degenerative but complementary mutations affect different subfunctions of the gene, and (iii) furthermore, may actually cooperate with the DDC (duplication-degeneration-complementation) process.", 
"Aphid life cycles can encompass cyclical parthenogenesis, obligate parthenogenesis, obligate parthenogenesis with male production and an intermediate 'bet-hedging' strategy where an aphid genotype will over-winter by continuing to reproduce by parthenogenesis and by investment in sexually produced eggs. In this paper, we focus on aphid lineages that reproduce entirely parthenogenetically (asexual aphids), in contrast to those that have any sexual forms in the annual cycle. Using modern molecular techniques, aphid biologists have made many empirical observations showing that asexual lineages are widespread both geographically and temporally. Indeed, we are collectively beginning to gather data on the evolution and persistence of these lineages through time. Here we review aphid karyology and parthenogenesis, both essential for interpretation of the molecular and ecological evolution of aphid asexual lineages. We describe the growing list of studies that have identified aphid genotypes that are both temporally and geographically widespread. We then collate examples of molecular and chromosomal evolution in asexual aphids and review the literature pertaining to phenotypic evolution and ecological diversification of asexual aphid lineages. In addition, we briefly discuss the potential of bacterial endosymbionts and epigenetic effects to influence the evolution of asexual aphid lineages. Lastly we provide a list of aphid taxa believed to be obligately asexual. This will be a useful resource for those seeking parthenogenetic animals as study systems. In conclusion, we present guidelines for the use of the term clone in aphid biology and stress the need for well-designed and well-executed studies examining the potential of asexual aphid lineages for adaptive evolution. (C) 2003 The Linnean Society of London.", 
"We suggest simulating evolution of complex organisms using a model constrained solely hv the requirement of robustness in its expression patterns. This scenario is illustrated by evolving discrete logical networks with epigenetic properties. Evidence for dynamical features in the evolved networks is found that carl be related to biological observables.", 
"Several models of speciation suggest that in species that are phenotypically plastic, selection can act on phenotypic variation that is environmentally induced in the earliest stages of divergence. One trait that could be subject to this process is foraging behaviour, where discrete foraging strategies are common. One species which is highly plastic in the expression of phenotype, the Arctic charr, Salvelinus alpinus (L.), is characterized by discrete variation in the anatomy of the head and mouthparts. These traits have been shown to have a functional significance, but the expression of which is thought to be at least partly phenotypically plastic. Here we test the hypothesis that foraging behaviour may regulate the anatomy of the head and mouthparts in Arctic charr. In a dyad experiment, size-matched pairs of fish from a mixed family group were fed a diet of either Mysis (a hard-bodied shrimp) or Chironomid larvae. Nine morphometric measures of head dimensions that describe wild trophic morphs were measured at the start of the experiment and 24 weeks later. Principal component scores of size-corrected morphometric measures showed highly significant differences between fish exposed to the two diets. Univariate ANOVA analysis of the head morphometric variables showed that fish fed on Chironomids developed longer, wider jaws, longer heads and a larger eye for a given body length than did those fish fed upon Mysis. We conclude that foraging anatomy in Arctic charr is phenotypically plastic and that variation in foraging behaviour that results in feeding specialization in the wild could induce variation in head anatomy. This in turn could reinforce foraging specialization. Very rapid epigenetic divergence into distinct feeding morphs (as demonstrated here) would allow selection to act at more than one mode and thus could promote rapid evolutionary divergence, initially prior to genetic segregation, in species which are highly plastic. (C) 2003 The Linnean Society of London, Biological Journal of the Linnean Society, 2003, 78, 43-49.", 
"In gnathostomes, the adenohypophysis, a component of the hypothalamo-hypophysial complex, is believed to develop through hierarchically organized epigenetic interactions based primarily on the topographical relationships between tissues. From a comparison of developmental processes and gene expression patterns of pituitary-related genes between the agnathan species, lampreys and gnathostomes, we speculate on the evolutionary pathway of the vertebrate adenohypophysis. In the lamprey, this is derived from the nasohypophysial placode (NHP) that develops anterior to the oral ectoderm. The NHP can be identified by the expression of LjPitxA, before actual histogenesis, but it is initially distant from the future hypothalamic region. Subsequently, the NHP expresses both LjFgf8/17 and LjBmp2/4a gene transcripts, and grows caudally to establish a de novo contact with the hypothalamic region by the mid-pharyngula stage. Later, the NHP gives rise to both the adenohypophysis and an unpaired nasal organ. Thus, the topographical relationship between the NHP and the hypothalamic region is established secondarily in the lamprey, unlike gnathostomes in which the equivalent relationship appears early in development. Comparing the developmental pattern of the amphioxus homologue of the adenohypophysis, we hypothesize that a modification of the regulation of the growth factor encoding gene lies behind the evolutionary changes recognized as heterochrony and heterotopy, which leads to the gnathostome hypophysial developmental pattern. (C) 2003, Wiley-Liss, Inc.", 
"In mammals and birds the genetic constitution established at the time of fertilization determines the type of gonad that develops, whereas in all crocodilians and many turtles it is the temperature experienced during the mid-trimester of embryogenesis that initiates gonadal differentiation. Research with the red-eared slider suggests considerable conservation in the genetic cascades that underlie the sex determination process in vertebrates and, further, that the patterns of expression of these genes appear to reflect phylogenetic relationships, with turtles being more similar to mammals than they are to birds and crocodilians. After the determination and differentiation of an individual's gonadal sex, epigenetic forces shape those morphological, physiological, and behavioral traits that characterize each individual's unique sexuality. Research with the leopard gecko thus relates to the fundamental question of what factors determine individual variability, particularly as it relates to sexually dimorphic behaviors. Taken together, this research illustrates how sexuality depends on sex, but sex should not be confused with sexuality. That is, sex is merely a means of categorizing individuals or gonads, whereas sexuality serves as a descriptor of concordant traits each of which is typically sexually dimorphic in its expression.", 
"Saccharomyces cerevisiae's ability to form the prion [PSI+] may increase the rate of evolvability, defined as the rate of appearance of heritable and potentially adaptive phenotypic variants. The increase in evolvability occurs when the appearance of the prion causes read-through translation and reveals hidden variation in untranslated regions. Eventually the portion of the phenotypic variation that is adaptive loses its dependence on the revealing mechanism. The mechanism is reversible, so the restoration of normal translation termination conceals the revealed deleterious variation, leaving the yeast without a permanent handicap. Given that the ability to form [PSI+] is known to be fixed and conserved in yeast, we construct a mathematical model to calculate whether this ability is more likely to have become fixed due to chance alone or due to its evolvability characteristics. We find that evolvability is a more likely explanation, as long as environmental change makes partial read-through of stop codons adaptive at a frequency of at least once every million years.", 
"Arabidopsis is a model system not only for studying numerous aspects of plant biology, but also for understanding mechanisms of the rapid evolutionary process associated with genome duplication and polyploidization. Although in animals interspecific hybrids are often sterile and aneuploids are related to disease syndromes, both Arabidopsis autopolyploids and allopolyploids occur in nature and can be readily formed in the laboratory, providing an attractive system for comparing changes in gene expression and genome structure among relatively 'young' and 'established' or 'ancient' polyploids. Powerful reverse and forward genetics in Arabidopsis offer an exceptional means by which regulatory mechanisms of gene and genome duplication may be revealed. Moreover, the Arabidopsis genome is completely sequenced; both coding and non-coding sequences are available. We have developed spotted oligo-gene and chromosome microarrays using the complete Arabidopsis genome sequence. The oligo-gene microarray consists of -26 000 70-mer oligonucleotides that are designed from all annotated genes in Arabidopsis, and the chromosome microarray contains 1 kb genomic tiling fragments amplified from a chromosomal region or the complete sequence of chromosome 4. We have demonstrated the utility of microarrays for genome-wide analysis of changes in gene expression, genome organization and chromatin structure in Arabidopsis polyploids and related species. (C) 2004 The Linnean Society of London.", 
"Historically, interpretations of both biological and cultural change within the prehistoric Azapa Valley, northern Chile, have cited large-scale population movements, with replacement from complex societies located in the adjacent highlands to the east. Biological estimates of this change have traditionally relied upon biodistance estimates, using craniofacial measures of both deformed and nondeformed skulls. In order to evaluate whether large-scale prehistoric migrations occurred in the Azapa Valley, we examine biodistance results from non-metric cranial traits for eight mortuary samples that represent all time periods of prehistoric occupation of the valley. None of the mean measures of divergences (MMD) among mortuary samples examined by this study were significant. These results suggest biological continuity in the Azapa Valley during 5,000 years of prehistory, with nonsignificant gene flow during the late Middle Horizon (AD 750-1100) and Late Intermediate (AD 1100-1476) periods. Biodistance results also suggest endogamy within coastal and inland populations. The broader implications of these results are discussed. (C) 2004 Wiley-Liss, Inc.", 
"Why are there two sexes in certain species, instead of one hermaphroditic sex? Why are Y chromosomes shorter than X chromosomes, but only in certain lineages? I propose that differences between sexes are initially determined by differential methylation in nuclear DNA between females and males, driving Muller's ratchet. Methylation of promoters suppresses transcription, including loci coding for gamete production, thereby converting hermaphroditic individuals into females or males. Differential methylation of sex chromosomes suppresses recombination and increases mutation rate, thereby geometrically increasing the speed of Muller's ratchet. Higher mutability of methylated nucleotides plus loss of sex-determining function of previously methylated nucleotides provides selective pressure to excise these loci, resulting in shorter Y or W chromosomes. Derived lineages usually have more methylation than do ancestral ones, and hence have relatively shorter sex chromosomes. Methylation canalizes dioecy and degeneration of sex chromosomes. Latter stages of sex chromosome evolution may have occurred via other mechanisms, for example sexually antagonistic genes or chromosomal rearrangements. A few aberrant derived lineages lost most methylation, and their sex determination and sex chromosomes may have evolved via other means. Differential methylation provides a mechanism for early evolution of dioecy in anisogamous sexual diploid eukaryotes and of sex chromosomes in metazoans. (C) 2003 The Linnean Society of London.", 
"Efforts to test hypotheses about small-scale shifts in development (tinkering) that can only be observed in the fossil record pose many challenges. Here we use the origin of modern human craniofacial form to explore a series of analytical steps with which to propose and test evolutionary developmental hypotheses about the basic modules of evolutionary change. Using factor and geometric morphometric analyses of craniofacial variation in modern humans, fossil hominids, and chimpanzee crania, we identify several key shifts in integration (defined as patterns of covariation that result from interactions between components of a system) among units of the cranium that underlie the unique shape of the modern human cranium. The results indicate that facial retraction in modern humans is largely a product of three derived changes: a relatively longer anterior cranial base, a more flexed cranial base angle, and a relatively shorter upper face. By applying the Atchley-Hall model of morphogenesis, we show that these shifts are most likely the result of changes in epigenetic interactions between the cranial base and both the brain and the face. Changes in the size of the skeletal precursors to these regions may also have played some role. This kind of phenotype-to-genotype approach is a useful and important complement to more standard genotype-to-phenotype approaches, and may help to identify candidate genes involved in the origin of modern human craniofacial form. (C) 2004 Wiley-Liss, Inc.", 
"Epigenesis concerns the interactions through which the inherited potentials of the genome become actualized into an adult organism. In addition to epigenetic interactions occurring within the developing embryo, there are also critical epigenetic interactions occurring between the embryo and its environment. These interactions can determine the sex of the embryo, increase its fitness, or even be involved in the formation of particular organs. This essay will outline the history of environmental concerns in developmental biology and provide some reasons for the decline and resurgence of these ideas, and it will then focus on two areas that have recently gained much attention: predator-induced polyphenisms and developmental symbioses. Research in these two areas of interspecies cooperation in morphogenesis has profound implications for what we consider to be normal development and how we proceed to study it. Studies of predator-induced polyphenism have shown that soluble factors from predators can change the development of prey in specific ways. Prey has evolved mechanisms to sense compounds released from their predators and to use these chemical cues to change their development in ways that prevent predation. New techniques in molecular biology, especially polymerase chain reaction and microarray analysis, have shown that symbioses between embryos and bacteria are widespread and that animals may use bacterial cues to complete their development.", 
"Development is the process by which genotypes are transformed into phenotypes. Consequently, development determines the relationship between allelic and phenotypic variation in a population and, therefore, the patterns of quantitative genetic variation and covariation of traits. Understanding the developmental basis of quantitative traits may lead to insights into the origin and evolution of quantitative genetic variation, the evolutionary fate of populations, and, more generally, the relationship between development and evolution. Herein, we assume a hierarchical, modular structure of trait development and consider how epigenetic interactions among modules during ontogeny affect patterns of phenotypic and genetic variation. We explore two developmental models, one in which the epigenetic interactions between modules result in additive effects on character expression and a second model in which these epigenetic interactions produce nonadditive effects. Using a phenotype landscape approach, we show how changes in the developmental processes underlying phenotypic expression can alter the magnitude and pattern of quantitative genetic variation. Additive epigenetic effects influence genetic variances and covariances, but allow trait means to evolve independently of the genetic variances and covariances, so that phenotypic evolution can proceed without changing the genetic covariance structure that determines future evolutionary response. Nonadditive epigenetic effects, however, can lead to evolution of genetic variances and covariances as the mean phenotype evolves. Our model suggests that an understanding of multivariate evolution can be considerably enriched by knowledge of the mechanistic basis of character development.", 
"I have tried to unpack the polysemy of the word epigenetics by adopting a historical point of view and by focusing on the models that were proposed at the beginning of the 1960s to explain variations in gene Activity during cell differentiation and development. Most of the questions that were or are at the core of epigenetics were posed in this period. This was due to the fact that the regulatory models and their extension to the notion of the genetic program were proposed as genetic answers to the questions raised by Waddington when he defined epigenetics in the 1940s. Studies of DNA methylation and chromatin structure, which became increasingly important in the 1960s and 1970s, were seen as alternative explanations to the regulatory mechanisms that had been previously proposed. This historical detour shows that epigenetics cannot be defined per se, but only as an evolving opposition to the piecemeal, reductionist approach of genetics.", 
"Genetic assimilation occurs when an acquired trait loses dependency on its environmental trigger and becomes an inherited trait. According to the standard quantitative genetic model for genetic assimilation, the trait is determined by the contributions of multiple genes. Trait expression occurs at a lower threshold with the trigger. Selection for the trait in the presence of the trigger increases the frequency of the trait-determining alleles. Eventually these alleles become frequent enough to breach the higher threshold for expression in the absence of the trigger. This loss of dependence on the trigger signifies genetic assimilation. Here I show that genetic assimilation can occur in the absence of selection for the trait in an evolutionary simulation of a gene network model. This contradicts the prediction of the standard quantitative genetic model, but is consistent with an explanation in terms of the canalization heuristic.", 
"Insecticide treatment of the aphid Myzus persicae (Sulzer) has led to the evolution of several insecticide resistance mechanisms, including the detoxification of insecticides by elevated esterases. This results from amplification of one of two closely related esterase genes (E4 or FE4) with up to 80 copies in the most resistant aphids. The amplified E4 genes are at a single site linked to a chromosomal translocation and resistance can be unstable. Individuals within a clone lose their elevated esterase and resistant phenotype, a good example of 'clonal variation'. This loss of esterase is accompanied by a loss of the corresponding mRNA but the amplified genes are retained with no detectable sequence differences. However, the expressed E4 genes contain 5-methylcytosine, which is lost at the same time as the genes are turned off. This is in direct contrast with vertebrate genes where DNA methylation causes gene silencing, but it does suggest that the resistant phenotype in M. persicae is under epigenetic control. One hypothesis is that 5-methylcytosine in E4 genes facilitates expression by preventing the production of incorrectly initiated transcripts. It is interesting that we have never detected silencing of amplified FE4 genes, possibly because they are at multiple loci and therefore less likely to be subject to synchronous control. (C) 2003 The Linnean Society of London.", 
"The value of quantitative intracranial nonmetric variation is examined in the study of population relationships by using samples from populations originating from five major geographic regions: Australia (two populations), Africa, East Asia, Europe, and Polynesia. According to the nonspecificity hypothesis, there are no distinct large classes of genes affecting one group of attributes exclusively; thus intracranial nonmetric traits should compare with other osteologic data sets in addressing questions of population relationships. By using the mean measure of divergence, intracranial nonmetric traits are shown to be useful in separating populations, particularly when using female and pooled-sex samples. The two Australian female samples (New South Wales coastal Australian and South Australian Aboriginals) are shown to be closer than any other two samples. The picture of intrapopulation and interpopulation variation in intracranial nonmetric traits is extended and clarified. Distance studies with intracranial nonmetric traits are possible but more illuminating if the sexes are first separated. Intracranial nonmetric variation does extend the knowledge of human population studies in yielding biologically meaningful results relating to development and ontogeny. (C) 2000 Wiley-Liss, Inc.", 
"What counts as epigenetic depends on what counts as genetic. It is argued that Weismannism, the doctrine of genetic continuity and somatic discontinuity, is the basis for an overly inclusive concept of epigenetics as every inherited resource beyond the genes. An alternative theoretical perspective, the reproducer concept, is introduced to facilitate analysis of multiple inheritance systems without labeling all nongenetic inheritance epigenetic..", 
"Given the quantity of molecular data now available, including complete genomes for some organisms. one can ask whether there is a need for any data beyond complete genomic sequences for phylogenetic analysis. One reason to look beyond the genome is that not all character information is encoded in organismal genomes. We propose a hierarchy of characters that ranges from biologically transmitted but nongenomically encoded characters, such as bird songs, to characters that are genomically encoded. All of these characters can retain historical information and are potentially useful for phylogenetic analysis. In addition, a number of phenotypic levels that are expressions of the genome can be identified. The question whether it is worth including any of these levels if all of the underlying sequence data have been collected arises, since issues of redundancy occur. Utilization of phenotypic levels that are ultimately based on sequences may facilitate reconstructing homologies that are not evident from sequence data alone. We propose the use of simultaneous analysis of sequence data and as many levels of phenotypic characters as possible to take advantage of homology information that may be more easily recovered from the latter. A method that eliminates redundancy to the degree that it can be detected is proposed. (C) 2003 The Willi Hennig Society. Published by Elsevier Inc. All rights reserved.", 
"In this paper, it is argued that differences in how one relates the genome to its surrounding contexts leads to diverse interpretations of the term epigenetics. Three different approaches are considered, ranging from gene-centrism, over gene-regulation, to dynamic systems approaches. Although epigenetics receives its widest interpretation in a systems approach, a paradigmatic shift has taken place in biology from the abandonment of a gene-centric position on to the present. The epistemological and ontological consequences of this shift are made explicit.", 
"To understand the evolution of Fot1, a member of the pogo family widely dispersed in ascomycetes, we have performed a phylogenetic survey across the genus Fusarium divided into six sections, The taxonomic distribution of Fot1 is not homogeneous but patchy; it is prevalent in the Fusarium oxysporum complex. absent in closely related sections, and found in five species from the most distant section Martiella. Multiple copies of Fot1 were sequenced from each strain in which the element occurs. In three species. the Fot1 nucleotide sequence is 98% identical to that from F. oxysporum (Fox), whereas nucleotide divergence for host genes is markedly higher: 11% for partial nuclear 28S rDNA and up to 30% for the gene encoding nitrate reductase (nia). In two species, sequence divergence of Fot1-related elements relative to Fox ranged from 7% to 23% (16% average). Most of the sequence differences (82%) were C-to-T and G-to-A transitions. These mutations are distributed throughout the Fot1 sequences, although they tend to be concentrated in the middle portion of the elements. Analysis of the local sequence context of transitions revealed a hierarchy of site preferences. These characteristics are typical of the repeat-induced point mutation process. first discovered in Neurospora crassa. The spotty distribution of Fot1 elements among species together with the high degree of similarity between Fot1 sequences present in distant species strongly suggests a case of horizontal transfer.", 
"In rainbow trout (Oncorhynchus mykiss), the acknowledged sex-determining system is genetic sex determination (GSD) with female homogamety (female XX-male XY). Subsequently, mitotic gynogens are all expected to be females. Unexpected maleness was fortuitously observed in a mitotic gynogenetic family of rainbow trout (13 males out of 27). An equal ratio of males and females suggested the possible segregation of some Mendelian sex-influencing factor. In order to perform a comprehensive analysis of the inheritance and expression of the factor involved, the transmission of maleness was studied across the next three generations, using both conventional and/or meiotic and mitotic gynogenetic offspring. On the whole, males as well as intersexes were observed in crosses between two expected carrier parents, and in gynogenetic offspring of expected carrier females, but not in crosses between one expected carrier parent and one normal XX control. Sex ratios in the different crosses often fitted Mendelian proportions, but not always. Both excess and lack of maleness were observed. The simplest hypothesis consistent with most results is a one-locus model, assuming the existence of a mutation (termed man of a sex-determining gene, which is able to override the primary XX mechanism of sex determination and to induce the development of testicular tissue in the gonads of expected XX individuals. The one-locus model requires that the mal mutation usually, but not systematically, behave as a recessive mutation and have a limited penetrance, that is, heterozygous (mal/+) may be sex reversed, homozygous (mal/mal may remain female, and carrier individuals may undergo partial masculinization alone (many intersexes were recorded). Inconsistency in sex ratios among offspring of parents expected to respond the same way was recorded, indicating that other modifier loci may also be involved. Finally, the occurrence of both males and females in clonal progenies showed that epigenetic factors also likely influence the expression of maleness. The effects of the mal mutation are compared to similar mutations recently described in other fish species. The nature and location of the mal gene (carried by heterochromosomes or an autosomal pair) is briefly discussed in view of the knowledge recently acquired on the subject.", 
"This volume deals with the role of epigenetics in life and evolution. The most dynamic forms of functional genome formatting involve DNA interacting with cellular complexes that do not alter sequence information. Such important epigenetic phenomena are the main subjects of other articles in this volume. This article focuses on the long-lived form of genome formatting that lies within the DNA sequence itself. I argue for a computational view of genome function as the long-term information storage organelle of each cell. Structural formatting consists of organizing various signals and coding sequences into computationally ready systems facilitating genome expression and genome transmission. The basic features of genome organization can be understood by examining the E. coli lac operon as a paradigmatic genomic system. Multiple systems are connected through distributed signals and repetitive DNA to form higher-order genome system architectures. Molecular discoveries about mechanisms of DNA restructuring show that cells possess the natural genetic engineering functions necessary for evolutionary change by rearranging genomic components and reorganizing system architectures. The concepts of cellular computation and decision-making, genome system architecture, and natural genetic engineering combine to provide a new way of framing evolutionary theories and understanding genome sequence information.", 
"The modified base 5-methylcytosine (C-m) plays an important functional role in the biology of mammals as an epigenetic modification and appears to exert a striking impact on the molecular evolution of mammal genomes. The collective epigenetic functions of C-m revolve around its effect on gene transcription, while the influence of this modified base on the evolution of mammal genomes derives from the greatly elevated spontaneous mutation rate of C-m to T. In mammals, C-m occurs at the dinucleotides CpG, CpA, and CpT. As a step toward a comprehensive statistical examination of the role of C-m in mammal molecular evolution, we have developed novel Markov models of codon substitution that incorporate dinucleotide-level terms relevant to C-m mutation. We apply these models to two data sets of aligned BRCA1 exon 11 sequences from bats and primates. In all cases, terms specific to mutations that affect the dinucleotides CpG, CpA, and CpT significantly improved model fit. For the CpG-specific terms, both transition and transversion substitution rates were elevated. These rates differed between the data sets. Bats exhibited a lower relative rate of substitutions at CpG-containing codons. Transition substitutions were significantly less than 1 at CpA-containing codons but greater than 1 at CpT-containing codons. The inclusion of interaction terms in the codon models to represent possible confounding with the effect of natural selection were supported for codons that contained CpG and CpT, but not CpA. From the results, we infer that mutation of C-m is a probable factor that affects BRCA1 codons containing the dinucleotide CpG, a possible factor for CpA-containing codons, and an unlikely factor that affects CpT-containing codons. The confounding of estimated terms with the effect of natural selection indicate this confounding must be addressed for comparisons between different coding and noncoding regions.", 
"Repetitive DNA sequences comprise a substantial portion of most eukaryotic and some prokaryotic chromosomes. Despite nearly forty years of research, the functions of various sequence families as a whole and their monomer units remain largely unknown. The inability to map specific functional roles onto many repetitive DNA elements (RIES), coupled with the taxon-specificity of sequence families, have led many to speculate that these genomic components are selfish replicators generating genomic junk. The purpose of this paper is to critically examine the selfishness, evolutionary effects, and functionality of REs. First, a brief overview of the range of ideas pertaining to RE function is presented. Second, the argument is presented that the selfish DNA hypothesis is actually a narrative scheme, that it serves to protect neo-Darwinian assumptions from criticism, and that this story is untestable and therefore not a hypothesis. Third, attempts to synthesize the selfish DNA concept with complex systems models of the genome and RE functionality are critiqued. Fourth, the supposed connection between RE-induced mutations and macro-evolutionary events are stated to be at variance with empirical evidence and theoretical considerations. Hypotheses that base phylogenetic transitions in repetitive sequence changes thus remain speculative. Fifth and finally, the case is made for viewing RIES as integrally functional components of chromosomes, genomes, and cells. It is argued throughout that a new conceptual framework is needed for understanding the roles of repetitive DNA in genomic/epigenetic systems, and that neo-Darwinian narratives have been the primary obstacle to elucidating the effects of these enigmatic components of chromosomes.", 
"Documentation of variation in phalangeal formulae in land tortoises combined with ontogenetic information from turtles in general were used, in a phylogenetic context, to infer the potential effect of size and developmental constraints upon patterns of morphological variation. A sample of 201 specimens and published illustrations of 37 tortoise species were examined, representing all but one living genera and most species of the Testudinidae. Specimens were either articulated dry skeletons or preserved animals that were x-rayed. The patterns of digital and phalangeal loss in tortoises were predicted from developmental studies of the manus and pes in other turtles. If a digit is lost, it is the first digit, which is the last one to develop. If a digit has a single phalanx, it is usually the fifth digit. The primitive phalangeal formula for land tortoises is probably 2-2-2-2-1, the most common pattern found in living testudinid species. The presence of a second phalanx in the fifth digit evolved independently many times and usually in large tortoises. Such additions were interpreted as instances of peramorphosis. Many small tortoises have a full complement of digits (five) and phalanges (two in each digit); nevertheless, phalangeal and digital loss is associated with small size. Small and medium size tortoises exhibit greater variation in phalangeal number than do large tortoises. We hypothesize that epigenetic processes, and not simply adaptation, played a major role in the evolution of the variation in phalangeal formulae in tortoises. J. Exp. Zool. (Mol. Dev. Evol.) 302B:134-146, 2004. K, 2004 (C) Wiley-Liss, Inc.", 
"In current biological and philosophical literature, the use of the terms epigenesis and epigenetics has increased tremendously. As these terms are often confused, this paper aims at clarifying the distinction between them by drawing their conceptual and historical evolutions. The evolution of the term epigenesis is situated in the context of early embryological studies. Departing from Aristotle's natural philosophy, it is shown that epigenesis gained alternating attention from the 17th century onwards, as it was introduced into neo-classical embryology and considered to be the opposite of the preformationist tradition. Where preformation stated that the germ cells of each organism contain preformed miniature adults that unfold during development, epigenesis held that the embryo forms by successive gradual exchanges in an amorphous zygote. Although both traditions tried to explain developmental organization, religious and metaphysical arguments on the conception of embryonic matter as either active or passive determined the scope of their respective explanations. It is shown that these very arguments still underlie the use of gene-centric metaphors in the molecular revolution of the 20th century.", 
"Polyploidization has occurred often in the evolutionary history of eukaryotes, yet we are only now learning about the genetic and epigenetic events that occur in newly formed polyploids. A long-standing assumption has been that each member of a gene pair would contribute equally to expression levels after polyploidization. This assumption has now been shown to be false in cottons. Adams et al. studied expression levels in different plant tissues from a newly formed allotetraploid cotton (bearing two copies of the so-called 'A' genome and two copies of the 'D' genome). The authors found that the proportion of mRNA levels transcribed from the A and D genomes varied dramatically among tissues. This result implies that duplicated genes can undergo an immediate divergence in function as a pleiotropic effect of hybridization or polyploidization, providing a novel explanation for why selection often preserves duplicated genes.", 
"Endogenous pararetroviruses (EPRVs) represent a new class of dispersed repetitive DNA in plants. The genomes of many Nicotiana species and other solanaceous plants are rich in EPRVs. Distinct EPRV families are present in N. sylvestris (Ns) and in N. tomentosiformis (Nto), the two diploid progenitors of allotetraploid N. tabacum. Nicotiana EPRVs represent an interesting type of repetitive sequence to analyse in polyploids because of their potential impact on plant fitness and the epigenetic architecture of plant genomes. The NsEPRV family appears identical in N. sylvestris and N. tabacum, indicating little change has occurred in either species since polyploid formation. By contrast, the NtoEPRV family is larger in N. tomentosiformis than in N. tabacum, suggesting either preferential elimination from the polyploid genome or specific accumulation in the diploid genome following polyploidization. The lability of NtoEPRVs might be enhanced by a frequent association with gypsy retrotransposons. Although some EPRVs are probably benign, others are potentially pathogenic or, conversely, determinants of virus resistance. Normally quiescent EPRVs can be reactivated and cause symptoms of infection in hybrids of species that differ in their EPRV content. EPRVs that furnish immunity to the free virus exemplify the selective value of so-called 'junk' DNA. Variation in the abundance and distribution of EPRVs among related species can be useful in taxonomic and evolutionary studies. (C) 2004 The Linnean Society of London.", 
"Interspecific hybridization in the rodent genera Peromyscus and Mus results in abnormal placentation. In the Peromyscus interspecies hybrids, abnormal allelic interaction between an X-linked locus and the imprinted paternally expressed Peg3 locus was shown to cause the placental defects. In addition, loss-of-imprinting (LOI) of Peg3 was positively correlated with increased placental size. As in extreme cases this placental dysplasia constitutes a post-zygotic barrier against interspecies hybridization, this finding was the first direct proof that imprinted genes may be important in speciation and thus in evolution. In the Mus interspecies hybrids, a strong role of an X-linked locus in placental dysplasia has also been detected. However, here we show by backcross and allele specific expression analyses that neither LOI of Peg3 nor abnormal interactions between Peg3 and an X-linked locus are involved in generating placental dysplasia in Mus hybrids, although the placental phenotypes observed in the two genera seem to be identical. In contrast to this, another dysgenesis effect common to Peromyscus and Mus hybrids, altered foetal growth, is caused at least in part by the same X-chromosomal regions in both genera. These findings first underline the strong involvement of the X-chromosome in the genetics of speciation. Secondly, they indicate that disruption of epigenetic states, such as LOI, at specific loci may be involved in hybrid dysgenesis effects in one group, but not in another. Thus, we conclude that even in closely related groups divergent molecular mechanisms may be involved in the production of phenotypically similar post-zygotic barriers against hybridization.", 
"Generally speaking, epigenetic states or epigenetic regulation refer to situations in which several states of gene,expression may coexist in similar environmental conditions, despite the absence of significant changes in the genomic sequence. In one way or another, the mechanisms behind these phenomena involve vicious circles, so that each epigenetic. state tends to sustain itself, even after the disappearance of the inductive signal involved in the selection of that particular state. These vicious circles constitute positive feedback circuits and are found at the core of many developmental regulatory systems. In this paper, we present a qualitative model for the regulatory network formed by maternal and gap gene crossregulations. This network controls the initial anterior-posterior patterning during early Drosophila embryogenesis and encompasses several intertwined feedback circuits. On the basis of our model analysis, we derive interesting insights about how specific expression states of the gap genes are selected along the anterior-posterior axis, in particular in relation with the activity of one positive feedback circuit, namely that formed by giant and Kruppel cross-inhibitions. In addition, we are able to qualitatively simulate the patterns of gene expression in the wild-type, as well as to predict the phenotypes of various loss-of-function mutations at the maternal and gap genes, or cis-regulatory mutations at the gap genes, as well as the effects of ectopic expression of these genes.", 
"The Tetrahymena group I ribozyme catalyzes the cleavage of a phosphodiester linkage in specific sequences of RNA. This phenotype can be used in an in vitro selection-amplification process to evolve variants that are capable of RNA catalysis in the presence of Ca2+ as the sole available cation. With sufficient genotypic characterization of the population as it evolves, we have a rare opportunity of observing how the information stored in an evolving population responds to selective pressures, such as the requisite of catalyzing RNA cleavage in the absence of Mg2+ or Mn2+. In the present work, we examine the population dynamics of this system using sequence information from previous experimental work. We focus on two issues: How does the information content of the population evolve? and Is the system evolving as an adaptive walk on a rugged landscape? To investigate these questions, information theoretical parameters are examined. The evolution of the population is visualized by mapping the genotypic frequency distribution onto a two-dimensional projection of sequence space. The projection was generated using Hamming distances from the wild-type, starting sequence and a catalytically successful, evolved sequence. The evolution of the information content of the system was measured by calculating the grammar complexity of the observed sequences, which showed a very slight increase over 12 generations. This result is consistent with the system performing a search for a local optimum. The dynamics of the population in this sequence space is consistent with an adaptive walk on an uncorrelated, or rugged, genotypic landscape, despite the observation that the phenotypic progress of the population appears smooth. The relative insensitivity of the phenotypic landscape to the variegation of the genotypic landscape suggests that the former is buffered against variation in the latter through various epigenetic-like mechanisms.", 
"A subset of autosomal genes undergo genomic imprinting which results in expression from only the paternal or maternal chromosome. While this phenomenon is restricted to mammals and angiosperms, the underlying silencing mechanisms appear to be evolutionarily conserved. A biallelically unmethylated DNaseI hypersensitive region (A6-A4) between the imprinted Igf2 and H19 genes is conserved in humans and mice and functions as a tissue-specific maintenance element for the imprinted growth factor IGF2. In order to analyse A6-A4 for potentially conserved transcriptional maintenance properties, we have generated transgenic Drosophila harbouring the element in a reporter construct. These flies depicted silencing of the reporter genes lacZ and mini-white. The silenced state of the mini-white gene showed variegation and sensitivity to temperature changes. In addition, two members of the conserved Polycomb group, Enhancer of zeste and Posterior sex combs, were needed for repression. Polycomb group proteins are essential for gene silencing during development. Our results indicate that Polycomb group proteins may also be involved in the regulation of mammalian imprinted genes.", 
"Developmental stability (DS) may confer an advantage in competition for mates. The present study tests this hypothesis using Drosophila immigrans, and proposes a novel approach to help broadly define the epigenetic factors causing such an effect. We first estimated the magnitude of isofemale heritability in sternopleural bristle fluctuating asymmetry (FA), using replicate genetic lines extracted from nature. Positional FA (PFA) exhibited significant among-line variation, and the heritability estimate of 0.10 (0.046 s.e.m.) was statistically significant. Among individual males, there was a significant positive relationship between PFA and copulation latency (time elapsed between introduction of females and copulation) and duration, but not copulation frequency. Moreover, high-DS lines exhibited significantly shorter copulation latency and duration compared with low-DS lines. When these components of sexual performance were again contrasted between lines with among-individual differences in bristle asymmetry controlled statistically, significant line effects on copulation latency and duration disappeared. The results suggest that deficits in the developmental apparatus underlying one particular trait can compromise individual sexual performance, and weaken the hypothesis that FA is a cue of overall 'genetic quality'.", 
"It is well known that repositioning of a gene often exerts a strong impact on its own expression and whole development. Here we report the results of genome-wide analyses suggesting that repositioning may also radically change the evolutionary fate of gene duplicates. As an indicator of these changes, we used the GC content of gene pairs which originated by duplication. This indicator turned out to be duplicate-asymmetric, which means that genes in a pair differ significantly in GC content despite their apparent origin from a common ancestor. Such an asymmetry necessarily implies that after duplication two originally identical genes mutated in opposite directions-toward GC-rich and GC-poor content, respectively. In mammalian genomes, this trend is definitely associated with presumably methylated hypermutable CpG sites, and in a typical GC-asymmetric gene pair, its two member genes are embedded in GC-contrasting isochores. However, we unexpectedly found similar significant GC asymmetry in fish, fly, worm, and yeast. This means that neither methylation alone nor methylation in combination with isochores can be counted as a primary cause of the GC asymmetry; rather they represent specific realizations of some universal principle of genome evolution. Remarkably, genes from pairs with the greatest GC asymmetry tend to be on different chromosomes, suggesting that the mutational difference between gene duplicates is associated with translocation of a new gene to a different place in the genome, whereas GC symmetric pairs demonstrate the opposite tendency. A recently emerged extra gene copy is usually on the same chromosome as is its parent but quickly, by 0.05 substitution per synonymous site, either has perished or occupies a different chromosome. During this earliest posttranslocation period, the ratio of nonsynonymous/synonymous base substitutions is unusually high, suggesting a rapid adaptive evolution of novel functions. In a general context of evolution by gene duplication, our interpretation of this position-dependent GC asymmetry between duplicated genes is that evolution of redundant genes toward a new function has often been associated with their very early, postduplication repositioning in the genome, with a concomitant abrupt change in epigenetic control of tissue/stage-specific expression and an increase in the mutation rate. Of eight eukaryotic genomes studied, the most distinguished in this respect is the human genome.", 
"The complexity of some sexual reproductive systems in arthropods still leaves both their genetic and epigenetic determinism and their evolutionary significance poorly understood. Pseudoarrhenotoky is characterized by obligate fertilization and differential inactivation and/or elimination of paternal chromosomes in embryos that develop into males. Here, we investigate how the paternal genome is transmitted in a pseudoarrhenotokous mite, Neoseiulus californicus, using codominant genetic markers detected by DALP (direct amplification of length polymorphism). Transmission patterns of parental alleles through one and two generations are reported at four or five loci corresponding to four linkage groups. Our data provide strong evidence for selective elimination of the paternal genome among male tissues. Sperm contained maternal genes exclusively, whereas some male somatic tissues retained most if not all paternal chromosomes. No recombination between parental genomes prior to paternal genome elimination from the embryonic germ line was observed. These data allow a reinterpretation of previous phenotypic and cytogenetic observations in these mites, from which we suggest some relevant mechanistic and evolutionary implications. In addition, this is the first published study using polymorphic codominant loci detected by the recently developed DALP method.", 
"During the development of a multicellular organism from a zygote, a large number of epigenetic interactions take place on every level of suborganismal organization. This raises the possibility that the system of epigenetic interactions may compensate or ''buffer'' some of the changes that occur as mutations on its lowest levels, and thus stabilize the phenotype with respect to mutations. This hypothetical phenomenon will be called ''epigenetic stability.'' Its potential importance stems from the fact that phenotypic variation with a genetic basis is an essential prerequisite for evolution. Thus, variation in epigenetic stability might profoundly affect attainable rates of evolution. While representing a systemic property of a developmental system, epigenetic stability might itself be genetically determined and thus be subject to evolutionary change, Whether or not this is the case should ideally be answered directly, that is, by experimentation. The time seals involved and our insufficient quantitative understanding of developmental pathways will probably preclude such an approach in the foreseeable future. Preliminary answers are sought here by using a biochemically motivated model of a small bur central part of a developmental pathway. Modeled are sets of transcriptional regulators that mutually regulate each other's expression and thereby form stable gene expression patterns. Such gene-expression patterns, crucially involved in determining developmental pattern formation events, are most likely subject to strong stabilizing natural selection, After long periods of stabilizing selection. the fraction of mutations causing changes in gene-expression pattern is substantially reduced in the model. Epigenetic stability has increased. This phenomenon is found for widely varying regulatory scenarios among transcription factor genes. Its discussed that only epistatic (nonlinear) gene interactions can cause such change in epigenetic stability. Evidence from paleontology, molecular evolution, development, and genetics, consistent with the existence of variation in epigenetic stability, is discussed, The relation of epigenetic stability to developmental canalization is outlined. Experimental scenarios at suggested thar may provide further evidence.", 
"Expression of imprinted genes is dependent on their parental origin. This is reflected in the heritable differential methylation of parental alleles. The gametic imprints are however reversible as they do not endure for more than one generation. To investigate if the epigenetic changes in male and female germ line are similar or not, we derived embryonic germ (EG) cells from primordial germ cells (PGCs) of day 11.5 and 12.5 male and female embryos. The results demonstrate that they have an equivalent epigenotype. First, chimeras made with EG cells derived from both male and female embryos showed comparable fetal overgrowth and skeletal abnormalities, which are similar to but less severe than those induced by androgenetic embryonic stem (ES) cells. Thus, EG cells derived from female embryos resemble androgenetic ES cells more than parthenogenetic cells. Furthermore, the methylation status of both alleles of a number of loci in EG cells was similar to that of the paternal allele in normal somatic cells. Hence, both alleles of Igf2r region 2, Peg1/Mest, Peg3, Nnat were consistently unmethylated in EG cells as well as in the primary embryonic fibroblasts (PEFs) rescued from chimeras. More strikingly, both alleles of p57kip2 that were also unmethylated in EG cells, underwent de novo methylation in PEFs to resemble a paternal allele in somatic cells. The exceptions were the H19 and Igf2 genes that retained the methylation pattern in PEFs as seen in normal somatic tissues. These studies suggest that the initial epigenetic changes in germ cells of male and female embryos are similar.", 
"Many insects are attacked by internal parasitoids against which they mount a largely cellular immunological defense. The resistance of a host and the virulence of a parasitoid determine which species survives after parasitism. Drosophila is parasitized by several hymenopterous parasitoids, especially those in the genera Asobara and Leptopilina. Geographic patterns have been found in parasitoid virulence and host resistance, the dearest of which is a dine in Asobara tabida virulence from the north (low) to the south thigh) of Europe. Drosophila melanogaster resistance is highest in central-southern Europe and lower elsewhere. We review and interpret these patterns in the light of recent experimental and theoretical studies of the evolution and coevolution of these traits. We find no evidence for genotype-specific virulence and defense, which makes Red Queen-type coevolution unlikely. The most important explanation for the patterns is geographic differences in host-parasitoid community structure. Asobara tabida virulence is positively correlated with the resistance of its main hosts, and there is more limited evidence that D. melanogaster resistance is influenced by the virulence of its parasitoids. We critically appraise whether the evidence available so far supports a coevolutionary explanation for the levels of these trails.", 
"We discuss the role of cell memory in heredity and evolution. We describe the properties of the epigenetic inheritance systems (EISs) that underlie cell memory and enable environmentally and developmentally induced cell phenotypes to be transmitted in cell lineages, and argue that transgenerational epigenetic inheritance is an important and neglected part of heredity. By looking at the part EISs have played in the evolution of multicellularity, ontogeny, chromosome organization, and the origin of some post-mating isolating mechanisms, we show how considering the role of epigenetic inheritance can sometimes shed light on major evolutionary processes.", 
"Cytosine methylation is widely distributed in multicellular organisms. We present a comprehensive survey of the existing data on the phylogenetic distribution of DNA methylation in invertebrates, together with new data for the crustacean Penaeus semisulcatus, the annelid Aporrectodea caliginosa trapezoides, and the parasitic platyhelminth Schistosoma mansoni. Two alternative hypotheses addressing the function of cytosine methylation in invertebrates are evaluated: (1) cytosine methylation is an ancient regulatory mechanism which was lost in species with low rates of cell turnover, and (2) cytosine methylation is primarily a defense mechanism against genomic parasites and is expected to be present in all species with large genomes. We discuss the role of DNA methylation in the evolution of development in light of these hypotheses and conclude that gene control and cell memory are important and primitive functions of DNA methylation.", 
"Redundant gene functions are ubiquitous, and they are a potentially important source of evolutionary innovations on the biochemical level. It is therefore highly desirable to understand the mechanisms governing their evolution. Gene duplication is clearly a prominent mechanism generating redundant genes. However, because redundancy provides a protective effect against deleterious mutations, natural selection might be involved in generating and maintaining partial redundancy. Although much experimental data on redundant genes have accumulated, no data are available that could elucidate what role selection has in their evolution. As a first step towards answering this question, a conceptually simple mathematical model for the evolution of redundancy is introduced. its main result is that selection cannot only maintain but also increase redundancy among genes in a population provided (i) that mutation generates sufficient variation in redundancy and (ii) that populations are large. The population biological process at work is somewhat unusual. Selection does not act on the (nonexisting) differential fitness between individuals with different degrees of redundancy. Rather, it acts through the low number of offspring with deleterious mutations that individuals with redundant genes will generate. Moreover, even if populations are small and variation in redundancy is low, selection will substantially slow the 'decay' of redundancy caused by mutation and genetic drift. Methodological problems in determining degrees of redundancy experimentally are discussed, as well as issues concerning the relation of redundancy to genetic canalization. The latter two phenomena necessitate a differentiated view of neutral mutations, where some neutral mutations are only neutral because their effects on gene products are absorbed by the epigenetic system.", 
"Since the Modern Synthesis, evolutionary biologists have assumed that the genetic system is the sole provider of heritable variation, and that the generation of heritable variation is largely independent of environmental changes, However, adaptive mutation, epigenetic inheritance, behavioural inheritance through social learning, and language-based information transmission have properties that allow the inheritance of induced or learnt characters. The role of induced heritable variation in evolution therefore needs to be reconsidered, and the evolution of the systems that produce induced variation needs to be studied.", 
"Three different human femoro-tibial characters are selected as functionally relevant and derived hominid characters: femoral bicondylar angle, shape of the femoral distal epiphysis, and the tibial insertion of the lateral meniscus. The timing and mode of formation of these characters are investigated during human ontogeny and are shown to differ considerably. The available hominid fossils (Australopithecus afarensis and early Homo) are interpreted in the light of this ontogenetic analysis with the conclusion that, during hominid evolution, different modes of selection of these features must have occurred. In modern humans, the femoral bicondylar angle proves to be an epigenetic functional feature, which develops during early childhood growth. It is present in all australopithecines and we suggest that it developed following a change in their locomotor behavior and not upon a genomic change: the early practice of bipedal walking, with adducted knee joints, in the locomotor repertoire of infant australopithecines, was sufficient to promote this angle. Later in hominid evolution, the knee joint evolved from having a single insertion of the lateral meniscus on the tibia to a double one. While Australopithecus afarensis exhibits a single insertion, early Homo clearly exhibits a double insertion of the lateral meniscus on the tibia. The double insertion restricts the mobility of the meniscus on the tibial plateau, indicating a habitual practice of full extension movements of the knee joint. Among modern humans, the posterior insertion of the lateral meniscus appears early in fetal life. Consequently in early Homo, this new selected feature developed directly as a result of a genomic change. The derived shape of human distal femoral epiphysis includes a prominence of the lateral lip of the femoral trochlea, an elliptical profile of the lateral condyle, and an anteroposterior lengthening of the epiphysis. Analysis of human fetal and neonatal distal epiphyses shows that the prominence of the lateral lip of the trochlea arises before any use, and thus appears to be genetically determined. However, the postnatal development of this joint shows that this feature is also modified epigenetically by use. It is argued that the hominid femoropatellar joint would have been reshaped following the process of genetic assimilation (Waddington [1942] Nature 3811:563-565). The prominence of the lateral lip of the femoral trochlea was probably selected following a two-staged process-first epigenetic, then genetic. Far from being a Lamarckian explanation, this concept applies precisely to adaptive characters that are induced by an external stimulus during a single lifetime and are replaced through natural selection by genetically based equivalent characters. The nature of the structures involved in the studied features is shown to be an important parameter determining their mode of development and selection. (C) 1999 Wiley-Liss, Inc.", 
"The adaptive value of epigenetic inheritance systems is investigated in a simple mathematical framework. These systems enable the environmentally induced phenotypes to be transmitted between generations. The frequencies of the different epigenetic variants are determined by the plasticity and the efficiency of transmission (called memory). Plasticity and memory are genetically determined. This paper studies the evolution of a quantitative character, its plasticity and memory, on the adaptive landscape. Due to the dual inheritance of the character, selection acts on two revels: on the phenotypes of the same genotype, and on the different genotypes. Plasticity generates the raw material, and memory increases the strength of phenotypic selection. If the character is far from the peak of the landscape, then dual inheritance of the character can be advantageous for the genotype. Near the peak it is more favourable to suppress phenotypic variation. This would lead to genetic assimilation.", 
"Evolutionary biologists are largely polarized in their approaches to integrating microevolutionary and macroevolutionary processes. Neo-Darwinians typically seek to identify population-level selective and genetic processes that culminate in macroevolutionary events. Epigeneticists and structuralists, on the other hand, emphasize developmental constraints on the action of natural selection, and highlight the role of epigenetic shifts in producing evolutionary change in morphology. Accordingly, the ways in which these paradigms view and address morphological contrasts between classes of related organisms differ. These paradigms, although seldomly explicitly stated, emerge in paleoanthropology as well. Considerations of postcranial morphological contrasts between archaic and modern humans typically fall into one of two broad interpretive models. The first derives from the neo-Darwinian perspective and holds that evolution in the postcranial skeleton was largely mosaic (operating in a particulate manner), and that temporal change in specific traits informs us about behavioral shifts or genetic evolution affecting isolated anatomical regions (i.e., adaptive behavioral inferences can be made from comparative studies of individual trait complexes). The alternative model follows from the epigeneticist paradigm and sees change in specific postcranial traits as correlated responses to change in overall body form (involving shifts in regulation of skeletal growth, or selective and developmental responses to broad adaptive shifts). By this view, ilztegration of functional systems both constrains and directs evolution of various traits, and morphological contrasts inform us about overall change in body form related to change in such things as overall growth patterns, climatic adaptation, and technological dependency. These models were tested by confirmatory factor analysis using measures of upper body form and upper limb morphological traits in Eurasian Neandertal and early modern fossils and recent human samples. Results indicate (1) a model of morphological integration fits the data better than a model of no integration, but (2) this integration accounts for less than half of the variance in upper limb traits, suggesting a high degree of tolerance for particulate evolution in the context of an integrated upper body plan. Significant relationships were detected between joint shapes and body size, between humeral shaft shape and body size and chest shape, and between measures of biomechanical efficiency and robusticity. The observed morphological differences between late archaic and early modern humans reflect particulate evolution in the context of constraints imposed by genetic and morphological integration. While particulate approaches to interpreting the fossil record appear to be justified, attention must also be paid to delineating the nature and extent of morphological integration and its role in both constraining and producing observed patterns of variation between groups. Confirmatory factor analysis provides a means of examining trait covariance matrices, and sert es as a useful method of identifying patterns of integration in morphology. (C) 1996 Wiley-Liss, Inc.", 
"Most traditional views of homology rely on two unwarranted premises: the pervasively hierarchical nature of biology, inclusive of the levels of genes, development, and morphology and the linear mapping of genes onto developmental schedules and of developmental schedules onto phenotypes. These premises are only occasionally verified. Hierarchical behavior is negated by gene duplication and exon shuffling at the level of genes, by the coexistence of autonomous vs nonautonomous gene expression at the level of development, by ontogenetic repatterning at the level of morphology. The linearity of mapping of genes onto development is disturbed by genetic piracy, uncoupling of positional vs spatial control, and pleiotropy. The independence of developmental modules affects the mapping of development onto morphology and, finally, the peculiar topology of the epigenetic code affects the linearity of the gene to phenotype mapping. To cope with this complex behavior, a combinatorial approach to homology is recommended, (C) 1998 Academic Press.", 
"The occurrence and distribution of 35 cranial epigenetic traits in the Italian Neandertals (Saccopastore 1 and 2, Gualtari 1) were examined according to morpho-functional cranial regions and with respect to a distinction between hypostosis (i.e., weak osseous development, arrested morphogenesis, retention of infantile features) and hyperostosis (i.e. excess of ossification, not reaching the pathological condition); The results, expressed as hypostotic scores, showed higher levels of hypostosis in these Neandertal specimens than in recent European samples. The highest expressions of hypostotis were observed in those regions of the Neandertal cranium where disequilibrium between skeletal and cerebral growth factors was expected. These results-interpreted as expression of developmental stress-are consistent with a heterochronic interpretation of the development of the Neandertal cranium; namely, a faster ossification of the cranial vault relative to brain growth rates. (C) 1996 Academic Press Limited.", 
"Six lines of flax (Linum usitatissimum) were derived from six short, early flowering plants that were first-generation progeny of plants grown from seeds treated with 5-azacytidine. The third and fourth progeny generations of the lines, and the F-1 and F-2 progeny of crosses between three of the lines and their corresponding untreated controls, demonstrated the relatively stable inheritance of the induced phenotypes and provided information on the genetic control of the induced changes, Some reversion of the induced phenotypes was detected, but much of the phenotypic variability observed in the progeny generations could be attributed to segregation, Segregation patterns demonstrated that the early flowering phenotype is determined by the combined effects of 5-azacytidine-induced changes at three (or more) independent loci, Furthermore, the induced form of one locus is dominant or codominant, but the induced forms of the other loci are recessive. A working model describes the proposed genetic control, and the significance of 5-azacytidine-induced effects which are recessive is discussed.", 
"Cortical maps often contain global spatial structure; however, theoretical accounts for their development have generally concentrated on reproducing only local structure. We show that the elastic net model of cortical map formation can closely approximate the global structure of the ocular dominance column map observed in macaque primary visual cortex. A key component is the assumption of spatially non-uniform and anisotropic correlations in the retina. This work shows how genetic and epigenetic effects could combine to establish characteristic global structure in cortical maps.", 
" ", 
" ", 
"Genomic imprinting results in the functional specialization of the maternal and paternal genomes during development whereby offspring inherit only one active copy of a gene. Although the reason for its evolutionary genesis remains speculative, the consequences of genomic imprinting are evidenced by the failure of parthenogenetic and androgenetic development in mammals, and parental-specific effects in the etiology of a number of human diseases, While the precise molecular mechanism of imprinting is unknown, evidence suggests that the specialization of the parental genomes is established during gametogenesis when the parental genomes are epigenetically modified to reflect the parent of origin, To examine the epigenetic modification of specific imprinted genes and subsequent differential expression, an assay is required that can distinguish between the maternal and paternal alleles and their respective transcripts, During the past 5 years several strategies have been used to identify imprinted genes, ranging from the fortuitous disruption of specific parental alleles to subtraction hybridization between cDNAs from normal and parthenogenetic embryos, To study the developmental regulation of these imprinted genes during mammalian development, we describe a relatively simple interspecies ''mRNA phenotyping'' approach applicable to the analysis of allele-specific expression as well as the identification of candidate imprinted genes.", 
"Because development is epigenetic, diverse aspects of morphology are integrated during ontogeny. Using the method of thin-plate splines, and the decomposition of these splines by their principal warps, we examine the ontogeny of integrated features of skull growth of the cotton rat, Sigmodon fulviventer as observed in landmark locations in the ventral view. Postnatal growth of the skull in Sigmodon is not adequately described by the familiar contrast between relatively rapid facial elongation and slow, precocial growth of the cranial base. No developmental units corresponding to facial skull and cranial base emerge from analysis of geometric shape change. Rather, skull growth is both more integrated and more complex, exhibiting both skull-wide integration and locally individualized regions. Like skull shape, integration has an ontogeny; different regions of the skull can be partitioned into developmentally individualized parts in different ways at different ages. The effective count of individualized parts decreases substantially before weaning occurs, suggesting that the integration required by the functionally demanding activity of chewing gradually develops before the functional transition occurs. Our description of skull growth and integration does not depend upon arbitrary a priori choices about what to measure; rather, we base our decomposition of the whole into parts upon results of the data analysis. Our approach complicates the study of heterochrony, but, because it expresses the spatiotemporal organization of ontogeny, it enables the study of heterotopy.", 
"Fluctuating asymmetry is an epigenetic measure of the ability of individuals to undergo identical development of a usually bilaterally symmetrical character on both sides of the body. Horns of beetles and spurs of birds are elaborate structures usually restricted to or enlarged in individuals of the limited sex and frequently used in intrasexual combat. Individuals with large horns usually win fights against individuals with small horns which suggests that horn size may reliable signal male fighting ability. A comparative study of the patterns of fluctuating asymmetry in horns of beetles and spurs of birds showed that the degree of asymmetry in these secondary sex traits was considerably larger than in elytra and wing length. which are apparently not subjected to sexual selection. There was a negative relationship between the degree of asymmetry and size of the secondary sex trait while the relationship for elytra and wing length was flat or U-shaped. This demonstrates that horn and spur size may reliable reflect male quality as determined by the degree of fluctuating asymmetry.", 
"Herein we describe a general multivariate quantitative genetic model that incorporates two potentially important developmental phenomena, maternal effects and epigenetic effects. Maternal and epigenetic effects are defined as partial regression coefficients and phenotypic variances are derived in terms of age-specific genetic and environmental variances. As a starting point, the traditional quantitative genetic model of additive gene effects and random environmental effects is cast in a developmental time framework. From this framework, we first extend a maternal effects model to include multiple developmental ages for the occurrence of maternal effects. An example of maternal effects occurring at multiple developmental ages is prenatal and postnatal maternal effects in mammals. Subsequently, a model of intrinsic and epigenetic effects in the absence of maternal effects is described. It is shown that genetic correlations can arise through epigenetic effects, and in the absence of other developmental effects, epigenetic effects are in general confounded with age-specific intrinsic genetic effects. Finally, the two effects are incorporated into the basic quantitative genetic model. For this more biologically realistic model combining maternal and epigenetic effects, it is shown that the phenotypic regressions of offspring on mother and offspring on father can be used in some cases to estimate simultaneously maternal effects and epigenetic effects.", 
"Fluctuating asymmetry occurs when an individual is unable to undergo identical development of an otherwise bilaterally symmetric trait on both sides of its body. Since both sides of a bilaterally symmetric trait are the result of the actions of a single genome, fluctuating asymmetry represents an epigenetic measure of the sensitivity of development to stress. Different morphological traits may show a direct relationship between their functional importance and their degree of developmental canalization. This may explain why some characters show high degrees of fluctuating asymmetry, and why these characters more often become exaggerated secondary sexual ornaments. The degree of fluctuating asymmetry is generally larger in small marginal populations living in novel environments, and this will particularly lead to relatively large degrees of asymmetry in the least developmentally canalized traits. More stringent selection against heterozygotes in marginal populations may further break down developmental stability and linkage groups which would lead to increased genetic variance. Females may prefer to mate with males having large, but relatively symmetric morphological characters, because it is more difficult to make large traits (a good genes argument), a large trait is more easily perceived (a sensory bias preference), and because symmetry signals ability to cope with stress (a good genes argument). The low degree of developmental stability and the large amount of genetic variance in secondary sexual characters in small, marginal populations could set the scene for rapid development of divergence and speciation in marginal populations.", 
"Extravagant secondary sexual characters are assumed to have arisen and be maintained by sexual selection. While traits like horns, antlers and spurs can be ascribed to intrasexual competition, other traits such as extravagant feather ornaments, displays and pheromones have to be ascribed to mate choice. A number of studies have tested whether females exert selection on the size of male ornaments, but only some of these have recorded female preferences for the most extravagantly ornamented males. Here I demonstrate that female choice can be directly predicted from the relationship between the degree of fluctuating asymmetry and the size of a secondary sexual character. Fluctuating asymmetry is an epigenetic measure of the ability of individuals to cope with stress, and it occurs when an individual is unable to undergo identical development of an otherwise bilaterally symmetric trait on both sides of its body. There is a negative relationship between the degree of fluctuating asymmetry and the absolute size of an ornament in those bird species with a female preference for the largest male sex trait, while there is a flat or U-shaped relationship among species without a female preference. These results suggest that females prefer exaggerated secondary sexual characters if they reliably demonstrate the ability of males to cope with genetic and environmental stress. Some species may demonstrate a flat or U-shaped relationship between the degree of fluctuating asymmetry and the absolute size of an ornament because (i) the genetic variance in viability signalled by the secondary sex trait has been depleted; (ii) the secondary sex trait is not particularly costly and therefore does not demonstrate condition dependence; or because (iii) the sex traits can be considered arbitrary traits rather than characters reflecting good genes.", 
"Reciprocal embryo transfer experiments show that skeletal dimensions in adult mice are significantly influenced by the genotype of the female providing the uterine environment in which they were raised. Embryo transfers among C3HeB/FeJ, SWR/J, and the C3SWF1 hybrid strain (C3H females x SWR males) permit separation of uterine maternal genotype effects from effects arising from the progeny's own genotype. Many different aspects of adult skeletal form are significantly influenced by uterine genotype and, in some instances, the pattern of these effects correlates with events during skeletal embryology. Analyses involving the highly heterozygous C3SWF1 strain demonstrate the existence of significant dominance in maternal genes affecting skeletal development in the progeny. Further, there is a large skeletal effect due to progeny heterosis. Uterine litter size can be manipulated as a nonheritable component of variability in embryo transfer experiments, and it has a large and systemic effect on skeletal growth and morphogenesis that persists in adult mice. Heritable uterine maternal effects are epigenetic interactions during development that can be incorporated into models of evolutionary change to provide a more complete picture of the causal agents producing morphological change.", 
"Developmental constraint is a theoretically important construct bridging ontogenetic and evolutionary studies. We propose a new operationalization of this notion that exploits the unusually rich measurement structure of landmark data. We represent landmark configurations by their partial warps, a basis for morphospace that represents a set of localized features of form. A finding of developmental constraint arises from the interplay between age-varying means and age-specific variances in these subspaces of morphospace. Examination of variances and means in 16 ventral skull landmarks in the cotton rat S. fulviventer at ages 1, 10, 20, and 30 days yielded three types of developmental constraint: canalization (constraint to relatively constant form age by age); chreods (reduction of variance orthogonal to the mean trajectory over ages); and opposition (reduction of age-specific variance along the mean trajectory over ages). While canalization and chreodic constraints have been noted previously, the oppositional type of constraint appears novel. Only one of our characters, relative length and orientation of the incisive foramen, appears to be canalized. Although skull growth becomes increasingly integrated through ontogeny, our characters display a remarkable spatiotemporal complexity in patterns of variance reduction. The specific assortment of constraints observed may be related to the precociality of Sigmodon. We suggest that Waddington's diagrammatic presentation of the ''epigenetic landscape'' may be misleading in quantitative studies of developmental regulation.", 
"Cladistic analysis of osteological and dental characters in a monophyletic group of Miocene and younger tayassuids demonstrates a pattern of changes in the degree of sexual dimorphism in canine tooth diameter and zygomatic arch width, and in phenotypic correlations between these characters. Primitively, tayassuids have canine teeth that are sexually dimorphic and discretely bimodal in size, and zygomatic arches that are narrow in both sexes. Many late Miocene and Pliocene tayassuids have broad, winglike zygomatic processes. In some species, these processes are large in both sexes, but in others, those of females are much smaller than those of males. The presence of large processes in both sexes is primitive relative to the condition of strong sexual dimorphism. In five separate clades, the zygomatic processes of both sexes become reduced in size, and the degree of sexual dimorphism in canine size becomes reduced as well. The pattern is congruent with predictions derived from a theoretical model of the evolution of sexual dimorphism, and it further indicates the emergence of a new phenotypic correlation between two previously uncorrelated characters, canine size and zygoma size. The advent of this new correlation coincides with the advent of pronounced sexual dimorphism in zygomatic processes. Although such a pattern could be explained by genetically modifying phenotypic expression of homologous characters in one sex or the other, an epigenetic modification of expression is equally plausible: the evolution of sexual dimorphism in homologous characters could be accomplished by placing phenotypic expression of an originally monomorphic character under the control of steroid sex hormones. This hypothesis is consistent with evidence from many vertebrate groups, and it provides testable predictions.", 
" ", 
"We manipulated developmental time and dry weight at eclosion in 15 genotypes of Drosophila melanogaster by growing the larvae in 9 environments defined by 3 yeast concentrations at 3 temperatures. We observed how the genetic and various environmental components of phenotypic variation scaled with the mean values of the traits. Temperature, yeast, within-environmental factors and genotype influenced the genotypic and environmental standard deviations of the two traits in patterns that point to very different modes of physiological and developmental action of these factors. Since different factors affected the environmental and genetic components of the phenotypic variation either in parallel or inversely, we conclude that environmental heterogeneity may have small or large effects on evolutionary rates depending on which factors cause the heterogeneity. The analysis also suggests that the scaling of variances with the mean is not as trivial as is often assumed when coefficients of variation are computed to ''standardize'' variation.", 
]


In [8]:
###import pandas as pd

###df = pd.read_excel (r'C:/Users/brady/Desktop/TM/Evo-Bio All Time/Test1.xlsx')
###print (df)

In [9]:
print(df)

["Individual animals frequently exhibit repeatable differences from other members of their population, differences now commonly referred to as 'animal personality'. Personality differences can arise, for example, from differences in permanent environmental effects-including parental and epigenetic contributors- and the effect of additive genetic variation. Although several studies have evaluated the heritability of behaviour, less is known about general patterns of heritability and additive genetic variation in animal personality. As overall variation in behaviour includes both the among-individual differences that reflect different personalities and temporary environmental effects, it is possible for personality to be largely genetically influenced even when heritability of behaviour per se is quite low. The relative contribution of additive genetic variation to personality variation can be estimated whenever both repeatability and heritability are estimated for the same data. Using p

In [10]:
###CLEAN DATA###

In [11]:
# Convert to list
data = [simple_preprocess(doc) for doc in df]

In [12]:
print(data)

[['individual', 'animals', 'frequently', 'exhibit', 'repeatable', 'differences', 'from', 'other', 'members', 'of', 'their', 'population', 'differences', 'now', 'commonly', 'referred', 'to', 'as', 'animal', 'personality', 'personality', 'differences', 'can', 'arise', 'for', 'example', 'from', 'differences', 'in', 'permanent', 'environmental', 'effects', 'including', 'parental', 'and', 'epigenetic', 'contributors', 'and', 'the', 'effect', 'of', 'additive', 'genetic', 'variation', 'although', 'several', 'studies', 'have', 'evaluated', 'the', 'heritability', 'of', 'behaviour', 'less', 'is', 'known', 'about', 'general', 'patterns', 'of', 'heritability', 'and', 'additive', 'genetic', 'variation', 'in', 'animal', 'personality', 'as', 'overall', 'variation', 'in', 'behaviour', 'includes', 'both', 'the', 'among', 'individual', 'differences', 'that', 'reflect', 'different', 'personalities', 'and', 'temporary', 'environmental', 'effects', 'it', 'is', 'possible', 'for', 'personality', 'to', 'be', 

In [13]:
###MAY NEED CLEANING STEP, MAY NOT BECAUSE ABSTRACTS ARE SO CLEAN###

In [14]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['individual', 'animals', 'frequently', 'exhibit', 'repeatable', 'differences', 'from', 'other', 'members', 'of', 'their', 'population', 'differences', 'now', 'commonly', 'referred', 'to', 'as', 'animal', 'personality', 'personality', 'differences', 'can', 'arise', 'for', 'example', 'from', 'differences', 'in', 'permanent', 'environmental', 'effects', 'including', 'parental', 'and', 'epigenetic', 'contributors', 'and', 'the', 'effect', 'of', 'additive', 'genetic', 'variation', 'although', 'several', 'studies', 'have', 'evaluated', 'the', 'heritability', 'of', 'behaviour', 'less', 'is', 'known', 'about', 'general', 'patterns', 'of', 'heritability', 'and', 'additive', 'genetic', 'variation', 'in', 'animal', 'personality', 'as', 'overall', 'variation', 'in', 'behaviour', 'includes', 'both', 'the', 'among', 'individual', 'differences', 'that', 'reflect', 'different', 'personalities', 'and', 'temporary', 'environmental', 'effects', 'it', 'is', 'possible', 'for', 'personality', 'to', 'be', 

In [15]:
###THIS STEP NEEDS TO BE CAREFULLY MONITORED TO MAKE SURE IT'S WORKING AS INTENDED###

In [16]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['individual', 'animals', 'frequently', 'exhibit', 'repeatable', 'differences', 'from', 'other', 'members', 'of', 'their', 'population', 'differences', 'now', 'commonly', 'referred', 'to', 'as', 'animal', 'personality', 'personality', 'differences', 'can', 'arise', 'for', 'example', 'from', 'differences', 'in', 'permanent', 'environmental', 'effects', 'including', 'parental', 'and', 'epigenetic', 'contributors', 'and', 'the', 'effect', 'of', 'additive', 'genetic', 'variation', 'although', 'several', 'studies', 'have', 'evaluated', 'the', 'heritability', 'of', 'behaviour', 'less', 'is', 'known', 'about', 'general', 'patterns', 'of', 'heritability', 'and', 'additive', 'genetic', 'variation', 'in', 'animal', 'personality', 'as', 'overall', 'variation', 'in', 'behaviour', 'includes', 'both', 'the', 'among', 'individual', 'differences', 'that', 'reflect', 'different', 'personalities', 'and', 'temporary', 'environmental', 'effects', 'it', 'is', 'possible', 'for', 'personality', 'to', 'be', '

In [17]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [18]:
###en_core_web_sm needs to be explicitly loaded and I believe you need to use "python -m spacy download en" in cmd.exe to properly load everything.###

In [19]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['individual', 'animal', 'frequently', 'exhibit', 'repeatable', 'difference', 'member', 'population', 'difference', 'commonly', 'refer', 'animal', 'personality', 'personality', 'difference', 'arise', 'example', 'difference', 'permanent', 'environmental', 'effect', 'include', 'parental', 'epigenetic', 'contributor', 'effect', 'additive', 'genetic', 'variation', 'several', 'study', 'evaluate', 'heritability', 'behaviour', 'less', 'know', 'general', 'pattern', 'heritability', 'additive', 'genetic', 'variation', 'animal', 'personality', 'overall', 'variation', 'behaviour', 'include', 'individual', 'difference', 'reflect', 'different', 'personality', 'temporary', 'environmental', 'effect', 'possible', 'personality', 'largely', 'genetically', 'influence', 'even', 'heritability', 'behaviour', 'quite', 'low', 'relative', 'contribution', 'additive', 'genetic', 'variation', 'personality', 'variation', 'estimate', 'whenever', 'repeatability', 'heritability', 'estimate', 'datum', 'use', 'publish'

In [20]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 4), (1, 1), (2, 5), (3, 1), (4, 1), (5, 1), (6, 4), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 6), (13, 1), (14, 1), (15, 3), (16, 2), (17, 1), (18, 4), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 5), (29, 1), (30, 6), (31, 1), (32, 2), (33, 2), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 2), (43, 1), (44, 1), (45, 1), (46, 1), (47, 2), (48, 1), (49, 1), (50, 1), (51, 1), (52, 9), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 2), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 8), (72, 1)]]


In [21]:
#to call an individual dictionary term switch value as appropriate
id2word[0]

'additive'

In [22]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('additive', 4),
  ('address', 1),
  ('animal', 5),
  ('approximately', 1),
  ('arise', 1),
  ('attributable', 1),
  ('behaviour', 4),
  ('commonly', 1),
  ('contribution', 1),
  ('contributor', 2),
  ('datum', 1),
  ('demonstrate', 1),
  ('difference', 6),
  ('different', 1),
  ('draw', 1),
  ('effect', 3),
  ('environmental', 2),
  ('epigenetic', 1),
  ('estimate', 4),
  ('evaluate', 1),
  ('even', 1),
  ('evolutionary', 1),
  ('example', 1),
  ('exhibit', 1),
  ('find', 1),
  ('frequently', 1),
  ('gambit', 1),
  ('general', 1),
  ('genetic', 5),
  ('genetically', 1),
  ('heritability', 6),
  ('high', 1),
  ('include', 2),
  ('individual', 2),
  ('inference', 1),
  ('influence', 1),
  ('issue', 1),
  ('justify', 1),
  ('know', 1),
  ('largely', 1),
  ('less', 1),
  ('likely', 1),
  ('low', 2),
  ('major', 1),
  ('member', 1),
  ('moderate', 1),
  ('much', 1),
  ('often', 2),
  ('overall', 1),
  ('parental', 1),
  ('pattern', 1),
  ('permanent', 1),
  ('personality', 9),
  ('phenot

In [23]:
###Loads of room for customization at this step. Read carefully and adjust accordingly.###

In [24]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=14, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [25]:
# Print the Keyword in the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.022*"vertebrate" + 0.020*"promoter" + 0.019*"absence" + 0.014*"start" + '
  '0.011*"tree" + 0.009*"forage" + 0.009*"wild" + 0.008*"group" + 0.008*"life" '
  '+ 0.008*"baboon"'),
 (1,
  '0.009*"coral" + 0.005*"grandmother" + 0.004*"ocean" + 0.004*"acute" + '
  '0.004*"mrcs" + 0.004*"germline" + 0.002*"housekeeping" + 0.002*"reef" + '
  '0.002*"enrichment" + 0.002*"acclimatize"'),
 (2,
  '0.036*"fluctuating_asymmetry" + 0.027*"allele" + 0.023*"part" + '
  '0.015*"embryonic" + 0.012*"imprint" + 0.012*"genetic" + 0.012*"global" + '
  '0.012*"undergo" + 0.012*"incorporate" + 0.011*"theory"'),
 (3,
  '0.091*"cell" + 0.021*"inheritance" + 0.018*"defense" + '
  '0.016*"multicellular" + 0.015*"argument" + 0.013*"heredity" + '
  '0.011*"primordial_germ" + 0.011*"evolutionary" + 0.010*"somatic" + '
  '0.010*"eiss"'),
 (4,
  '0.021*"program" + 0.016*"toad" + 0.007*"accommodation" + 0.007*"invasion" + '
  '0.006*"mass" + 0.005*"cane" + 0.004*"encounter" + 0.004*"avenue" + '
  '0.003*"disp

In [26]:
###Model perplexity and topic coherence provide a convenient measure to judge how good a given topic model is. 
###In my experience, topic coherence score, in particular, has been more helpful.

###Need to learn more about these values.

In [27]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.78047352578307

Coherence Score:  0.4299500496963463


In [28]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\brady\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:232: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10     0.300164 -0.080481       1        1  42.455907
9      0.252198 -0.096700       2        1  28.032221
8      0.147734  0.025985       3        1   7.057474
13     0.104123  0.326922       4        1   5.445719
2      0.019177 -0.047936       5        1   4.456586
6     -0.018458  0.014251       6        1   3.223000
11    -0.074938 -0.059415       7        1   2.247343
3     -0.081480 -0.045912       8        1   2.130698
12    -0.111754 -0.000260       9        1   1.484608
5     -0.126939  0.001036      10        1   1.413707
0     -0.115396 -0.004255      11        1   1.285610
7     -0.100307 -0.012225      12        1   0.370992
4     -0.099092 -0.010759      13        1   0.257328
1     -0.095031 -0.010252      14        1   0.138808, topic_info=               Term        Freq       Total Category  logprob  loglift
15           effect  215.000000  215.000000  Default  30.0000  30.0000
160          genome  162.000000  162.000000  Default  29.0000  29.0000
159            gene  372.000000  372.000000  Default  28.0000  28.0000
197           trait  147.000000  147.000000  Default  27.0000  27.0000
1219           cell   64.000000   64.000000  Default  26.0000  26.0000
...             ...         ...         ...      ...      ...      ...
159            gene    0.008569  372.420656  Topic14  -8.3543  -4.0998
16    environmental    0.008568   92.963584  Topic14  -8.3543  -2.7121
171     methylation    0.008567  255.067822  Topic14  -8.3544  -3.7215
206     acclimation    0.008567    6.191380  Topic14  -8.3544  -0.0031
1260   invertebrate    0.008567   11.265279  Topic14  -8.3544  -0.6017

[693 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
73        1  0.236095    ability
73        2  0.196746    ability
73        4  0.550889    ability
124       4  0.214434    absence
124      11  0.750518    absence
...     ...       ...        ...
919      11  0.176517       wild
1841      4  0.968188       wing
2086      2  0.972805      yeast
1574     12  0.586310       zone
4010      7  0.939936  zygomatic

[902 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 10, 9, 14, 3, 7, 12, 4, 13, 6, 1, 8, 5, 2])

In [29]:
###rename file.html before running command. Search computer and it should be in the user directory.
p = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
pyLDAvis.save_html(p, 'file.html')

C:\Users\brady\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:232: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  head(R).drop('saliency', 1)
